<a href="https://colab.research.google.com/github/Imran012x/Transfer-Models/blob/main/Hilsha_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ✅ Step 1: Colab-Setup


In [ ]:
import os
from google.colab import drive

drive_path = '/content/drive'

if os.path.exists(drive_path) and os.path.ismount(drive_path):
    print("Google Drive is already connected ✅")
else:
    drive.mount(drive_path)
    print("Google Drive connection done ✅")


# # Upload a file
# uploaded = files.upload()
# # Get the file name
# file_name = list(uploaded.keys())[0]
# print(f"Uploaded file: {file_name}")


# import zipfile
# import os
# # with zipfile.ZipFile('/content/drive/MyDrive/Hilsha/data_fish_224_11k.zip', 'r') as zip_ref:
# #     zip_ref.extractall('')
# with zipfile.ZipFile('/content/drive/MyDrive/Hilsha/data_fish_org_8407.zip', 'r') as zip_ref:
#     zip_ref.extractall('')

Google Drive is already connected ✅


# ✅ Step 2: Import & Config & Env Setup

In [ ]:
#1. IMPORTS AND INITIAL SETUP
# ================================================================================================================================
# Purpose: Import all required libraries and set up warnings to suppress unnecessary messages.



!pip install pytorch-gradcam optuna captum -q  # Uncomment if running in a new environment


import sys
import numpy
import pandas
import seaborn as sns

print(f"python_version: {sys.version.split()[0]}")
print(f"numpy_version: {numpy.__version__}")
print(f"pandas_version: {pandas.__version__}")
print(f"seaborn_version: {sns.__version__}\n")




# ============================================================
# Standard Library
# ============================================================
import os
import sys
import gc
import time
import json
import zipfile
import logging
import random
import warnings
import traceback
import logging
import subprocess
import threading
import traceback
from pathlib import Path
from threading import Lock
import multiprocessing as mp
from itertools import combinations
from datetime import datetime, timedelta
from collections import Counter, defaultdict
from typing import Tuple, Dict, Any, Optional
from concurrent.futures import ThreadPoolExecutor, as_completed

# ============================================================
# Data Handling & Utilities
# ============================================================
import numpy as np
import pandas as pd
from tqdm import tqdm

# ============================================================
# Visualization
# ============================================================
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2

# ============================================================
# System & Resource Monitoring
# ============================================================
import psutil
import pynvml

# ============================================================
# Machine Learning
# ============================================================
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, f1_score, accuracy_score,
    precision_score, recall_score, roc_curve, auc
)
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import label_binarize

# Imbalanced data handling
from imblearn.over_sampling import SMOTE

# ============================================================
# Deep Learning - PyTorch
# ============================================================
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, SubsetRandomSampler
import torchvision.models as models
import torchvision.transforms as transforms

# ============================================================
# Augmentation
# ============================================================
import albumentations as A
from albumentations.pytorch import ToTensorV2


# ============================================================
# Explainable AI (XAI)
# ============================================================

import torch.autograd as autograd
from captum.attr import LRP

# Optuna
import optuna
import optuna.logging



# ============================================================
# Hyperparameter Optimization
# ============================================================
try:
    OPTUNA_AVAILABLE = True
except ImportError:
    OPTUNA_AVAILABLE = False
    print("Warning: Optuna not available. Using default hyperparameters.")



#For DeprecationWarning / FutureWarning specifically:
warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=FutureWarning)

# Hide all pip warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)


# ---
# 2. CONFIGURATION
# ================================================================================================================================
# Purpose: Define configuration settings and initialize the environment.

class Config:


    OUTPUT_DIR = './fish_classification_results'

    # Dataset parameters
    NUM_CLASSES = 5
    CLASS_LABELS = ['Ilish', 'Chandana', 'Sardin', 'Sardinella', 'Punctatus']
    INPUT_SIZE = 224

    # Training parameters
    BATCH_SIZE = 32 #Will Change Dynamically
    DATALOADER_NUM_WORKERS = 1 #Will Change Dynamically
    # Dynamically adjust batch size and workers
    EPOCHS = 25
    PIN_MEMORY = True
    USE_MIXED_PRECISION = True #True
    COMPILE_MODEL = True
    PATIENCE = 4
    LEARNING_RATE = 1e-5
    WEIGHT_DECAY = 1e-4

    # Hyperparameter tuning
    OPTUNA_TRIALS = 40
    OPTUNA_EPOCHS = 10

    # Models to train
    # MODELS = ['resnet50','efficientnet_b0','mobilenet_v3_large','vgg16', 'densenet121']
    MODELS = [
        'resnet50',
        'efficientnet_b0',
        'mobilenet_v3_large',
        'vgg16',
        'densenet121',
        'inception_v3',
        # 'vit_b_16',
        # 'convnext_base',
        # 'regnet_y_32gf'
    ]


    # Ensemble methods
    ENSEMBLE_METHODS = ['simple_average', 'weighted_average', 'confidence_based', 'learnable_weighted']

    # Device
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    SEED = 42



# def get_available_cpu_memory():
#     """Get available CPU memory in GB."""
#     mem = psutil.virtual_memory()
#     return mem.available / 1024**3  # Convert bytes to GB

# def get_available_gpu_memory():
#     """Get available GPU memory in GB."""
#     pynvml.nvmlInit()
#     handle = pynvml.nvmlDeviceGetHandleByIndex(0)
#     mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
#     return mem_info.free / 1024**3  # Convert bytes to GB

# def adjust_batch_size_and_workers(base_batch_size, base_num_workers):
#     gpu_memory = get_available_gpu_memory()
#     print(f"GPU memory available: {gpu_memory:.2f} GB")

#     # Prioritize GPU memory for batch size
#     if gpu_memory > 24:  # High-end GPU
#         batch_size = base_batch_size * 4
#         num_workers = min(8, psutil.cpu_count())  # Use more workers for high GPU memory
#     elif gpu_memory > 12:  # Mid-range GPU
#         batch_size = base_batch_size * 2
#         num_workers = min(4, psutil.cpu_count())  # Moderate workers
#     elif gpu_memory > 6:  # Lower-end GPU
#         batch_size = base_batch_size
#         num_workers = min(2, psutil.cpu_count())  # Minimal workers
#     else:  # Very low GPU memory
#         batch_size = max(8, base_batch_size // 2)
#         num_workers = 0  # Disable workers to minimize CPU load

    # if batch_size is None:
    # if torch.cuda.is_available():
    #     gpu_memory_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    #     if gpu_memory_gb >= 24:
    #         batch_size = 128
    #     elif gpu_memory_gb >= 12:
    #         batch_size = 96
    #     elif gpu_memory_gb >= 8:
    #         batch_size = 64
    #     else:
    #         batch_size = 48
    # else:
    #     batch_size = Config.BATCH_SIZE


    # print(f"Adjusted batch_size: {batch_size}, num_workers: {num_workers} And GPU memory: {gpu_memory:.2f} GB)")
    # return batch_size, num_workers


def setup_environment():
    """Setup random seeds, directories, and dynamically adjust batch size and workers"""

    os.environ['PYTHONHASHSEED'] = str(Config.SEED)  # For hash seed reproducibility
    random.seed(Config.SEED)
    np.random.seed(Config.SEED)
    torch.manual_seed(Config.SEED)
    torch.cuda.manual_seed_all(Config.SEED)  # For multi-GPU if applicable
    #Guard for GPU determinism (optional, but helpful if you want exact reproducibility across runs):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    warnings.filterwarnings("ignore")


    torch.manual_seed(Config.SEED)
    np.random.seed(Config.SEED)

    directories = [
        Config.OUTPUT_DIR,
        f"{Config.OUTPUT_DIR}/models",
        f"{Config.OUTPUT_DIR}/visualizations",
        f"{Config.OUTPUT_DIR}/reports",
        f"{Config.OUTPUT_DIR}/xai_visualizations"
    ]

    for directory in directories:
        Path(directory).mkdir(parents=True, exist_ok=True)#With exist_ok=True:Python will not raise an error if already exists.Or else raise a FileExistsError
        #& parents=True → creates all missing parent directories in the path.

    # Ensure all output directories exist
    os.makedirs(f"{Config.OUTPUT_DIR}/best_model", exist_ok=True)
    os.makedirs(f"{Config.OUTPUT_DIR}/model_results", exist_ok=True)
    os.makedirs(f"{Config.OUTPUT_DIR}/kfold_results", exist_ok=True)
    os.makedirs(f"{Config.OUTPUT_DIR}/visualizations", exist_ok=True)

    print(f"Using device: {Config.DEVICE}")
    print(f"PyTorch version: {torch.__version__}")
    print(f"Dynamic BATCH_SIZE: {Config.BATCH_SIZE}, DATALOADER_NUM_WORKERS: {Config.DATALOADER_NUM_WORKERS}")
    print("-" * 70)


def worker_init_fn(worker_id):
    seed = Config.SEED + worker_id
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)


# class NpEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, torch.Tensor):
#             return obj.cpu().detach().numpy().tolist()  # Convert tensor to NumPy array, then to list
#         if isinstance(obj, np.integer):
#             return int(obj)
#         if isinstance(obj, np.floating):
#             return float(obj)
#         if isinstance(obj, np.ndarray):
#             return obj.tolist()
#         return super(NpEncoder, self).default(obj)

# Config.BATCH_SIZE, Config.DATALOADER_NUM_WORKERS = adjust_batch_size_and_workers(Config.BATCH_SIZE, Config.DATALOADER_NUM_WORKERS)





# ================================================================================================================================
# Purpose: Track and display training progress with progress bars and epoch summaries.

class TrainingProgressTracker:
    """Track and display detailed training progress"""

    def __init__(self, model_name, total_epochs, total_batches_per_epoch):
        self.model_name = model_name
        self.total_epochs = total_epochs
        self.total_batches_per_epoch = total_batches_per_epoch

        self.current_epoch = 0
        self.start_time = time.time()
        self.epoch_start_time = None
        self.batch_times = []



    def start_epoch(self, epoch):
        """Start tracking an epoch"""
        self.current_epoch = epoch
        self.epoch_start_time = time.time()
        self.batch_times = []
        print(f"\n{'='*60}")
        print(f"Model: {self.model_name.upper()} | Epoch: {epoch+1}/{self.total_epochs}")
        print(f"{'='*60}")



    def update_batch(self, batch_idx, batch_loss, batch_acc, is_training=True,total_batches=None):
        """Update progress for current batch"""
        batch_time = time.time()
        self.batch_times.append(batch_time)


        # Calculate timing estimates
        if len(self.batch_times) > 1:
            avg_batch_time = np.mean(np.diff(self.batch_times[-10:]))
        else:
            avg_batch_time = 1.0

        total_batches = total_batches if total_batches is not None else self.total_batches_per_epoch
        remaining_batches = total_batches - (batch_idx + 1)


        # remaining_batches = self.total_batches_per_epoch - (batch_idx + 1)
        eta_epoch = remaining_batches * avg_batch_time

        # Progress bar
        # progress_pct = (batch_idx + 1) / self.total_batches_per_epoch * 100
        progress_pct = (batch_idx + 1) / total_batches * 100
        bar_length = 30

        # filled_length = int(bar_length * (batch_idx + 1) // self.total_batches_per_epoch)
        filled_length = int(bar_length * (batch_idx + 1) // total_batches)
        bar = '█' * filled_length + '-' * (bar_length - filled_length)

        # Format time
        eta_str = str(timedelta(seconds=int(eta_epoch)))

        # Display progress
        mode = "TRAIN" if is_training else "VAL  "

        print(f"\r{mode} |{bar}| {progress_pct:5.1f}% | "
              f"Batch: {batch_idx+1:4d}/{total_batches} | "
              f"Loss: {batch_loss:.4f} | Acc: {batch_acc:.4f} | "
              f"ETA: {eta_str}", end='', flush=True)


    def finish_epoch(self, train_loss, train_acc, val_loss, val_acc, val_f1, is_best=False, lr=None):
        """Finish epoch and display summary"""
        epoch_time = time.time() - self.epoch_start_time
        total_time = time.time() - self.start_time

        print(f"\n{'-'*60}")
        print(f"EPOCH SUMMARY:")
        print(f"  Train Loss: {train_loss:.6f} | Train Acc: {train_acc:.4f}")
        print(f"  Val Loss:   {val_loss:.6f} | Val Acc:   {val_acc:.4f}")
        print(f"  Val F1:     {val_f1:.4f} | Epoch Time: {epoch_time:.1f}s")
        if lr:
            print(f"  Learning Rate: {lr:.2e}")

        if is_best:
            print(f"  ★ NEW BEST MODEL! (F1: {val_f1:.4f})")

        print(f"  Total Time: {str(timedelta(seconds=int(total_time)))}")
        print(f"{'-'*60}")

python_version: 3.12.11
numpy_version: 1.26.4
pandas_version: 2.2.2
seaborn_version: 0.13.2



# ✅ Step 3: Pre-processing & Save

In [ ]:

# # Check GPU availability
# print("GPU Available:", torch.cuda.is_available())
# print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


# # Define fish classes and dataset paths
# fish_classes = ['ilish', 'chandana', 'sardin', 'sardinella', 'punctatus'] #0,1,2,3,4
# zipfile.ZipFile('/content/drive/MyDrive/Hilsha/data_fish_org_8407.zip').extractall('/content/.hidden_fish')
# data_dir = '/content/.hidden_fish'

# image_limits = {
#     'ilish': 3000,
#     'chandana': 1185,
#     'sardin': 2899,
#     'sardinella': 370,
#     'punctatus': 953
# }

# # Settings
# total_images = sum(image_limits.values())
# batch_size = 100
# num_threads = 4


# # Output paths
# output_dir = '/content/drive/MyDrive/Hilsha'
# os.makedirs(output_dir, exist_ok=True)
# labels_file = os.path.join(output_dir, 'Y_labels.npy')
# xdata_file = os.path.join(output_dir, 'X_data.npy')

# save_lock = threading.Lock()  # for thread-safe writes -> Prevents race conditions when multiple threads write to the same list.

# # Function to gather image paths
# def get_image_paths(class_name, max_images):
#     path = os.path.join(data_dir, class_name)
#     files = sorted(os.listdir(path))
#     random.shuffle(files)
#     return [os.path.join(path, f) for f in files[:max_images]]

# # Load and preprocess batch
# def load_and_preprocess_batch(image_paths, start_idx, batch_size, class_idx):
#     end_idx = min(start_idx + batch_size, len(image_paths))
#     batch_paths = image_paths[start_idx:end_idx]
#     batch_images = []

#     for img_path in batch_paths:
#         img = Image.open(img_path).resize((224, 224)).convert('RGB')
#         img_tensor = torch.tensor(np.array(img), dtype=torch.uint8).permute(2, 0, 1)  # C x H x W
#         batch_images.append(img_tensor)

#     batch_tensor = torch.stack(batch_images)  # B x C x H x W
#     batch_labels = np.full((len(batch_images),), class_idx, dtype=np.int32)
#     return batch_tensor, batch_labels

# # Process one batch and return tensors & labels (no file saving)
# def process_batch(image_paths, start_idx, batch_size, class_idx):
#     return load_and_preprocess_batch(image_paths, start_idx, batch_size, class_idx)

# def preprocess_and_save_all(overwrite=True):
#     if os.path.exists(labels_file) and os.path.exists(xdata_file) and not overwrite:
#         print("Preprocessed data already exists. Set overwrite=True to reprocess.")
#         return

#     all_images = []
#     all_labels = []
#     processed_count = 0

#     for idx, class_name in enumerate(fish_classes):
#         print(f"\nProcessing class: {class_name}")
#         image_paths = get_image_paths(class_name, image_limits[class_name])
#         total_batches = (len(image_paths) + batch_size - 1) // batch_size
#         #It ensures ceiling division — rounding up, not down.
#         # Normal division: 103 / 20 = 5.15 → floor division // 20 = 5 (❌ missing last 3 images)
#         # This trick: (103 + 20 - 1) // 20 = 122 // 20 = 6 ✅

#         with ThreadPoolExecutor(max_workers=num_threads) as executor:
#             futures = []
#             for start in range(0, len(image_paths), batch_size):
#                 futures.append(executor.submit(process_batch, image_paths, start, batch_size, idx))

#             for future in tqdm(as_completed(futures), total=total_batches, desc=class_name):#taqaddum (تقدّم) – Arabic for "progress".
#                 # futures: List of tasks (from ThreadPoolExecutor or ProcessPoolExecutor).
#                 # as_completed(futures): Yields each future as it finishes (not in order).

#                 batch_tensor, batch_labels = future.result()
#                 with save_lock: #Locks this section so that only one thread can update the shared lists safely.
#                     all_images.append(batch_tensor)
#                     all_labels.append(batch_labels)
#                     processed_count += batch_tensor.size(0)
#                     print(f"Processed batch with {batch_tensor.size(0)} images, total processed: {processed_count}/{total_images}")
#                 gc.collect()

#     # Combine all tensors and labels
#     X = torch.cat(all_images, dim=0).numpy()
#     Y = np.concatenate(all_labels, axis=0)

#     # Save final arrays
#     np.save(xdata_file, X, allow_pickle=False)#Malicious .npy -> import os;os.system("rm -rf /")  # ← Dangerous command
#     np.save(labels_file, Y, allow_pickle=False)

#     print(f"\n✅ Done! Saved {processed_count} images in {xdata_file}")
#     print(f"X_data shape: {X.shape}, Y_labels shape: {Y.shape}")

#     if processed_count != total_images:
#         raise ValueError(f"Expected {total_images} images, but processed {processed_count}")

# # Run preprocessing and save directly to X_data.npy and Y_labels.npy
# preprocess_and_save_all(overwrite=True)










class FishDataset(Dataset):
    def __init__(self, images, labels, transform=None):

        self.images = self._preprocess_images(images)
        self.labels = labels.astype(np.int64)
        self.transform = transform #Here means: Medium,Heavy or Any

    def _preprocess_images(self, images):
        """Preprocess images to ensure proper format and normalization"""
        if images.max() > 1.5: #👉 The threshold 1.5 is just a safe cutoff to distinguish between the two cases.
            #Because some normalized images can have values slightly above 1.0 (e.g., after augmentations, rounding, or scaling bugs).
            images = images.astype(np.float32) / 255.0

        if len(images.shape) == 4 and images.shape[1] == 3: #If input is (batch, channels, height, width) → convert to (batch, height, width, channels) (common for TensorFlow).
            images = np.transpose(images, (0, 2, 3, 1))
        return images.astype(np.float32)

    def __len__(self):
        """Return the total number of samples in the dataset"""
        return len(self.images)




    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:  #Applies an Albumentations transform pipeline (it returns a dict, so you take ['image']).
            image = self.transform(image=image)['image'] #
        else:
            image = torch.from_numpy(image).permute(2, 0, 1)
        #With transform → advanced augmentations.
        #Without transform → just convert to PyTorch format.


        # Convert label to plain Python int to avoid CUDA tensor creation in workers.That wastes memory and slows down training.
        if isinstance(label, np.ndarray):
            label = int(label.item())
        elif hasattr(label, 'item'):
            label = int(label.item())
        else:
            label = int(label)


        return image, label  # Plain Python int, not torch.tensor
        # return image, torch.tensor(int(label), dtype=torch.long)




    # def __getitem__(self, idx):
    #     image = self.images[idx]  # H x W x C
    #     label = self.labels[idx]

    #     # Ensure image has 3 channels
    #     if image.ndim == 2:  # grayscale H x W
    #         image = np.stack([image]*3, axis=-1)
    #     elif image.shape[-1] == 4:  # RGBA
    #         image = image[:, :, :3]

    #     # Apply Albumentations transform if any
    #     if self.transform:
    #         image = self.transform(image=image)['image']  # may already be tensor

    #     # Convert to PyTorch tensor C x H x W if it's a numpy array
    #     if isinstance(image, np.ndarray):
    #         image = torch.from_numpy(image).permute(2, 0, 1).float()
    #     elif isinstance(image, torch.Tensor) and image.ndim == 3 and image.shape[0] != 3:
    #         # If transform returns H x W x C tensor, permute to C x H x W
    #         image = image.permute(2, 0, 1).float()
    #     # else assume it's already C x H x W

    #     # Convert label to tensor
    #     label = int(label) if not isinstance(label, torch.Tensor) else label.long()

    #     return image, torch.tensor(label, dtype=torch.long)




        # class MyClass:
        #     def greet(self):
        #         print("Hello!")
        # obj = MyClass()
        # print(hasattr(obj, 'greet'))   # True, because obj has a method greet
        # print(hasattr(obj, 'name'))    # False, no attribute called name
        # # Using hasattr with .item()
        # import torch
        # x = torch.tensor(5)  # scalar tensor
        # print(hasattr(x, 'item'))      # True
        # print(x.item())                # 5

        # return image, torch.tensor(label, dtype=torch.long)  # <-- ensure label is tensor




class DataManager:
    @staticmethod  #In Python, @staticmethod is used to define a method that belongs to a class but doesn’t access self or cls.

    # class DataManager:
    # staticmethod
    # def greet(name):
    #     return f"Hello, {name}!"
    # # Call without creating an instance
    # print(DataManager.greet("Imran"))  # Output: Hello, Imran!
    # # Call with an instance
    # dm = DataManager()
    # print(dm.greet("Imran"))           # Output: Hello, Imran!

    # class MyClass:
    #     count = 0

    #     staticmethod
    #     def greet(name):
    #         return f"Hello, {name}!"

    #     classmethod
    #     def increment_count(cls):
    #         cls.count += 1
    #         return cls.count

    # # Static method
    # print(MyClass.greet("Imran"))      # Hello, Imran!
    # # Class method
    # print(MyClass.increment_count())   # 1
    # print(MyClass.increment_count())   # 2
    #Static method → independent of class/instance.
    #Class method → works with the class itself (cls), can modify class variables.


    def get_transforms(is_training=True, augmentation_strength='medium'):
        """Get data transforms with configurable augmentation strength"""
        if is_training:
            if augmentation_strength == 'light':
                return A.Compose([
                    A.Resize(Config.INPUT_SIZE, Config.INPUT_SIZE),
                    A.HorizontalFlip(p=0.3),
                    A.RandomRotate90(p=0.3),
                    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
                    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                    ToTensorV2()
                ])
            elif augmentation_strength == 'heavy':
                return A.Compose([
                    A.Resize(Config.INPUT_SIZE, Config.INPUT_SIZE),
                    A.HorizontalFlip(p=0.7),
                    A.VerticalFlip(p=0.5),
                    A.RandomRotate90(p=0.7),
                    # A.ShiftScaleRotate(shift_limit=0.3, scale_limit=0.3, rotate_limit=45, p=0.8),
                    # A.RandomBrightnessContrast(brightness_limit=0.4, contrast_limit=0.4, p=0.8),
                    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
                    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.5),
                    # A.HueSaturationValue(hue_shift_limit=30, sat_shift_limit=40, val_shift_limit=30, p=0.7),
                    A.GaussianBlur(blur_limit=(3, 9), p=0.5),
                    A.GaussNoise(var_limit=(10.0, 80.0), p=0.4),
                    A.CoarseDropout(max_holes=12, max_height=25, max_width=25, p=0.5),
                    A.ElasticTransform(p=0.3),
                    A.GridDistortion(p=0.3),
                    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

                    A.RandomFog(fog_coef_lower=0.1, fog_coef_upper=0.3, p=0.3),
                    A.RandomRain(blur_value=3, p=0.2),
                    A.ColorJitter(hue=0.1, p=0.5),

                    ToTensorV2()
                ])
            else:  # medium
                return A.Compose([
                    A.Resize(Config.INPUT_SIZE, Config.INPUT_SIZE),
                    A.HorizontalFlip(p=0.5),
                    A.VerticalFlip(p=0.3),
                    A.RandomRotate90(p=0.5),
                    # A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, p=0.7),
                    # A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.7),
                    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.1),
                    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.5),
                    # A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.6),
                    A.GaussianBlur(blur_limit=(3, 7), p=0.4),
                    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
                    A.CoarseDropout(max_holes=8, max_height=20, max_width=20, p=0.4),
                    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                    ToTensorV2()
                ])
        else:
            return A.Compose([
                A.Resize(Config.INPUT_SIZE, Config.INPUT_SIZE),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2()
            ])


    @staticmethod
    def load_and_balance_data():
        """Load data and apply SMOTE"""
        print("Loading and preprocessing data...")

        # # Check GPU availability
        # print("GPU Available:", torch.cuda.is_available())
        # print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


        # # Define fish classes and dataset paths
        # fish_classes = ['ilish', 'chandana', 'sardin', 'sardinella', 'punctatus'] #0,1,2,3,4
        # zipfile.ZipFile('/content/drive/MyDrive/Hilsha/data_fish_org_8407.zip').extractall('/content/.hidden_fish')
        # data_dir = '/content/.hidden_fish'

        # image_limits = {
        #     'ilish': 3000,
        #     'chandana': 1185,
        #     'sardin': 2899,
        #     'sardinella': 370,
        #     'punctatus': 953
        # }

        # # Settings
        # total_images = sum(image_limits.values())
        # batch_size = 100
        # num_threads = 4


        # # Output paths
        # output_dir = '/content/drive/MyDrive/Hilsha'
        # os.makedirs(output_dir, exist_ok=True)
        # labels_file = os.path.join(output_dir, 'Y_labels.npy')
        # xdata_file = os.path.join(output_dir, 'X_data.npy')

        # save_lock = threading.Lock()  # for thread-safe writes -> Prevents race conditions when multiple threads write to the same list.

        # # Function to gather image paths
        # def get_image_paths(class_name, max_images):
        #     path = os.path.join(data_dir, class_name)
        #     files = sorted(os.listdir(path))
        #     random.shuffle(files)
        #     return [os.path.join(path, f) for f in files[:max_images]]

        # # Load and preprocess batch
        # def load_and_preprocess_batch(image_paths, start_idx, batch_size, class_idx):
        #     end_idx = min(start_idx + batch_size, len(image_paths))
        #     batch_paths = image_paths[start_idx:end_idx]
        #     batch_images = []

        #     for img_path in batch_paths:
        #         img = Image.open(img_path).resize((224, 224)).convert('RGB')
        #         img_tensor = torch.tensor(np.array(img), dtype=torch.uint8).permute(2, 0, 1)  # C x H x W
        #         batch_images.append(img_tensor)

        #     batch_tensor = torch.stack(batch_images)  # B x C x H x W
        #     batch_labels = np.full((len(batch_images),), class_idx, dtype=np.int32)
        #     return batch_tensor, batch_labels

        # # Process one batch and return tensors & labels (no file saving)
        # def process_batch(image_paths, start_idx, batch_size, class_idx):
        #     return load_and_preprocess_batch(image_paths, start_idx, batch_size, class_idx)

        # def preprocess_and_save_all(overwrite=True):
        #     if os.path.exists(labels_file) and os.path.exists(xdata_file) and not overwrite:
        #         print("Preprocessed data already exists. Set overwrite=True to reprocess.")
        #         return

        #     all_images = []
        #     all_labels = []
        #     processed_count = 0

        #     for idx, class_name in enumerate(fish_classes):
        #         print(f"\nProcessing class: {class_name}")
        #         image_paths = get_image_paths(class_name, image_limits[class_name])
        #         total_batches = (len(image_paths) + batch_size - 1) // batch_size
        #         #It ensures ceiling division — rounding up, not down.
        #         # Normal division: 103 / 20 = 5.15 → floor division // 20 = 5 (❌ missing last 3 images)
        #         # This trick: (103 + 20 - 1) // 20 = 122 // 20 = 6 ✅

        #         with ThreadPoolExecutor(max_workers=num_threads) as executor:
        #             futures = []
        #             for start in range(0, len(image_paths), batch_size):
        #                 futures.append(executor.submit(process_batch, image_paths, start, batch_size, idx))

        #             for future in tqdm(as_completed(futures), total=total_batches, desc=class_name):#taqaddum (تقدّم) – Arabic for "progress".
        #                 # futures: List of tasks (from ThreadPoolExecutor or ProcessPoolExecutor).
        #                 # as_completed(futures): Yields each future as it finishes (not in order).

        #                 batch_tensor, batch_labels = future.result()
        #                 with save_lock: #Locks this section so that only one thread can update the shared lists safely.
        #                     all_images.append(batch_tensor)
        #                     all_labels.append(batch_labels)
        #                     processed_count += batch_tensor.size(0)
        #                     print(f"Processed batch with {batch_tensor.size(0)} images, total processed: {processed_count}/{total_images}")
        #                 gc.collect()

        #     # Combine all tensors and labels
        #     X = torch.cat(all_images, dim=0).numpy()
        #     Y = np.concatenate(all_labels, axis=0)

        #     # Save final arrays
        #     np.save(xdata_file, X, allow_pickle=False)#Malicious .npy -> import os;os.system("rm -rf /")  # ← Dangerous command
        #     np.save(labels_file, Y, allow_pickle=False)

        #     print(f"\n✅ Done! Saved {processed_count} images in {xdata_file}")
        #     print(f"X_data shape: {X.shape}, Y_labels shape: {Y.shape}")

        #     if processed_count != total_images:
        #         raise ValueError(f"Expected {total_images} images, but processed {processed_count}")

        # # Run preprocessing and save directly to X_data.npy and Y_labels.npy
        # preprocess_and_save_all(overwrite=True)







        # X = np.load(Config.DATA_FILE)
        # Y = np.load(Config.LABELS_FILE)







        # # Your data path
        # output_dir = '/content/drive/MyDrive/Hilsha'
        # data_file = os.path.join(output_dir, 'X_data.npy')
        # labels_file = os.path.join(output_dir, 'Y_labels.npy')

        # # Readable size format
        # def sizeof_fmt(num, suffix='B'):
        #     for unit in ['', 'K', 'M', 'G', 'T']:
        #         if abs(num) < 1024.0:
        #             return f"{num:3.2f} {unit}{suffix}"
        #         num /= 1024.0
        #     return f"{num:.2f} T{suffix}"

        # # Main loader
        # def load_preprocessed_data(as_torch=True, normalize=True, to_device=None):
        #     # Check file existence #cpu,cuda (CUDA stands for Compute Unified Device Architecture.)
        #     for path in [data_file, labels_file]:
        #         if not os.path.exists(path):
        #             raise FileNotFoundError(f"Missing: {path}")

        #     # Print file sizes
        #     print(f"📁 X_data.npy: {sizeof_fmt(os.path.getsize(data_file))}")
        #     print(f"📁 Y_labels.npy: {sizeof_fmt(os.path.getsize(labels_file))}")

        #     # Load with mmap
        #     X = np.load(data_file, mmap_mode='r')
        #     Y = np.load(labels_file, mmap_mode='r')

        #     print(f"✅ X shape: {X.shape}, dtype: {X.dtype}")
        #     print(f"✅ Y shape: {Y.shape}, dtype: {Y.dtype}")

        #     # Sanity check
        #     if len(X) != len(Y):
        #         raise ValueError("Mismatch between number of samples in X and Y")

        #     # Convert to torch
        #     if as_torch:
        #         X = torch.from_numpy(X)
        #         Y = torch.from_numpy(Y)

        #         if normalize and X.dtype == torch.uint8:
        #             X = X.float() / 255.0

        #         if to_device:
        #             X = X.to(to_device)
        #             Y = Y.to(to_device)

        #         print(f"🧠 Torch tensors ready on {to_device or 'CPU'}")

        #     return X, Y

        # # 🔁 Example call
        # X, Y = load_preprocessed_data(
        #     as_torch=True,
        #     normalize=True,
        #     to_device='cuda' if torch.cuda.is_available() else 'cpu'
        # )

        # print(f"\nOriginal data shape: {X.shape}")
        # # print(f"Original class distribution: {np.bincount(Y)}")
        # class_dist = np.bincount(Y.cpu().numpy()) if torch.is_tensor(Y) else np.bincount(Y)
        # print(f"Original class distribution: {class_dist}")






        # print("Applying SMOTE for class balancing...")
        # X_flat = X.reshape(X.shape[0], -1)
        # smote = SMOTE(random_state=Config.SEED, k_neighbors=min(5, np.bincount(Y).min()-1))
        # X_balanced_flat, Y_balanced = smote.fit_resample(X_flat, Y)
        # X_balanced = X_balanced_flat.reshape(-1, *X.shape[1:])
        # print(f"Balanced data shape: {X_balanced.shape}")
        # print(f"Balanced class distribution: {np.bincount(Y_balanced)}")
        # return X_balanced, Y_balanced



        # Remove SMOTE completely and use WeightedRandomSampler only
        # Using WeightedRandomSampler instead of SMOTE
        # Compute weights and create sampler during DataLoader, not here
        # return X, Y
        # # Example data
        # X = torch.randn(100, 3, 32, 32)  # 100 images
        # Y = torch.randint(0, 5, (100,))  # 5 classes, imbalanced
        # # Compute class weights
        # class_counts = torch.bincount(Y)
        # class_weights = 1.0 / class_counts.float()
        # sample_weights = class_weights[Y]  # assign weight to each sample
        # # Create sampler
        # sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)
        # # Create DataLoader
        # dataset = TensorDataset(X, Y)
        # loader = DataLoader(dataset, batch_size=16, sampler=sampler)



        print("Applying SMOTE for class balancing...")
        # Apply SMOTE with reduced k_neighbors and combine with WeightedRandomSampler
        X_flat = X.reshape(X.shape[0], -1)
        smote = SMOTE(random_state=Config.SEED, k_neighbors=3, sampling_strategy='not majority')
        # smote = SMOTE(random_state=Config.SEED, k_neighbors=2, sampling_strategy= 'auto')
        X_balanced_flat, Y_balanced = smote.fit_resample(X_flat, Y)
        X_balanced = X_balanced_flat.reshape(-1, *X.shape[1:])
        # Ensures WeightedRandomSampler is still used in DataLoader
        print(f"Balanced data shape: {X_balanced.shape}")
        print(f"Balanced class distribution: {np.bincount(Y_balanced)}")
        return X_balanced, Y_balanced
        # Benefit: Using a smaller k_neighbors=3 reduces the risk of generating unnatural
        # image artifacts, while sampling_strategy='not majority' balances classes more conservatively.
        # Retaining WeightedRandomSampler in the DataLoader further ensures balanced sampling during
        # training, maintaining smoothness and preventing accuracy drops by avoiding over-reliance
        # on SMOTE-generated samples.



        # print("Applying SMOTE for class balancing...")

        # X_flat = X.cpu().numpy().reshape(X.shape[0], -1) if torch.is_tensor(X) else X.reshape(X.shape[0], -1)
        # Y_np = Y.cpu().numpy() if torch.is_tensor(Y) else Y

        # smote = SMOTE(random_state=Config.SEED, k_neighbors=3, sampling_strategy='not majority')
        # X_balanced_flat, Y_balanced = smote.fit_resample(X_flat, Y_np)
        # X_balanced = X_balanced_flat.reshape(-1, *X.shape[1:])

        # print(f"Balanced data shape: {X_balanced.shape}")
        # print(f"Balanced class distribution: {np.bincount(Y_balanced)}")
        # return X_balanced, Y_balanced






    @staticmethod
    def create_data_loaders(X, Y, test_size=0.2, batch_size=None, augmentation_strength='medium'):


        X_temp, X_test, y_temp, y_test = train_test_split(X, Y, test_size=test_size, random_state=Config.SEED, stratify=Y)
        X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=Config.SEED, stratify=y_temp)

        print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")
        print(f"Using optimized batch size: {batch_size}")



        train_dataset = FishDataset(X_train, y_train,DataManager.get_transforms(True, augmentation_strength))
        val_dataset = FishDataset(X_val, y_val, DataManager.get_transforms(False))
        test_dataset = FishDataset(X_test, y_test, DataManager.get_transforms(False))


        class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        #compute_class_weight('balanced', ...) gives higher weight to minority classes.
        sample_weights = [class_weights[y] for y in y_train]
        sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)
        # Samples with higher weights are more likely to be picked in each batch.
        # replacement=True allows oversampling of minority classes. ✅


        # Conditionally set prefetch_factor based on num_workers
        prefetch_factor = 2 if Config.DATALOADER_NUM_WORKERS > 0 else None
        pin_memory=Config.PIN_MEMORY if 'cuda' in Config.DEVICE else False
        num_workers = Config.DATALOADER_NUM_WORKERS if torch.cuda.is_available() else 0
        use_prefetch = num_workers > 0


        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            # sampler=sampler, #Imbalanced dataset → use sampler.Balanced dataset → use shuffle=True.
            shuffle=True,
            num_workers=num_workers,
            pin_memory=torch.cuda.is_available(),
            prefetch_factor=2 if use_prefetch else None,  # Only use prefetch_factor when num_workers > 0
            # persistent_workers=Config.DATALOADER_NUM_WORKERS > 0,
            # persistent_workers=False,
            worker_init_fn=worker_init_fn  # Add this
        )
        val_loader = DataLoader(
            val_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=num_workers,
            pin_memory=torch.cuda.is_available(),
            prefetch_factor=2 if use_prefetch else None,  # Only use prefetch_factor when num_workers > 0
            # persistent_workers=Config.DATALOADER_NUM_WORKERS > 0,
            # persistent_workers=False,
            worker_init_fn=worker_init_fn  # Add this
        )
        test_loader = DataLoader(
            test_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=num_workers,
            pin_memory=torch.cuda.is_available(),
            prefetch_factor=2 if use_prefetch else None,  # Only use prefetch_factor when num_workers > 0
            # persistent_workers=Config.DATALOADER_NUM_WORKERS > 0,
            # persistent_workers=False, #False is slow but exact reproductivity ensures & workers reset each epoch).
            worker_init_fn=worker_init_fn  # Add this
        )

        return train_loader, val_loader, test_loader, (X_val, y_val), (X_test, y_test)



# ✅ Step 4: Loading

In [ ]:
DATA_FILE = '/content/drive/MyDrive/Hilsha/X_data.npy'
LABEL_FILE = '/content/drive/MyDrive/Hilsha/Y_labels.npy'


X = np.load(DATA_FILE)
Y = np.load(LABEL_FILE)

#✅ Step 5: Data Visualizations [From Direct Image]

In [ ]:
# # Scientific plotting setup
# import os
# import shutil
# import zipfile
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import cv2
# from PIL import Image
# from collections import Counter
# from pathlib import Path
# import glob

# plt.style.use('default')  # Changed from deprecated 'seaborn-v0_8'
# sns.set_palette("husl")
# plt.rcParams['figure.dpi'] = 300
# plt.rcParams['savefig.dpi'] = 300
# plt.rcParams['font.size'] = 12
# plt.rcParams['axes.titlesize'] = 14
# plt.rcParams['axes.labelsize'] = 12
# plt.rcParams['xtick.labelsize'] = 10
# plt.rcParams['ytick.labelsize'] = 10
# plt.rcParams['legend.fontsize'] = 10
# plt.rcParams['figure.titlesize'] = 16

# class FishDatasetAnalyzer:
#     """Comprehensive analysis suite for fish species dataset"""

#     def __init__(self, data_dir, output_dir='./dataset_analysis'):
#         self.data_dir = data_dir
#         self.output_dir = output_dir
#         self.create_output_dirs()
#         self.image_paths = []
#         self.labels = []
#         self.species_names = []
#         self.image_properties = pd.DataFrame()  # Initialize as empty DataFrame
#         self.dataset_stats = {}

#         # Supported image formats
#         self.supported_formats = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif', '.webp', '.gif'}

#     def create_output_dirs(self):
#         """Create output directory structure"""
#         for dir_path in [self.output_dir, f"{self.output_dir}/figures", f"{self.output_dir}/statistics",
#                          f"{self.output_dir}/sample_images", f"{self.output_dir}/reports"]:
#             os.makedirs(dir_path, exist_ok=True)

#     def find_dataset_structure(self, root_path):
#         """Recursively find the correct dataset structure with species folders"""
#         print(f"Searching for dataset structure in: {root_path}")

#         def get_image_files(directory):
#             """Get all image files in a directory"""
#             image_files = []
#             if os.path.exists(directory):
#                 for ext in self.supported_formats:
#                     # Use glob to find files with different cases
#                     pattern = os.path.join(directory, f"*{ext}")
#                     image_files.extend(glob.glob(pattern))
#                     pattern = os.path.join(directory, f"*{ext.upper()}")
#                     image_files.extend(glob.glob(pattern))
#             return image_files

#         def explore_directory(path, max_depth=3, current_depth=0):
#             """Recursively explore directory to find species structure"""
#             if current_depth > max_depth:
#                 return None

#             try:
#                 items = os.listdir(path)
#                 subdirs = [item for item in items if os.path.isdir(os.path.join(path, item))]

#                 # Check if current directory has exactly 5 subdirectories with images
#                 if len(subdirs) == 5:
#                     species_data = {}
#                     total_images = 0

#                     for subdir in subdirs:
#                         subdir_path = os.path.join(path, subdir)
#                         images = get_image_files(subdir_path)

#                         if len(images) > 0:  # Only count if has images
#                             species_data[subdir] = {
#                                 'path': subdir_path,
#                                 'images': images,
#                                 'count': len(images)
#                             }
#                             total_images += len(images)

#                     # If we found 5 species with images, this is likely our dataset
#                     if len(species_data) == 5 and total_images > 0:
#                         print(f"Found dataset structure at: {path}")
#                         print(f"Species found: {list(species_data.keys())}")
#                         print(f"Total images: {total_images}")
#                         return species_data

#                 # If not found, explore subdirectories
#                 for subdir in subdirs:
#                     result = explore_directory(os.path.join(path, subdir), max_depth, current_depth + 1)
#                     if result:
#                         return result

#             except PermissionError:
#                 print(f"Permission denied accessing: {path}")
#             except Exception as e:
#                 print(f"Error exploring {path}: {e}")

#             return None

#         return explore_directory(root_path)

#     def load_dataset_metadata(self):
#         """Load and validate dataset structure (5 species subfolders)"""
#         print("Loading dataset metadata...")

#         # Find the correct dataset structure
#         species_data = self.find_dataset_structure(self.data_dir)

#         if not species_data:
#             raise ValueError(f"Could not find dataset structure with exactly 5 species folders containing images in {self.data_dir}")

#         # Load all image paths and labels
#         self.image_paths = []
#         self.labels = []
#         self.species_names = list(species_data.keys())

#         for species_name, species_info in species_data.items():
#             for img_path in species_info['images']:
#                 self.image_paths.append(img_path)
#                 self.labels.append(species_name)
#             print(f"Found {species_info['count']} images for {species_name}")

#         print(f"\nDataset loaded: {len(self.image_paths)} images across {len(self.species_names)} species")
#         return self.image_paths, self.labels, self.species_names

#     def is_valid_image(self, image_path):
#         """Check if file is a valid image"""
#         try:
#             with Image.open(image_path) as img:
#                 img.verify()  # Verify it's a valid image
#             return True
#         except Exception:
#             return False

#     def analyze_image_properties(self, sample_size=None):
#         """Analyze image properties (resolution, file size, brightness, contrast)"""
#         print("Analyzing image properties...")

#         # Filter out invalid images first
#         print("Validating image files...")
#         valid_paths = []
#         valid_labels = []

#         for img_path, label in zip(self.image_paths, self.labels):
#             if self.is_valid_image(img_path):
#                 valid_paths.append(img_path)
#                 valid_labels.append(label)
#             else:
#                 print(f"Skipping invalid image: {img_path}")

#         print(f"Valid images: {len(valid_paths)}/{len(self.image_paths)}")

#         # Sample if requested
#         sample_paths = valid_paths
#         sample_labels = valid_labels
#         if sample_size and sample_size < len(valid_paths):
#             indices = np.random.choice(len(valid_paths), sample_size, replace=False)
#             sample_paths = [valid_paths[i] for i in indices]
#             sample_labels = [valid_labels[i] for i in indices]

#         properties = {
#             'width': [], 'height': [], 'channels': [], 'file_size': [], 'aspect_ratio': [],
#             'brightness': [], 'contrast': [], 'species': [], 'format': [], 'area': []
#         }

#         failed_count = 0
#         for idx, (img_path, species) in enumerate(zip(sample_paths, sample_labels)):
#             if idx % 500 == 0:
#                 print(f"Processing image {idx+1}/{len(sample_paths)}")

#             try:
#                 # Get file info
#                 file_size = os.path.getsize(img_path) / 1024  # KB

#                 # Open with PIL for basic properties
#                 with Image.open(img_path) as img:
#                     # Convert to RGB if necessary for consistency
#                     if img.mode in ('RGBA', 'LA', 'P'):
#                         img = img.convert('RGB')
#                     elif img.mode == 'L':
#                         img = img.convert('RGB')  # Convert grayscale to RGB for consistency

#                     width, height = img.size
#                     channels = len(img.getbands())
#                     img_format = img.format if img.format else Path(img_path).suffix[1:].upper()

#                     # Convert PIL image to numpy array for brightness/contrast analysis
#                     img_array = np.array(img)

#                     # Calculate brightness and contrast
#                     if len(img_array.shape) == 3:  # Color image
#                         gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
#                     else:  # Grayscale
#                         gray = img_array

#                     brightness = np.mean(gray)
#                     contrast = np.std(gray)

#                 # Store properties
#                 properties['width'].append(width)
#                 properties['height'].append(height)
#                 properties['channels'].append(channels)
#                 properties['file_size'].append(file_size)
#                 properties['aspect_ratio'].append(width / height if height > 0 else 1.0)
#                 properties['brightness'].append(brightness)
#                 properties['contrast'].append(contrast)
#                 properties['species'].append(species)
#                 properties['format'].append(img_format)
#                 properties['area'].append(width * height)

#             except Exception as e:
#                 failed_count += 1
#                 print(f"Error processing {img_path}: {e}")
#                 continue

#         self.image_properties = pd.DataFrame(properties)
#         print(f"Successfully analyzed {len(self.image_properties)} images")
#         if failed_count > 0:
#             print(f"Failed to process {failed_count} images")

#         return self.image_properties

#     def generate_dataset_statistics(self):
#         """Generate dataset statistics"""
#         print("Generating dataset statistics...")
#         stats = {
#             'total_images': len(self.image_paths),
#             'total_species': len(self.species_names),
#             'species_distribution': Counter(self.labels),
#             'images_per_species': {species: len([p for p, l in zip(self.image_paths, self.labels) if l == species])
#                                   for species in self.species_names}
#         }

#         if not self.image_properties.empty:
#             stats.update({
#                 'resolution_stats': {
#                     'mean_width': self.image_properties['width'].mean(),
#                     'mean_height': self.image_properties['height'].mean(),
#                     'std_width': self.image_properties['width'].std(),
#                     'std_height': self.image_properties['height'].std(),
#                     'min_resolution': f"{self.image_properties['width'].min()}x{self.image_properties['height'].min()}",
#                     'max_resolution': f"{self.image_properties['width'].max()}x{self.image_properties['height'].max()}"
#                 },
#                 'file_stats': {
#                     'mean_file_size_kb': self.image_properties['file_size'].mean(),
#                     'std_file_size_kb': self.image_properties['file_size'].std(),
#                     'total_dataset_size_mb': self.image_properties['file_size'].sum() / 1024
#                 },
#                 'visual_stats': {
#                     'mean_brightness': self.image_properties['brightness'].mean(),
#                     'std_brightness': self.image_properties['brightness'].std(),
#                     'mean_contrast': self.image_properties['contrast'].mean(),
#                     'std_contrast': self.image_properties['contrast'].std()
#                 }
#             })
#         self.dataset_stats = stats
#         return stats

#     def plot_species_distribution(self, figsize=(15, 8)):
#         """Create species distribution visualization"""
#         species_counts = pd.Series(self.dataset_stats['images_per_species'])
#         plt.figure(figsize=figsize)
#         bars = plt.bar(range(len(species_counts)), species_counts.sort_values(ascending=False).values,
#                       color=plt.cm.Set3(np.linspace(0, 1, len(species_counts))))
#         plt.title('Images per Species', fontweight='bold')
#         plt.xlabel('Species')
#         plt.ylabel('Number of Images')
#         plt.xticks(range(len(species_counts)), species_counts.sort_values(ascending=False).index, rotation=45)

#         # Add value labels on bars
#         for i, bar in enumerate(bars):
#             height = bar.get_height()
#             plt.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
#                     f'{int(height)}', ha='center', va='bottom')

#         plt.tight_layout()
#         plt.savefig(f'{self.output_dir}/figures/species_distribution.png', bbox_inches='tight')
#         plt.close()

#     def plot_image_properties(self, figsize=(20, 15)):
#         """Comprehensive image properties visualization"""
#         if self.image_properties.empty:
#             print("No image properties available for plotting")
#             return

#         fig, axes = plt.subplots(3, 3, figsize=figsize)
#         fig.suptitle('Dataset Image Properties Analysis', fontsize=16, fontweight='bold')

#         # Resolution distribution
#         axes[0, 0].hist(self.image_properties['width'], bins=50, alpha=0.7, color='blue', label='Width')
#         axes[0, 0].hist(self.image_properties['height'], bins=50, alpha=0.7, color='red', label='Height')
#         axes[0, 0].set_title('Resolution Distribution')
#         axes[0, 0].set_xlabel('Pixels')
#         axes[0, 0].set_ylabel('Frequency')
#         axes[0, 0].legend()

#         # Aspect ratio
#         axes[0, 1].hist(self.image_properties['aspect_ratio'], bins=50, alpha=0.7, color='green')
#         axes[0, 1].set_title('Aspect Ratio Distribution')
#         axes[0, 1].set_xlabel('Width/Height Ratio')
#         axes[0, 1].set_ylabel('Frequency')
#         axes[0, 1].axvline(x=1.0, color='red', linestyle='--', label='Square (1:1)')
#         axes[0, 1].legend()

#         # File size
#         axes[0, 2].hist(self.image_properties['file_size'], bins=50, alpha=0.7, color='orange')
#         axes[0, 2].set_title('File Size Distribution')
#         axes[0, 2].set_xlabel('Size (KB)')
#         axes[0, 2].set_ylabel('Frequency')

#         # Brightness
#         axes[1, 0].hist(self.image_properties['brightness'], bins=50, alpha=0.7, color='yellow')
#         axes[1, 0].set_title('Brightness Distribution')
#         axes[1, 0].set_xlabel('Mean Pixel Intensity')
#         axes[1, 0].set_ylabel('Frequency')

#         # Contrast
#         axes[1, 1].hist(self.image_properties['contrast'], bins=50, alpha=0.7, color='purple')
#         axes[1, 1].set_title('Contrast Distribution')
#         axes[1, 1].set_xlabel('Standard Deviation of Intensity')
#         axes[1, 1].set_ylabel('Frequency')

#         # Image area
#         axes[1, 2].hist(self.image_properties['area'] / 1e6, bins=50, alpha=0.7, color='brown')
#         axes[1, 2].set_title('Image Area Distribution')
#         axes[1, 2].set_xlabel('Area (Megapixels)')
#         axes[1, 2].set_ylabel('Frequency')

#         # Brightness by species
#         species_brightness_data = [self.image_properties[self.image_properties['species'] == species]['brightness'].values
#                                   for species in self.species_names]
#         box_plot = axes[2, 0].boxplot(species_brightness_data, labels=self.species_names, patch_artist=True)
#         axes[2, 0].set_title('Brightness by Species')
#         axes[2, 0].set_ylabel('Mean Brightness')
#         axes[2, 0].tick_params(axis='x', rotation=45)

#         # Color the boxes
#         colors = plt.cm.Set3(np.linspace(0, 1, len(self.species_names)))
#         for patch, color in zip(box_plot['boxes'], colors):
#             patch.set_facecolor(color)

#         # Contrast by species
#         species_contrast_data = [self.image_properties[self.image_properties['species'] == species]['contrast'].values
#                                 for species in self.species_names]
#         box_plot2 = axes[2, 1].boxplot(species_contrast_data, labels=self.species_names, patch_artist=True)
#         axes[2, 1].set_title('Contrast by Species')
#         axes[2, 1].set_ylabel('Contrast (Std Dev)')
#         axes[2, 1].tick_params(axis='x', rotation=45)

#         # Color the boxes
#         for patch, color in zip(box_plot2['boxes'], colors):
#             patch.set_facecolor(color)

#         # Resolution scatter plot
#         unique_species = self.image_properties['species'].unique()
#         colors_scatter = plt.cm.tab10(np.linspace(0, 1, len(unique_species)))
#         for i, species in enumerate(unique_species):
#             species_data = self.image_properties[self.image_properties['species'] == species]
#             axes[2, 2].scatter(species_data['width'], species_data['height'],
#                              c=[colors_scatter[i]], alpha=0.6, label=species, s=20)
#         axes[2, 2].set_title('Resolution Scatter Plot')
#         axes[2, 2].set_xlabel('Width (pixels)')
#         axes[2, 2].set_ylabel('Height (pixels)')
#         axes[2, 2].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

#         plt.tight_layout()
#         plt.savefig(f'{self.output_dir}/figures/image_properties_analysis.png', bbox_inches='tight')
#         plt.close()

#     def create_correlation_matrix(self, figsize=(12, 10)):
#         """Create correlation matrix for numerical properties"""
#         if self.image_properties.empty:
#             print("No image properties available for correlation analysis")
#             return

#         numerical_cols = ['width', 'height', 'file_size', 'aspect_ratio', 'brightness', 'contrast', 'area']
#         correlation_matrix = self.image_properties[numerical_cols].corr()

#         plt.figure(figsize=figsize)
#         mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
#         sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
#                    square=True, fmt='.3f', cbar_kws={"shrink": .8})
#         plt.title('Image Properties Correlation Matrix', fontweight='bold', fontsize=14)
#         plt.tight_layout()
#         plt.savefig(f'{self.output_dir}/figures/correlation_matrix.png', bbox_inches='tight')
#         plt.close()
#         return correlation_matrix

#     def create_sample_grid(self, samples_per_species=3, figsize=(15, 10)):
#         """Create a grid of sample images from each species"""
#         rows, cols = len(self.species_names), min(samples_per_species, 5)
#         fig, axes = plt.subplots(rows, cols, figsize=figsize)

#         # Handle single row or column cases
#         if rows == 1 and cols == 1:
#             axes = np.array([[axes]])
#         elif rows == 1:
#             axes = axes.reshape(1, -1)
#         elif cols == 1:
#             axes = axes.reshape(-1, 1)

#         fig.suptitle('Sample Images by Species', fontsize=16, fontweight='bold')

#         for row, species in enumerate(self.species_names):
#             species_images = [p for p, l in zip(self.image_paths, self.labels) if l == species]
#             # Filter valid images
#             valid_species_images = [img for img in species_images if self.is_valid_image(img)]

#             if len(valid_species_images) > 0:
#                 sample_images = np.random.choice(valid_species_images,
#                                                min(samples_per_species, len(valid_species_images)),
#                                                replace=False)

#                 for col in range(cols):
#                     if col < len(sample_images):
#                         try:
#                             img = Image.open(sample_images[col])
#                             # Convert to RGB if necessary
#                             if img.mode in ('RGBA', 'LA', 'P'):
#                                 img = img.convert('RGB')
#                             axes[row, col].imshow(img)
#                             axes[row, col].axis('off')
#                             if col == 0:
#                                 axes[row, col].set_ylabel(species, rotation=90, fontsize=10)
#                         except Exception as e:
#                             axes[row, col].text(0.5, 0.5, 'Error loading image', ha='center', va='center',
#                                                transform=axes[row, col].transAxes)
#                             axes[row, col].axis('off')
#                     else:
#                         axes[row, col].axis('off')
#             else:
#                 # No valid images for this species
#                 for col in range(cols):
#                     axes[row, col].text(0.5, 0.5, 'No valid images', ha='center', va='center',
#                                        transform=axes[row, col].transAxes)
#                     axes[row, col].axis('off')
#                     if col == 0:
#                         axes[row, col].set_ylabel(species, rotation=90, fontsize=10)

#         plt.tight_layout()
#         plt.savefig(f'{self.output_dir}/figures/sample_images_grid.png', bbox_inches='tight')
#         plt.close()

#     def generate_statistical_summary(self):
#         """Generate statistical summary"""
#         summary = {
#             'Dataset Overview': {
#                 'Total Images': self.dataset_stats['total_images'],
#                 'Number of Species': self.dataset_stats['total_species'],
#                 'Average Images per Species': np.mean(list(self.dataset_stats['images_per_species'].values())),
#                 'Std Images per Species': np.std(list(self.dataset_stats['images_per_species'].values()))
#             }
#         }

#         if not self.image_properties.empty:
#             summary.update({
#                 'Image Resolution': {
#                     'Mean Width': f"{self.image_properties['width'].mean():.1f} ± {self.image_properties['width'].std():.1f} pixels",
#                     'Mean Height': f"{self.image_properties['height'].mean():.1f} ± {self.image_properties['height'].std():.1f} pixels",
#                     'Min Resolution': f"{self.image_properties['width'].min()}×{self.image_properties['height'].min()}",
#                     'Max Resolution': f"{self.image_properties['width'].max()}×{self.image_properties['height'].max()}",
#                     'Mean Aspect Ratio': f"{self.image_properties['aspect_ratio'].mean():.3f} ± {self.image_properties['aspect_ratio'].std():.3f}"
#                 },
#                 'File Properties': {
#                     'Mean File Size': f"{self.image_properties['file_size'].mean():.1f} ± {self.image_properties['file_size'].std():.1f} KB",
#                     'Total Dataset Size': f"{self.image_properties['file_size'].sum() / 1024:.1f} MB",
#                     'File Formats': dict(self.image_properties['format'].value_counts())
#                 },
#                 'Visual Properties': {
#                     'Mean Brightness': f"{self.image_properties['brightness'].mean():.1f} ± {self.image_properties['brightness'].std():.1f}",
#                     'Mean Contrast': f"{self.image_properties['contrast'].mean():.1f} ± {self.image_properties['contrast'].std():.1f}",
#                     'Brightness Range': f"{self.image_properties['brightness'].min():.1f} - {self.image_properties['brightness'].max():.1f}",
#                     'Contrast Range': f"{self.image_properties['contrast'].min():.1f} - {self.image_properties['contrast'].max():.1f}"
#                 }
#             })
#         return summary

#     def save_statistical_report(self):
#         """Save statistical report"""
#         summary = self.generate_statistical_summary()
#         with open(f'{self.output_dir}/reports/statistical_summary.txt', 'w') as f:
#             f.write("FISH SPECIES DATASET - STATISTICAL ANALYSIS REPORT\n")
#             f.write("=" * 60 + "\n\n")
#             for section, data in summary.items():
#                 f.write(f"{section}:\n")
#                 f.write("-" * len(section) + "\n")
#                 for key, value in data.items():
#                     f.write(f"  {key}: {value}\n")
#                 f.write("\n")

#         # Save species distribution
#         species_df = pd.DataFrame.from_dict(self.dataset_stats['images_per_species'],
#                                           orient='index', columns=['Image_Count'])
#         species_df.index.name = 'Species'
#         species_df.to_csv(f'{self.output_dir}/statistics/species_distribution.csv')

#         # Save image properties if available
#         if not self.image_properties.empty:
#             self.image_properties.to_csv(f'{self.output_dir}/statistics/image_properties.csv', index=False)

#         print(f"Statistical reports saved to {self.output_dir}/reports/ and {self.output_dir}/statistics/")

#     def create_publication_summary(self):
#         """Create publication-ready dataset summary"""
#         pub_data = []
#         for species, count in self.dataset_stats['images_per_species'].items():
#             species_props = self.image_properties[self.image_properties['species'] == species] if not self.image_properties.empty else pd.DataFrame()
#             pub_data.append({
#                 'Species': species,
#                 'Sample Count': count,
#                 'Mean Width (px)': f"{species_props['width'].mean():.0f}" if not species_props.empty else 'N/A',
#                 'Mean Height (px)': f"{species_props['height'].mean():.0f}" if not species_props.empty else 'N/A',
#                 'Mean Brightness': f"{species_props['brightness'].mean():.1f}" if not species_props.empty else 'N/A',
#                 'Mean Contrast': f"{species_props['contrast'].mean():.1f}" if not species_props.empty else 'N/A'
#             })
#         pub_df = pd.DataFrame(pub_data)
#         pub_df.to_csv(f'{self.output_dir}/reports/publication_dataset_table.csv', index=False)
#         return pub_df

#     def run_complete_analysis(self, sample_size=None):
#         """Run complete analysis pipeline"""
#         print("Starting comprehensive dataset analysis...")
#         print("=" * 50)

#         self.load_dataset_metadata()
#         self.analyze_image_properties(sample_size=sample_size)
#         self.generate_dataset_statistics()

#         print("\nGenerating visualizations...")
#         self.plot_species_distribution()
#         self.plot_image_properties()
#         self.create_correlation_matrix()
#         self.create_sample_grid()

#         print("\nGenerating statistical reports...")
#         self.save_statistical_report()
#         pub_table = self.create_publication_summary()

#         print(f"\nAnalysis complete! Results saved to: {self.output_dir}")
#         print("\nGenerated files:")
#         print("- Figures: species_distribution.png, image_properties_analysis.png, correlation_matrix.png, sample_images_grid.png")
#         print("- Reports: statistical_summary.txt, publication_dataset_table.csv")
#         print("- Statistics: species_distribution.csv, image_properties.csv")
#         return pub_table

# def analyze_fish_dataset(data_directory, output_directory='./dataset_analysis', sample_size=None):
#     """Analyze fish dataset from zip file or directory"""
#     extracted_path = data_directory

#     if data_directory.lower().endswith(".zip"):
#         print(f"Unzipping dataset: {data_directory}")
#         extracted_path = "/content/fish_dataset"
#         if os.path.exists(extracted_path):
#             shutil.rmtree(extracted_path)
#         with zipfile.ZipFile(data_directory, 'r') as zip_ref:
#             zip_ref.extractall(extracted_path)
#         print(f"Dataset extracted to {extracted_path}")

#     analyzer = FishDatasetAnalyzer(extracted_path, output_directory)
#     publication_table = analyzer.run_complete_analysis(sample_size=sample_size)
#     print("\nPublication-ready dataset summary:")
#     print(publication_table)
#     return analyzer, publication_table

# # Example usage
# if __name__ == "__main__":
#     # Update these paths according to your setup
#     DATA_FILE_ZIP = '/content/drive/MyDrive/Hilsha/data_fish_org_8407.zip'
#     OUTPUT_DIR = '/content/dataset_analysis'

#     try:
#         analyzer, results = analyze_fish_dataset(
#             data_directory=DATA_FILE_ZIP,
#             output_directory=OUTPUT_DIR,
#             sample_size=3000  # Analyze 5000 random images
#         )
#         print("\nDataset analysis completed successfully!")
#         print("All results saved for publication use.")
#     except Exception as e:
#         print(f"Error during analysis: {e}")
#         import traceback
#         traceback.print_exc()

# ✅ Step 6:Data Visualization [From Processed Image]


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import pandas as pd
# from collections import Counter
# import os
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE
# import warnings
# warnings.filterwarnings('ignore')


# # Scientific plotting setup
# plt.style.use('seaborn-v0_8')
# sns.set_palette("husl")
# plt.rcParams['figure.dpi'] = 300
# plt.rcParams['savefig.dpi'] = 300
# plt.rcParams['font.size'] = 12
# plt.rcParams['axes.titlesize'] = 14
# plt.rcParams['axes.labelsize'] = 12
# plt.rcParams['xtick.labelsize'] = 10
# plt.rcParams['ytick.labelsize'] = 10
# plt.rcParams['legend.fontsize'] = 10
# plt.rcParams['figure.titlesize'] = 16

# class FishDatasetNumpyAnalyzer:
#     """Comprehensive analysis suite for fish species dataset from NumPy arrays"""

#     def __init__(self, X_data, Y_labels, output_dir='./fish_classification_results'):
#         self.X_data = X_data
#         self.Y_labels = Y_labels
#         self.output_dir = output_dir
#         self.create_output_dirs()

#         # Dataset metadata
#         self.n_samples = X_data.shape[0]
#         self.image_shape = X_data.shape[1:]
#         self.unique_labels = np.unique(Y_labels)
#         self.n_classes = len(self.unique_labels)

#         # Determine image format (channels first vs channels last)
#         self.channels_first = self._detect_channels_first()

#         # Create label mapping if labels are numeric
#         if np.issubdtype(Y_labels.dtype, np.number):
#             self.label_names = [f"Species_{i}" for i in self.unique_labels]
#             self.label_to_name = dict(zip(self.unique_labels, self.label_names))
#         else:
#             self.label_names = self.unique_labels.tolist()
#             self.label_to_name = dict(zip(self.unique_labels, self.label_names))

#         print(f"Dataset loaded: {self.n_samples} samples, {self.n_classes} classes")
#         print(f"Image shape: {self.image_shape}")
#         print(f"Data type: {X_data.dtype}")
#         print(f"Channels first format: {self.channels_first}")

#     def _detect_channels_first(self):
#         """Detect if images are in channels-first format"""
#         if len(self.image_shape) == 3:
#             # If first dimension is small (1-4), likely channels first
#             # If last dimension is small (1-4), likely channels last
#             if self.image_shape[0] <= 4 and self.image_shape[0] < min(self.image_shape[1], self.image_shape[2]):
#                 return True
#             elif self.image_shape[2] <= 4 and self.image_shape[2] < min(self.image_shape[0], self.image_shape[1]):
#                 return False
#             else:
#                 # Default assumption based on common formats
#                 return self.image_shape[0] <= 4
#         return False

#     def _prepare_image_for_display(self, img):
#         """Convert image to proper format for matplotlib display"""
#         if len(img.shape) == 3:
#             if self.channels_first:
#                 # Convert from (C, H, W) to (H, W, C)
#                 img = np.transpose(img, (1, 2, 0))

#             # Handle different channel counts
#             if img.shape[2] == 1:  # Grayscale with channel dimension
#                 img = img.squeeze(axis=2)
#                 return img, 'gray'
#             elif img.shape[2] == 3:  # RGB
#                 return img, None
#             elif img.shape[2] == 4:  # RGBA
#                 return img[:, :, :3], None  # Drop alpha channel
#             else:
#                 # Multi-channel, use first channel as grayscale
#                 return img[:, :, 0], 'gray'
#         else:  # 2D grayscale
#             return img, 'gray'

#     def create_output_dirs(self):
#         """Create organized output directory structure"""
#         dirs = [
#             self.output_dir,
#             f"{self.output_dir}/figures",
#             f"{self.output_dir}/statistics",
#             f"{self.output_dir}/sample_images",
#             f"{self.output_dir}/reports"
#         ]
#         for dir_path in dirs:
#             os.makedirs(dir_path, exist_ok=True)

#     def analyze_data_properties(self):
#         """Analyze basic properties of the loaded data"""
#         print("Analyzing data properties...")

#         properties = {
#             'dataset_size': self.n_samples,
#             'n_classes': self.n_classes,
#             'image_shape': self.image_shape,
#             'channels_first': self.channels_first,
#             'data_type': str(self.X_data.dtype),
#             'data_range': {
#                 'min': float(self.X_data.min()),
#                 'max': float(self.X_data.max()),
#                 'mean': float(self.X_data.mean()),
#                 'std': float(self.X_data.std())
#             },
#             'class_distribution': dict(Counter(self.Y_labels)),
#             'memory_usage_mb': self.X_data.nbytes / (1024 * 1024)
#         }

#         # Per-class statistics
#         class_stats = {}
#         for label in self.unique_labels:
#             mask = self.Y_labels == label
#             class_data = self.X_data[mask]
#             class_stats[self.label_to_name[label]] = {
#                 'count': int(np.sum(mask)),
#                 'mean_intensity': float(class_data.mean()),
#                 'std_intensity': float(class_data.std()),
#                 'min_intensity': float(class_data.min()),
#                 'max_intensity': float(class_data.max())
#             }

#         properties['class_statistics'] = class_stats
#         self.data_properties = properties

#         return properties

#     def plot_class_distribution(self, figsize=(15, 8)):
#         """Visualize class distribution"""
#         class_counts = Counter(self.Y_labels)
#         class_names = [self.label_to_name[label] for label in class_counts.keys()]
#         counts = list(class_counts.values())

#         fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)

#         # Bar plot
#         bars = ax1.bar(range(len(class_names)), counts, color='skyblue', alpha=0.7)
#         ax1.set_title('Class Distribution', fontweight='bold')
#         ax1.set_xlabel('Species')
#         ax1.set_ylabel('Number of Samples')
#         ax1.set_xticks(range(len(class_names)))
#         ax1.set_xticklabels(class_names, rotation=45, ha='right')

#         # Add value labels on bars
#         for bar, count in zip(bars, counts):
#             height = bar.get_height()
#             ax1.text(bar.get_x() + bar.get_width()/2., height,
#                     f'{count}', ha='center', va='bottom')

#         # Pie chart
#         ax2.pie(counts, labels=class_names, autopct='%1.1f%%', startangle=90)
#         ax2.set_title('Class Distribution (%)', fontweight='bold')

#         plt.tight_layout()
#         plt.savefig(f'{self.output_dir}/figures/class_distribution.png', bbox_inches='tight')
#         plt.show()

#     def plot_sample_images(self, samples_per_class=5, figsize=(20, 12)):
#         """Display sample images from each class"""
#         n_classes = len(self.unique_labels)

#         fig, axes = plt.subplots(n_classes, samples_per_class, figsize=figsize)
#         if n_classes == 1:
#             axes = axes.reshape(1, -1)
#         elif samples_per_class == 1:
#             axes = axes.reshape(-1, 1)

#         fig.suptitle('Sample Images by Class', fontsize=16, fontweight='bold')

#         for i, label in enumerate(self.unique_labels):
#             # Get indices for this class
#             class_indices = np.where(self.Y_labels == label)[0]

#             # Sample random images from this class
#             if len(class_indices) >= samples_per_class:
#                 sample_indices = np.random.choice(class_indices, samples_per_class, replace=False)
#             else:
#                 sample_indices = class_indices

#             for j in range(samples_per_class):
#                 if j < len(sample_indices):
#                     img = self.X_data[sample_indices[j]].copy()

#                     # Prepare image for display
#                     display_img, cmap = self._prepare_image_for_display(img)

#                     # Normalize if needed
#                     if display_img.max() > 1:
#                         display_img = display_img.astype(float) / 255.0

#                     axes[i, j].imshow(display_img, cmap=cmap)
#                     axes[i, j].axis('off')

#                     if j == 0:  # Label the first column with class names
#                         axes[i, j].set_ylabel(self.label_to_name[label],
#                                             rotation=90, fontsize=12, va='center')
#                 else:
#                     axes[i, j].axis('off')

#         plt.tight_layout()
#         plt.savefig(f'{self.output_dir}/figures/sample_images.png', bbox_inches='tight')
#         plt.show()

#     def plot_pixel_intensity_analysis(self, figsize=(20, 12)):
#         """Analyze pixel intensity distributions"""
#         fig, axes = plt.subplots(2, 3, figsize=figsize)
#         fig.suptitle('Pixel Intensity Analysis', fontsize=16, fontweight='bold')

#         # Overall intensity distribution
#         axes[0, 0].hist(self.X_data.flatten(), bins=100, alpha=0.7, color='blue', density=True)
#         axes[0, 0].set_title('Overall Pixel Intensity Distribution')
#         axes[0, 0].set_xlabel('Pixel Intensity')
#         axes[0, 0].set_ylabel('Density')

#         # Mean intensity per image
#         mean_intensities = np.mean(self.X_data.reshape(self.n_samples, -1), axis=1)
#         axes[0, 1].hist(mean_intensities, bins=50, alpha=0.7, color='green', density=True)
#         axes[0, 1].set_title('Mean Intensity per Image')
#         axes[0, 1].set_xlabel('Mean Intensity')
#         axes[0, 1].set_ylabel('Density')

#         # Standard deviation per image
#         std_intensities = np.std(self.X_data.reshape(self.n_samples, -1), axis=1)
#         axes[0, 2].hist(std_intensities, bins=50, alpha=0.7, color='red', density=True)
#         axes[0, 2].set_title('Intensity Standard Deviation per Image')
#         axes[0, 2].set_xlabel('Std Intensity')
#         axes[0, 2].set_ylabel('Density')

#         # Class-wise intensity comparison
#         class_intensities = []
#         class_labels = []
#         for label in self.unique_labels:
#             mask = self.Y_labels == label
#             class_data = self.X_data[mask]
#             class_mean_intensities = np.mean(class_data.reshape(np.sum(mask), -1), axis=1)
#             class_intensities.extend(class_mean_intensities)
#             class_labels.extend([self.label_to_name[label]] * len(class_mean_intensities))

#         intensity_df = pd.DataFrame({
#             'intensity': class_intensities,
#             'class': class_labels
#         })

#         # Create boxplot data
#         box_data = [intensity_df[intensity_df['class'] == name]['intensity'].values
#                    for name in self.label_names]

#         axes[1, 0].boxplot(box_data, labels=self.label_names)
#         axes[1, 0].set_title('Mean Intensity by Class')
#         axes[1, 0].set_ylabel('Mean Intensity')
#         axes[1, 0].tick_params(axis='x', rotation=45)

#         # Average image intensity heatmap by class
#         avg_images = np.zeros((len(self.unique_labels), *self.image_shape))
#         for i, label in enumerate(self.unique_labels):
#             mask = self.Y_labels == label
#             avg_images[i] = np.mean(self.X_data[mask], axis=0)

#         # Calculate average intensity across spatial dimensions
#         if len(self.image_shape) == 3:
#             if self.channels_first:
#                 # Average across height and width for each channel
#                 avg_intensities = np.mean(avg_images, axis=(2, 3))  # Shape: (n_classes, n_channels)
#             else:
#                 # Average across height and width for each channel
#                 avg_intensities = np.mean(avg_images, axis=(1, 2))  # Shape: (n_classes, n_channels)
#         else:
#             # Grayscale images - average across spatial dimensions
#             avg_intensities = np.mean(avg_images, axis=(1, 2))  # Shape: (n_classes,)
#             avg_intensities = avg_intensities.reshape(-1, 1)  # Make it 2D for heatmap

#         im = axes[1, 1].imshow(avg_intensities, cmap='viridis', aspect='auto')
#         axes[1, 1].set_title('Average Intensity by Class')
#         axes[1, 1].set_ylabel('Class Index')
#         axes[1, 1].set_yticks(range(len(self.unique_labels)))
#         axes[1, 1].set_yticklabels([self.label_to_name[label] for label in self.unique_labels])

#         if len(self.image_shape) == 3:
#             if self.channels_first:
#                 n_channels = self.image_shape[0]
#             else:
#                 n_channels = self.image_shape[2]

#             if n_channels > 1:
#                 axes[1, 1].set_xlabel('Channel')
#                 axes[1, 1].set_xticks(range(n_channels))
#                 if n_channels == 3:
#                     axes[1, 1].set_xticklabels(['R', 'G', 'B'])
#                 else:
#                     axes[1, 1].set_xticklabels([f'Ch{i}' for i in range(n_channels)])
#         else:
#             axes[1, 1].set_xlabel('Intensity')

#         plt.colorbar(im, ax=axes[1, 1])

#         # Channel analysis for color images
#         if len(self.image_shape) == 3:
#             if self.channels_first:
#                 n_channels = self.image_shape[0]
#                 channel_means = np.mean(self.X_data, axis=(0, 2, 3))  # Average over batch, height, width
#             else:
#                 n_channels = self.image_shape[2]
#                 channel_means = np.mean(self.X_data, axis=(0, 1, 2))  # Average over batch, height, width

#             if n_channels > 1:
#                 colors = ['red', 'green', 'blue'] if n_channels == 3 else ['gray'] * n_channels
#                 axes[1, 2].bar(range(n_channels), channel_means, color=colors)
#                 axes[1, 2].set_title('Mean Intensity by Channel')
#                 axes[1, 2].set_xlabel('Channel')
#                 axes[1, 2].set_ylabel('Mean Intensity')
#                 if n_channels == 3:
#                     axes[1, 2].set_xticks(range(3))
#                     axes[1, 2].set_xticklabels(['Red', 'Green', 'Blue'])
#                 else:
#                     axes[1, 2].set_xticks(range(n_channels))
#                     axes[1, 2].set_xticklabels([f'Ch{i}' for i in range(n_channels)])
#             else:
#                 axes[1, 2].text(0.5, 0.5, 'Single Channel\n(Grayscale)',
#                                ha='center', va='center', transform=axes[1, 2].transAxes)
#                 axes[1, 2].set_title('Channel Information')
#         else:
#             axes[1, 2].text(0.5, 0.5, 'Single Channel\n(Grayscale)',
#                            ha='center', va='center', transform=axes[1, 2].transAxes)
#             axes[1, 2].set_title('Channel Information')

#         plt.tight_layout()
#         plt.savefig(f'{self.output_dir}/figures/pixel_intensity_analysis.png', bbox_inches='tight')
#         plt.show()

#     def perform_dimensionality_reduction(self, n_components=2, sample_size=5000):
#         """Perform PCA and t-SNE for visualization"""
#         print("Performing dimensionality reduction...")

#         # Sample data if too large
#         if self.n_samples > sample_size:
#             indices = np.random.choice(self.n_samples, sample_size, replace=False)
#             X_sample = self.X_data[indices]
#             Y_sample = self.Y_labels[indices]
#         else:
#             X_sample = self.X_data
#             Y_sample = self.Y_labels

#         # Flatten images
#         X_flat = X_sample.reshape(len(X_sample), -1)

#         # Normalize data
#         X_normalized = (X_flat - X_flat.mean()) / (X_flat.std() + 1e-8)

#         # PCA
#         print("Computing PCA...")
#         pca = PCA(n_components=n_components)
#         X_pca = pca.fit_transform(X_normalized)

#         # t-SNE
#         print("Computing t-SNE...")
#         tsne = TSNE(n_components=n_components, random_state=42, perplexity=30)
#         X_tsne = tsne.fit_transform(X_normalized)

#         # Plot results
#         fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

#         # PCA plot
#         scatter1 = ax1.scatter(X_pca[:, 0], X_pca[:, 1],
#                               c=Y_sample, cmap='tab10', alpha=0.6)
#         ax1.set_title(f'PCA Visualization\nExplained Variance: {pca.explained_variance_ratio_.sum():.3f}')
#         ax1.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.3f})')
#         ax1.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.3f})')

#         # Add colorbar for PCA
#         cbar1 = plt.colorbar(scatter1, ax=ax1)
#         cbar1.set_label('Class')

#         # t-SNE plot
#         scatter2 = ax2.scatter(X_tsne[:, 0], X_tsne[:, 1],
#                               c=Y_sample, cmap='tab10', alpha=0.6)
#         ax2.set_title('t-SNE Visualization')
#         ax2.set_xlabel('t-SNE 1')
#         ax2.set_ylabel('t-SNE 2')

#         # Add colorbar for t-SNE
#         cbar2 = plt.colorbar(scatter2, ax=ax2)
#         cbar2.set_label('Class')

#         plt.tight_layout()
#         plt.savefig(f'{self.output_dir}/figures/dimensionality_reduction.png', bbox_inches='tight')
#         plt.show()

#         return pca, X_pca, X_tsne

#     def create_data_summary_report(self):
#         """Create comprehensive data summary"""
#         if not hasattr(self, 'data_properties'):
#             self.analyze_data_properties()

#         summary = f"""
# FISH SPECIES DATASET - DATA ANALYSIS REPORT
# {'='*60}

# Dataset Overview:
# - Total samples: {self.data_properties['dataset_size']:,}
# - Number of classes: {self.data_properties['n_classes']}
# - Image dimensions: {self.data_properties['image_shape']}
# - Channels first format: {self.data_properties['channels_first']}
# - Data type: {self.data_properties['data_type']}
# - Memory usage: {self.data_properties['memory_usage_mb']:.1f} MB

# Data Range:
# - Minimum value: {self.data_properties['data_range']['min']:.3f}
# - Maximum value: {self.data_properties['data_range']['max']:.3f}
# - Mean value: {self.data_properties['data_range']['mean']:.3f}
# - Standard deviation: {self.data_properties['data_range']['std']:.3f}

# Class Distribution:
# """

#         for class_name, stats in self.data_properties['class_statistics'].items():
#             summary += f"\n{class_name}:\n"
#             summary += f"  - Sample count: {stats['count']:,}\n"
#             summary += f"  - Mean intensity: {stats['mean_intensity']:.3f}\n"
#             summary += f"  - Std intensity: {stats['std_intensity']:.3f}\n"
#             summary += f"  - Intensity range: {stats['min_intensity']:.3f} - {stats['max_intensity']:.3f}\n"

#         # Save report
#         with open(f'{self.output_dir}/reports/data_summary.txt', 'w') as f:
#             f.write(summary)

#         print(summary)
#         return summary

#     def run_complete_analysis(self, sample_size_dr=5000):
#         """Run complete analysis pipeline"""
#         print("Starting comprehensive NumPy dataset analysis...")
#         print("=" * 60)

#         # Step 1: Analyze data properties
#         self.analyze_data_properties()

#         # Step 2: Create visualizations
#         print("\nGenerating visualizations...")
#         self.plot_class_distribution()
#         self.plot_sample_images()
#         self.plot_pixel_intensity_analysis()

#         # Step 3: Dimensionality reduction
#         pca, X_pca, X_tsne = self.perform_dimensionality_reduction(sample_size=sample_size_dr)

#         # Step 4: Generate report
#         print("\nGenerating comprehensive report...")
#         self.create_data_summary_report()

#         print(f"\nAnalysis complete! Results saved to: {self.output_dir}")
#         print("\nGenerated files:")
#         print("- Figures: class_distribution.png, sample_images.png, pixel_intensity_analysis.png, dimensionality_reduction.png")
#         print("- Reports: data_summary.txt")

#         return pca, X_pca, X_tsne


# # Usage example:
# analyzer = FishDatasetNumpyAnalyzer(X_Loaded, Y_Loaded, output_dir='./fish_classification_results')
# pca_model, X_pca_result, X_tsne_result = analyzer.run_complete_analysis(sample_size_dr=3000)
# print("\nDataset analysis completed successfully!")
# print("All visualizations and reports have been generated.")

# ✅ Step 7: Model Architecture

In [ ]:
class ModelFactory:
    @staticmethod
    # def create_model(model_name, num_classes=Config.NUM_CLASSES, dropout_rate=0.5, hidden_dim_multiplier=0.5):
    def create_model(model_name, params=None, num_classes=Config.NUM_CLASSES):
        #Create model with configurable architecture
        if params is None:
            params = {}
        dropout_rate = params.get('dropout', 0.5)
        hidden_dim_multiplier = params.get('hidden_dim_multiplier', 0.5)

        if model_name == 'resnet50':
            model = models.resnet50(weights='IMAGENET1K_V2')
            # Partial unfreeze for better accuracy: unfreeze layer4 and fc
            for name, param in model.named_parameters():
                param.requires_grad = False
                # if "layer4" in name or "fc" in name:
                if "layer3" in name or "layer4" in name or "fc" in name:
                    param.requires_grad = True

            num_features = model.fc.in_features
            hidden_dim = int(num_features * hidden_dim_multiplier)
            model.fc = nn.Sequential(
                nn.Dropout(dropout_rate),
                nn.Linear(num_features, hidden_dim),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate / 2),
                nn.Linear(hidden_dim, num_classes)
            )

        elif model_name == 'efficientnet_b0':
            model = models.efficientnet_b0(weights='IMAGENET1K_V1')
            # Partial unfreeze: last blocks
            for name, param in model.named_parameters():
                param.requires_grad = False
                if "_blocks.15" in name or "_blocks.16" in name or "classifier" in name:
                    param.requires_grad = True
            num_features = model.classifier[1].in_features
            hidden_dim = int(num_features * hidden_dim_multiplier)
            model.classifier = nn.Sequential(
                nn.Dropout(dropout_rate),
                nn.Linear(num_features, hidden_dim),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate / 2),
                nn.Linear(hidden_dim, num_classes)
            )

        elif model_name == 'mobilenet_v3_large':
            model = models.mobilenet_v3_large(weights='IMAGENET1K_V2')
            # Partial unfreeze: last features
            for name, param in model.named_parameters():
                param.requires_grad = False
                if "features.12" in name or "features.13" in name or "classifier" in name:
                    param.requires_grad = True
            num_features = 960
            hidden_dim = int(num_features * hidden_dim_multiplier)
            model.classifier = nn.Sequential(
                nn.Dropout(dropout_rate),
                nn.Linear(num_features, hidden_dim),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate / 2),
                nn.Linear(hidden_dim, num_classes)
            )

        elif model_name == 'vgg16':
            model = models.vgg16(weights='IMAGENET1K_V1')
            # Partial unfreeze: classifier and last features
            for name, param in model.named_parameters():
                param.requires_grad = False
                if "classifier" in name or "features.28" in name:
                    param.requires_grad = True
            hidden_dim = int(4096 * hidden_dim_multiplier)
            model.classifier = nn.Sequential(
                nn.Linear(512 * 7 * 7, 4096),
                nn.ReLU(True),
                nn.Dropout(),
                nn.Linear(4096, hidden_dim),
                nn.ReLU(True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate),
                nn.Linear(hidden_dim, num_classes)
            )

        elif model_name == 'densenet121':
            model = models.densenet121(weights='IMAGENET1K_V1')
            # Partial unfreeze: denseblock4 and classifier
            for name, param in model.named_parameters():
                param.requires_grad = False
                if "denseblock4" in name or "classifier" in name:
                    param.requires_grad = True
            num_features = model.classifier.in_features
            hidden_dim = int(num_features * hidden_dim_multiplier)
            model.classifier = nn.Sequential(
                nn.Dropout(dropout_rate),
                nn.Linear(num_features, hidden_dim),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate / 2),
                nn.Linear(hidden_dim, num_classes)
            )

        elif model_name == 'inception_v3':
            model = models.inception_v3(weights='IMAGENET1K_V1')
            # Partial unfreeze: Mixed_7a, Mixed_7b, Mixed_7c and classifiers
            for name, param in model.named_parameters():
                param.requires_grad = False
                if any(layer in name for layer in ["Mixed_7a", "Mixed_7b", "Mixed_7c", "fc", "AuxLogits"]):
                    param.requires_grad = True

            num_features = model.fc.in_features
            hidden_dim = int(num_features * hidden_dim_multiplier)

            # Main classifier
            model.fc = nn.Sequential(
                nn.Dropout(dropout_rate),
                nn.Linear(num_features, hidden_dim),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate / 2),
                nn.Linear(hidden_dim, num_classes)
            )

            # Auxiliary classifier (if exists)
            if hasattr(model, 'AuxLogits') and model.AuxLogits is not None:
                aux_features = model.AuxLogits.fc.in_features
                aux_hidden = int(aux_features * hidden_dim_multiplier)
                model.AuxLogits.fc = nn.Sequential(
                    nn.Dropout(dropout_rate),
                    nn.Linear(aux_features, aux_hidden),
                    nn.ReLU(inplace=True),
                    nn.BatchNorm1d(aux_hidden),
                    nn.Dropout(dropout_rate / 2),
                    nn.Linear(aux_hidden, num_classes)
                )

        elif model_name == 'vit_b_16':
            model = models.vit_b_16(weights='IMAGENET1K_V1')
            # Partial unfreeze: last encoder layers and head
            for name, param in model.named_parameters():
                param.requires_grad = False
                if any(layer in name for layer in ["encoder.layers.10", "encoder.layers.11", "heads"]):
                    param.requires_grad = True

            num_features = model.heads.head.in_features
            hidden_dim = int(num_features * hidden_dim_multiplier)
            model.heads.head = nn.Sequential(
                nn.Dropout(dropout_rate),
                nn.Linear(num_features, hidden_dim),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate / 2),
                nn.Linear(hidden_dim, num_classes)
            )

        elif model_name == 'convnext_base':
            model = models.convnext_base(weights='IMAGENET1K_V1')
            # Partial unfreeze: last stages and classifier
            for name, param in model.named_parameters():
                param.requires_grad = False
                if any(layer in name for layer in ["features.7", "classifier"]):
                    param.requires_grad = True

            num_features = model.classifier[2].in_features
            hidden_dim = int(num_features * hidden_dim_multiplier)
            model.classifier = nn.Sequential(
                model.classifier[0],  # Keep the LayerNorm
                model.classifier[1],  # Keep the Flatten
                nn.Dropout(dropout_rate),
                nn.Linear(num_features, hidden_dim),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate / 2),
                nn.Linear(hidden_dim, num_classes)
            )

        elif model_name == 'regnet_y_32gf':
            model = models.regnet_y_32gf(weights='IMAGENET1K_V2')
            # Partial unfreeze: last trunk stage and fc
            for name, param in model.named_parameters():
                param.requires_grad = False
                if "trunk_output" in name or "fc" in name:
                    param.requires_grad = True

            num_features = model.fc.in_features
            hidden_dim = int(num_features * hidden_dim_multiplier)
            model.fc = nn.Sequential(
                nn.Dropout(dropout_rate),
                nn.Linear(num_features, hidden_dim),
                nn.ReLU(inplace=True),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(dropout_rate / 2),
                nn.Linear(hidden_dim, num_classes)
            )

        elif model_name == 'cnn':
            # class SimpleCNN(nn.Module):

            class SimpleCNN(nn.Module):
                def __init__(self, num_classes=5, dropout_rate=0.3, hidden_dim_multiplier=0.3):
                    super(SimpleCNN, self).__init__()

                    # More conservative feature extractor to prevent overfitting
                    self.features = nn.Sequential(
                        # Block 1 - Start small
                        nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(32),
                        nn.ReLU(inplace=True),
                        nn.Dropout2d(0.1),  # Spatial dropout in conv layers
                        nn.MaxPool2d(2, 2),  # 224 -> 112

                        # Block 2
                        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(64),
                        nn.ReLU(inplace=True),
                        nn.Dropout2d(0.15),
                        nn.MaxPool2d(2, 2),  # 112 -> 56

                        # Block 3
                        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(128),
                        nn.ReLU(inplace=True),
                        nn.Dropout2d(0.2),
                        nn.MaxPool2d(2, 2),  # 56 -> 28

                        # Block 4 - Add one more conv before pooling
                        nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(128),
                        nn.ReLU(inplace=True),
                        nn.Dropout2d(0.25),
                        nn.MaxPool2d(2, 2),  # 28 -> 14

                        # Block 5 - Final feature extraction
                        nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(256),
                        nn.ReLU(inplace=True),
                        nn.Dropout2d(0.3),
                        nn.AdaptiveAvgPool2d((7, 7))  # Fixed spatial size
                    )

                    # Calculate features after adaptive pooling
                    conv_output_size = 256 * 7 * 7  # 12544

                    # Much smaller hidden dimension to prevent overfitting
                    hidden_dim = int(conv_output_size * hidden_dim_multiplier)
                    hidden_dim = max(64, min(hidden_dim, 512))  # Smaller range

                    # Simple but effective classifier
                    self.classifier = nn.Sequential(
                        nn.Dropout(dropout_rate),
                        nn.Linear(conv_output_size, hidden_dim),
                        nn.ReLU(inplace=True),
                        nn.BatchNorm1d(hidden_dim),
                        nn.Dropout(dropout_rate * 0.5),
                        nn.Linear(hidden_dim, num_classes)
                    )

                    # Initialize weights properly
                    self._initialize_weights()

                def _initialize_weights(self):
                    for m in self.modules():
                        if isinstance(m, nn.Conv2d):
                            # Use smaller initialization for better gradient flow
                            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                            if m.bias is not None:
                                nn.init.zeros_(m.bias)
                        elif isinstance(m, nn.Linear):
                            # Smaller initialization for linear layers
                            nn.init.xavier_uniform_(m.weight, gain=0.5)
                            if m.bias is not None:
                                nn.init.zeros_(m.bias)
                        elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d)):
                            if m.weight is not None:
                                nn.init.ones_(m.weight)
                            if m.bias is not None:
                                nn.init.zeros_(m.bias)

                def forward(self, x):
                    # Feature extraction
                    x = self.features(x)

                    # Flatten
                    x = torch.flatten(x, 1)

                    # Classification with gradient clipping
                    x = self.classifier(x)

                    # Clip outputs to prevent extreme values
                    x = torch.clamp(x, min=-10, max=10)

                    return x

            # # Example usage
            # model = SimpleCNN(num_classes=num_classes, dropout_rate=dropout_rate, hidden_dim_multiplier=hidden_dim_multiplier)
            # model = model.to(Config.DEVICE)  # Move to device right after creation
            model = SimpleCNN(num_classes=num_classes, dropout_rate=dropout_rate, hidden_dim_multiplier=hidden_dim_multiplier)

        else:
            raise ValueError(f"Unsupported model: {model_name}")

        return model

# ✅ Step 8: Ensamble Model Architecture

In [ ]:
# ---
# 6. ENSEMBLE METHODS
# ================================================================================================================================
# Purpose: Implement ensemble methods (simple, weighted, confidence-based, learnable).

class EnsembleManager:
    def __init__(self, models_dict, val_data):
        self.models = models_dict
        self.X_val, self.y_val = val_data
        self.model_predictions = self._get_predictions()
        self.histories = {}

    def _get_predictions(self):
        print("Getting model predictions for ensemble...")
        predictions = {}

        val_dataset = FishDataset(self.X_val, self.y_val, DataManager.get_transforms(False))
        val_loader = DataLoader(val_dataset, batch_size=Config.BATCH_SIZE, shuffle=False)

        for name, model in self.models.items():
            model.eval()
            all_preds = []
            all_probs = []
            all_losses = []
            all_labels = []
            total = 0
            correct = 0
            criterion = nn.CrossEntropyLoss()

            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
                    outputs = model(images)
                    probabilities = torch.softmax(outputs, dim=1)
                    loss = criterion(outputs, labels).item()

                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                    all_preds.extend(predicted.cpu().numpy())
                    all_probs.extend(probabilities.cpu().numpy())
                    all_losses.append(loss)
                    all_labels.extend(labels.cpu().numpy())

            accuracy = correct / total
            f1 = f1_score(self.y_val, all_preds, average='macro')
            avg_loss = np.mean(all_losses)

            predictions[name] = {
                'predictions': np.array(all_preds),
                'probabilities': np.array(all_probs),
                'loss': avg_loss,
                'accuracy': accuracy,
                'f1': f1,
                'true_labels': np.array(all_labels)
            }

            print(f"  {name}: F1 = {f1:.4f}, Acc = {accuracy:.4f}, Loss = {avg_loss:.4f}")

        return predictions

    def simple_average_ensemble(self, model_combo):
        selected_probs = [self.model_predictions[name]['probabilities'] for name in model_combo]
        avg_probs = np.mean(selected_probs, axis=0)
        predictions = np.argmax(avg_probs, axis=1)

        accuracy = accuracy_score(self.y_val, predictions)
        f1 = f1_score(self.y_val, predictions, average='macro')
        loss = np.mean([self.model_predictions[name]['loss'] for name in model_combo])

        avg_probs = np.mean(selected_probs, axis=0) if selected_probs else np.zeros((len(self.y_val), Config.NUM_CLASSES))

        return {
            'accuracy': accuracy,
            'f1': f1,
            'loss': loss,
            'predictions': predictions,
            'models': model_combo,
            # 'probabilities': avg_probs,
            'probabilities': avg_probs if avg_probs.ndim == 2 else np.zeros((0, Config.NUM_CLASSES)),
            'true_labels': self.y_val
        }

    def weighted_average_ensemble(self, model_combo):
        weights = []
        selected_probs = []

        for name in model_combo:
            f1 = self.model_predictions[name]['f1']
            weights.append(f1)
            selected_probs.append(self.model_predictions[name]['probabilities'])

        weights = np.array(weights) / np.sum(weights)
        weighted_probs = np.average(selected_probs, axis=0, weights=weights)
        predictions = np.argmax(weighted_probs, axis=1)

        accuracy = accuracy_score(self.y_val, predictions)
        f1 = f1_score(self.y_val, predictions, average='macro')
        loss = np.average([self.model_predictions[name]['loss'] for name in model_combo], weights=weights)

        return {
            'accuracy': accuracy,
            'f1': f1,
            'loss': loss,
            'predictions': predictions,
            'weights': weights,
            'models': model_combo,
            'probabilities': weighted_probs,
            'true_labels': self.y_val
        }

    def confidence_based_ensemble(self, model_combo):
        final_predictions = []
        all_probs = []

        for i in range(len(self.y_val)):
            confidences = []
            probs = []

            for name in model_combo:
                prob = self.model_predictions[name]['probabilities'][i]
                confidence = np.max(prob)
                confidences.append(confidence)
                probs.append(prob)

            confidences = np.array(confidences)
            weights = confidences / np.sum(confidences) if np.sum(confidences) > 0 else np.ones(len(confidences)) / len(confidences)

            final_prob = np.average(probs, axis=0, weights=weights)
            final_predictions.append(np.argmax(final_prob))
            all_probs.append(final_prob)

        predictions = np.array(final_predictions)
        accuracy = accuracy_score(self.y_val, predictions)
        f1 = f1_score(self.y_val, predictions, average='macro')
        loss = np.mean([self.model_predictions[name]['loss'] for name in model_combo])

        return {
            'accuracy': accuracy,
            'f1': f1,
            'loss': loss,
            'predictions': predictions,
            'models': model_combo,
            'probabilities': np.array(all_probs),
            'true_labels': self.y_val
        }

    def learnable_weighted_ensemble(self, model_combo, epochs=30):
        print(f"Training learnable weighted ensemble with {len(model_combo)} models...")

        selected_probs = []
        for name in model_combo:
            selected_probs.append(self.model_predictions[name]['probabilities'])

        ensemble_input = np.stack(selected_probs, axis=1)

        X_ensemble = torch.FloatTensor(ensemble_input).to(Config.DEVICE)
        y_ensemble = torch.LongTensor(self.y_val).to(Config.DEVICE)

        ensemble_model = LearnableWeightedEnsemble(
            num_models=len(model_combo),
            num_classes=Config.NUM_CLASSES
        ).to(Config.DEVICE)

        optimizer = optim.AdamW(ensemble_model.parameters(), lr=1e-3, weight_decay=1e-4)
        criterion = nn.CrossEntropyLoss()
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

        history = {'train_loss': [], 'train_acc': [], 'val_f1': []}
        best_loss = float('inf')

        for epoch in range(epochs):
            ensemble_model.train()
            optimizer.zero_grad()
            predictions, weights = ensemble_model(X_ensemble)
            loss = criterion(predictions, y_ensemble)
            loss.backward()
            optimizer.step()
            scheduler.step()

            accuracy = accuracy_score(y_ensemble.cpu().numpy(), predictions.argmax(dim=1).cpu().numpy())
            f1 = f1_score(y_ensemble.cpu().numpy(), predictions.argmax(dim=1).cpu().numpy(), average='macro')

            history['train_loss'].append(loss.item())
            history['train_acc'].append(accuracy)
            history['val_f1'].append(f1)

            print(f"Ensemble Epoch {epoch+1}/{epochs}: Loss = {loss.item():.4f}, "
                  f"Acc = {accuracy:.4f}, F1 = {f1:.4f}")

            if loss.item() < best_loss:
                best_loss = loss.item()
                torch.save(ensemble_model.state_dict(), f"{Config.OUTPUT_DIR}/models/learnable_ensemble_{'+'.join(model_combo)}.pt")

        ensemble_model.load_state_dict(torch.load(f"{Config.OUTPUT_DIR}/models/learnable_ensemble_{'+'.join(model_combo)}.pt"))
        ensemble_model.eval()
        with torch.no_grad():
            final_predictions, learned_weights = ensemble_model(X_ensemble)
            predictions = final_predictions.argmax(dim=1).cpu().numpy()
            probabilities = torch.softmax(final_predictions, dim=1).cpu().numpy()
            avg_weights = learned_weights.mean(dim=0).cpu().numpy()

        accuracy = accuracy_score(self.y_val, predictions)
        f1 = f1_score(self.y_val, predictions, average='macro')
        loss = np.mean([self.model_predictions[name]['loss'] for name in model_combo])

        self.histories[f"learnable_weighted_{'+'.join(model_combo)}"] = history

        return {
            'accuracy': accuracy,
            'f1': f1,
            'loss': loss,
            'predictions': predictions,
            'models': model_combo,
            'learned_weights': avg_weights,
            'probabilities': probabilities,
            'true_labels': self.y_val
        }

    def test_ensemble_combinations(self):
        print("Testing ensemble combinations...")

        model_names = list(self.models.keys())
        all_results = {}
        best_result = None
        best_score = 0

        for size in range(2, min(len(model_names) + 1, 5)):
            print(f"Testing {size}-model combinations...")

            for combo in list(combinations(model_names, size))[:5]:
                combo_name = f"combo_{size}_{'+'.join(combo)}"

                for method_name in Config.ENSEMBLE_METHODS:
                    full_name = f"{combo_name}_{method_name}"

                    try:
                        if method_name == 'simple_average':
                            result = self.simple_average_ensemble(combo)
                        elif method_name == 'weighted_average':
                            result = self.weighted_average_ensemble(combo)
                        elif method_name == 'confidence_based':
                            result = self.confidence_based_ensemble(combo)
                        elif method_name == 'learnable_weighted':
                            result = self.learnable_weighted_ensemble(combo)

                        # Verify result contains required keys
                        required_keys = ['accuracy', 'f1', 'loss', 'predictions', 'models', 'probabilities', 'true_labels']
                        if not all(key in result for key in required_keys):
                            missing = [key for key in required_keys if key not in result]
                            print(f"  {full_name}: Missing keys {missing}")
                            continue
                        # Ensure probabilities is 2D
                        if 'probabilities' in result and (result['probabilities'].ndim != 2 or result['probabilities'].shape[1] != Config.NUM_CLASSES):
                            result['probabilities'] = np.zeros((len(result['true_labels']), Config.NUM_CLASSES))

                        all_results[full_name] = result
                        print(f"  {full_name}: F1 = {result['f1']:.4f}, Acc = {result['accuracy']:.4f}, "
                              f"Loss = {result['loss']:.4f}, True Labels Shape = {result['true_labels'].shape}")

                        if result['f1'] > best_score:
                            best_score = result['f1']
                            best_result = (full_name, result)

                    except Exception as e:
                        print(f"  {full_name}: FAILED - {str(e)}")

        if best_result:
            print(f"\n✓ Best ensemble: {best_result[0]} (F1: {best_result[1]['f1']:.4f})")
        else:
            print("\nNo valid ensemble results generated.")

        return all_results, best_result





# LEARNABLE WEIGHTED ENSEMBLE MODEL
# ===============================================================================================================================
# Purpose: Define a neural network for learning optimal ensemble weights.

class LearnableWeightedEnsemble(nn.Module):
    """Ensemble model with per-class adaptive weights and attention"""
    def __init__(self, num_models, num_classes, hidden_dim=128, num_heads=4):
        super(LearnableWeightedEnsemble, self).__init__()
        self.num_models = num_models
        self.num_classes = num_classes

        # Attention mechanism to learn relations between model predictions
        self.attention = nn.MultiheadAttention(embed_dim=num_classes, num_heads=num_heads, batch_first=True)

        # Weight network outputs per-class weights for each model
        self.weight_network = nn.Sequential(
            nn.Linear(num_classes, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes),
            nn.Sigmoid()  # Per-class weight scaling
        )

        # Prediction head: combines weighted predictions + raw predictions
        self.prediction_head = nn.Sequential(
            nn.Linear(num_classes * (num_models + 1), hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes * 2),
            nn.ReLU(),
            nn.Linear(num_classes * 2, num_classes)
        )

    def forward(self, model_predictions):
        """
        model_predictions: (batch, num_models, num_classes)
        Returns:
            final_predictions: logits for classification
            weights: learned per-class weights for each model
        """
        batch_size = model_predictions.size(0)

        # --- Step 1: Attention over model predictions --- #The model looks at how predictions of different models relate to each other.
        attn_output, _ = self.attention(model_predictions, model_predictions, model_predictions)
        # shape: (batch, num_models, num_classes)


        # --- Step 2: Per-class weights for each model ---
        #Learns a weight for each model for each class.
        #softmax ensures weights across models sum to 1 for each class.
        #Basically: “For class 0, I trust model 2 more; for class 1, I trust model 0 more.”
        weights = self.weight_network(attn_output)  # (batch, num_models, num_classes)
        weights = F.softmax(weights, dim=1)  # normalize over models


        # --- Step 3: Weighted average across models ---
        #Combines the models’ predictions using the learned weights → smarter than a plain average.
        weighted_avg = torch.sum(model_predictions * weights, dim=1)  # (batch, num_classes)


        # --- Step 4: Residual connection with raw predictions ---
        #Combines the weighted average and all raw predictions.Gives the network more info to refine the final prediction.
        flat_preds = model_predictions.view(batch_size, -1)  # (batch, num_models * num_classes)
        final_input = torch.cat([weighted_avg, flat_preds], dim=1)  # (batch, num_classes + num_models*num_classes)


        # --- Step 5: Final refined prediction ---
        #A small feed-forward network refines the predictions.Output: (batch_size, num_classes) → logits for each class.
        final_predictions = self.prediction_head(final_input)  # (batch, num_classes)

        return final_predictions, weights
        #It learns which model is best for each class, combines their predictions smartly using attention, and produces a refined final prediction.

    # def entropy_regularization(self, weights):
    #     """Encourage diverse weight usage (optional loss term)."""
    #     # weights: (batch, num_models, num_classes)
    #     entropy = -torch.sum(weights * torch.log(weights + 1e-8), dim=1)  # (batch, num_classes)
    #     return torch.mean(entropy)


# ✅ Step 9: Evaluation

In [ ]:
class ModelEvaluator:
    @staticmethod
    def evaluate_model(model, test_loader, model_name):
        try:
            print(f"\nEvaluating For Model: {model_name}...")
            model.eval()

            all_preds = []
            all_labels = []
            misclassified = []
            total_loss = 0
            criterion = nn.CrossEntropyLoss()

            all_probs = []

            if test_loader is None or len(test_loader.dataset) == 0:
                print(f"Warning: No test data available for {model_name}")
                return ModelEvaluator._create_empty_result(model_name)

            with torch.no_grad():
                for batch_idx, (images, labels) in enumerate(test_loader):
                    # CPU version
                    # images, labels = images.cpu(), labels.cpu()
                    #GPU version
                    # Keep everything on the same device as the model
                    images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)



                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    total_loss += loss.item()

                    _, predicted = torch.max(outputs.data, 1)

                    # all_preds.extend(predicted.numpy())
                    # all_labels.extend(labels.numpy())
                    # To this:
                    all_preds.extend(predicted.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())



                    # Convert logits to probabilities
                    prob = torch.softmax(outputs, dim=1)
                    # Store probabilities
                    all_probs.append(prob.detach().cpu().numpy())




                    # Identify misclassified
                    incorrect_mask = predicted != labels
                    if incorrect_mask.any():
                        incorrect_images = images[incorrect_mask]
                        incorrect_labels = labels[incorrect_mask]
                        incorrect_preds = predicted[incorrect_mask]
                        for img, true_label, pred_label in zip(incorrect_images, incorrect_labels, incorrect_preds):
                            misclassified.append({
                                'image': img,
                                'true_label': int(true_label),
                                'pred_label': int(pred_label)
                            })
                    del images, labels, outputs, loss, predicted




                    # ---------------- GPU VERSION (commented) ----------------
                    # images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
                    # outputs = model(images)
                    # loss = criterion(outputs, labels)
                    # _, predicted = torch.max(outputs.data, 1)
                    # all_preds.append(predicted)
                    # all_labels.append(labels)
                    # --------------------------------------------------------



            # CPU metrics
            # Convert to single array
            all_probs = np.concatenate(all_probs, axis=0)  # shape: (num_samples, num_classes)
            # all_probs = [
            #     (32, 5),   # batch 1: 32 samples × 5 classes
            #     (32, 5),   # batch 2: 32 samples × 5 classes
            #     (16, 5)    # batch 3: 16 samples × 5 classes
            # ]
            # result shape = (32 + 32 + 16, 5) = (80, 5)




            all_preds = np.array(all_preds)
            all_labels = np.array(all_labels)
            avg_loss = total_loss / len(test_loader)
            accuracy = accuracy_score(all_labels, all_preds)
            f1_macro = f1_score(all_labels, all_preds, average='macro', zero_division=0)
            f1_weighted = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
            precision_macro = precision_score(all_labels, all_preds, average='macro', zero_division=0)
            recall_macro = recall_score(all_labels, all_preds, average='macro', zero_division=0)
            f1_per_class = f1_score(all_labels, all_preds, average=None, zero_division=0)
            precision_per_class = precision_score(all_labels, all_preds, average=None, zero_division=0)
            recall_per_class = recall_score(all_labels, all_preds, average=None, zero_division=0)
            conf_matrix = confusion_matrix(all_labels, all_preds)

            print(f"{model_name} Evaluation: Acc = {accuracy:.4f}, F1 (Macro) = {f1_macro:.4f}, Loss = {avg_loss:.4f}")
            print(f"Data shapes - Labels: {all_labels.shape}, Predictions: {all_preds.shape}")

            return {
                'all_preds': all_preds,
                'all_labels': all_labels,
                'all_probs': all_probs,


                'model_name': model_name,
                'accuracy': accuracy,
                'f1_macro': f1_macro,
                'f1_weighted': f1_weighted,
                'precision_macro': precision_macro,
                'recall_macro': recall_macro,
                'f1_per_class': f1_per_class,
                'precision_per_class': precision_per_class,
                'recall_per_class': recall_per_class,
                'predictions': all_preds,
                'true_labels': all_labels,
                # 'probabilities': None,  # CPU version
                'probabilities': all_probs,  # correctly collected probabilities
                'conf_matrix': conf_matrix,
                'loss': avg_loss,
                'misclassified': misclassified
            }

        except Exception as e:
            print(f"Evaluation error for {model_name}: {e}")
            return ModelEvaluator._create_empty_result(model_name)

    @staticmethod
    def _create_empty_result(model_name):
        return {
            'model_name': model_name,
            'accuracy': 0.0,
            'f1_macro': 0.0,
            'f1_weighted': 0.0,
            'precision_macro': 0.0,
            'recall_macro': 0.0,
            'f1_per_class': np.zeros(Config.NUM_CLASSES),
            'precision_per_class': np.zeros(Config.NUM_CLASSES),
            'recall_per_class': np.zeros(Config.NUM_CLASSES),
            'predictions': np.array([]),
            'true_labels': np.array([]),
            'probabilities': np.array([]),
            'conf_matrix': np.zeros((Config.NUM_CLASSES, Config.NUM_CLASSES)),
            'loss': 0.0,
            'misclassified': []
        }

# ✅ Step 10: Plot Visualizations

In [ ]:
# visualizations for training history, ROC curves, confusion matrices, model comparisons, and XAI visualizations.

class EnhancedVisualizations:
    def __init__(self):
        self.viz_dir = f"{Config.OUTPUT_DIR}/visualizations"
        Path(self.viz_dir).mkdir(parents=True, exist_ok=True)
        # Set better matplotlib parameters for spacing
        plt.rcParams.update({
            'figure.autolayout': True,
            'axes.titlepad': 20,
            'axes.labelpad': 10,
            'xtick.major.pad': 8,
            'ytick.major.pad': 8
        })


    def plot_single_model_history(self, history, model_name, alpha=0.2):  # (range: 0.1–0.3; lower = smoother, higher = more responsive)
        """Plot training history for individual model with better spacing"""
        if not history['train_loss']:
            print(f"Skipping {model_name}: No training data available")
            return

        # def smooth_ema(values, alpha):
        #     """Apply exponential moving average (EMA) smoothing to a list of values."""
        #     if not values:
        #         return values
        #     smoothed = [values[0]]
        #     for val in values[1:]:
        #         smoothed.append(alpha * val + (1 - alpha) * smoothed[-1])
        #     return smoothed
        def smooth_ema(values, alpha=0.1):
            """
            Apply exponential moving average smoothing to a list or tensor of values.

            Args:
                values: List, NumPy array, or PyTorch tensor of numerical values
                alpha: Smoothing factor (default: 0.1)

            Returns:
                List of smoothed values
            """
            # Convert tensor to list if necessary
            if torch.is_tensor(values):
                values = values.cpu().numpy().tolist()
            elif isinstance(values, np.ndarray):
                values = values.tolist()

            if not values:  # Check if the list is empty
                return []

            smoothed = [values[0]]
            for i in range(1, len(values)):
                smoothed.append(alpha * values[i] + (1 - alpha) * smoothed[-1])

            return smoothed



        epochs = range(1, len(history['train_loss']) + 1)

        # Create subplot with more space
        fig = plt.figure(figsize=(16, 10))
        gs = fig.add_gridspec(2, 2, hspace=0.4, wspace=0.3)

        # Loss plot
        ax1 = fig.add_subplot(gs[0, 0])
        smoothed_train_loss = smooth_ema(history['train_loss'], alpha)
        smoothed_val_loss = smooth_ema(history.get('val_loss', []), alpha)
        ax1.plot(epochs, smoothed_train_loss, 'b-', linewidth=2, label='Train Loss', marker='o', markersize=4)
        if smoothed_val_loss:
            ax1.plot(epochs, smoothed_val_loss, 'r-', linewidth=2, label='Val Loss', marker='s', markersize=4)
        ax1.set_title(f'{model_name} - Loss vs Epoch', fontsize=14, fontweight='bold', pad=20)
        ax1.set_xlabel('Epoch', fontsize=12, fontweight='bold')
        ax1.set_ylabel('Loss', fontsize=12, fontweight='bold')
        ax1.legend(frameon=True, shadow=True, fontsize=11)
        ax1.grid(True, alpha=0.3)
        ax1.tick_params(labelsize=10)

        # Accuracy plot
        ax2 = fig.add_subplot(gs[0, 1])
        ax2.plot(epochs, history['train_acc'], 'g-', linewidth=2, label='Train Acc', marker='o', markersize=4)
        if history.get('val_acc', []):
            ax2.plot(epochs, history['val_acc'], 'm-', linewidth=2, label='Val Acc', marker='s', markersize=4)
        ax2.set_title(f'{model_name} - Accuracy vs Epoch', fontsize=14, fontweight='bold', pad=20)
        ax2.set_xlabel('Epoch', fontsize=12, fontweight='bold')
        ax2.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
        ax2.legend(frameon=True, shadow=True, fontsize=11)
        ax2.grid(True, alpha=0.3)
        ax2.tick_params(labelsize=10)

        # F1 Score plot
        ax3 = fig.add_subplot(gs[1, 0])
        smoothed_val_f1 = smooth_ema(history.get('val_f1', []), alpha)
        if smoothed_val_f1:
            ax3.plot(epochs, smoothed_val_f1, 'orange', linewidth=2, label='Val F1', marker='d', markersize=4)
            ax3.set_title(f'{model_name} - F1 Score vs Epoch', fontsize=14, fontweight='bold', pad=20)
            ax3.set_xlabel('Epoch', fontsize=12, fontweight='bold')
            ax3.set_ylabel('F1 Score', fontsize=12, fontweight='bold')
            ax3.legend(frameon=True, shadow=True, fontsize=11)
            ax3.grid(True, alpha=0.3)
            ax3.tick_params(labelsize=10)

        # Learning Rate plot
        ax4 = fig.add_subplot(gs[1, 1])
        smoothed_lr = smooth_ema(history.get('learning_rates', []), alpha)
        if smoothed_lr:
            ax4.plot(epochs, smoothed_lr, 'purple', linewidth=2, label='Learning Rate', marker='x', markersize=6)
            ax4.set_title(f'{model_name} - Learning Rate vs Epoch', fontsize=14, fontweight='bold', pad=20)
            ax4.set_xlabel('Epoch', fontsize=12, fontweight='bold')
            ax4.set_ylabel('Learning Rate', fontsize=12, fontweight='bold')
            ax4.legend(frameon=True, shadow=True, fontsize=11)
            ax4.grid(True, alpha=0.3)
            ax4.tick_params(labelsize=10)
            ax4.set_yscale('log')

        plt.suptitle(f'{model_name} Training Progress', fontsize=18, fontweight='bold', y=0.98)
        save_path = f"{self.viz_dir}/{model_name}_individual_training_history.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
        plt.show()
        print(f"Individual training history saved: {save_path}")



    def plot_single_model_xai(self, model, model_name, test_loader, max_images=5):
        """Generate XAI visualizations for a single model using Grad-CAM++, Integrated Gradients, and LRP"""
        print(f"Generating XAI visualizations for {model_name}...")
        xai_visualizer = XAIVisualizer()

        # # Get sample images
        # sample_images, sample_labels = next(iter(test_loader))
        # sample_images = sample_images[:max_images].to(Config.DEVICE)
        # sample_labels = sample_labels[:max_images].numpy()

        # Get sample images
        sample_images = []
        sample_labels = []
        for i in range(Config.NUM_CLASSES):
            class_mask = np.where(test_loader.dataset.labels == i)[0]
            if len(class_mask) > 0:
                idx = np.random.choice(class_mask)
                image, label = test_loader.dataset[idx]
                if image.dim() == 4:
                    image = image.squeeze(0)
                sample_images.append(image.to(Config.DEVICE))
                sample_labels.append(label)


        for idx, (image, true_label) in enumerate(zip(sample_images, sample_labels)):
            try:
                # Create figure with larger size for clearer images
                fig = plt.figure(figsize=(24, 8))  # Standard size for clear visualization
                gs = fig.add_gridspec(1, 4, wspace=0.15, hspace=0.2)  # 4 columns: Original, Grad-CAM++, Integrated Gradients, LRP

                # Prepare original image
                image_np = image.detach().permute(1, 2, 0).cpu().numpy()
                image_np = image_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                image_np = np.clip(image_np, 0, 1)
                image_np = (image_np * 255).astype(np.uint8)

                # Plot original image
                ax_orig = fig.add_subplot(gs[0, 0])
                ax_orig.imshow(image_np)
                ax_orig.set_title(f'Original Image\nTrue: {Config.CLASS_LABELS[true_label]}',
                                 fontsize=14, fontweight='bold', pad=15)
                ax_orig.axis('off')

                # Forward pass for prediction
                model.eval()
                with torch.no_grad():
                    outputs = model(image.unsqueeze(0))
                    probabilities = torch.softmax(outputs, dim=1)
                    predicted_class = outputs.argmax(dim=1).item()
                    confidence = probabilities[0, predicted_class].item()
                    predicted_label = Config.CLASS_LABELS[predicted_class]

                # Generate Grad-CAM++ visualization
                gradcam_img, _ = xai_visualizer.grad_cam_plus_plus(model, image, predicted_class)
                ax_gradcam = fig.add_subplot(gs[0, 1])
                ax_gradcam.imshow(gradcam_img)
                ax_gradcam.set_title(f'{model_name}\nGrad-CAM++\nPred: {predicted_label}\nConf: {confidence:.2%}',
                                    fontsize=12, fontweight='bold', pad=10)
                ax_gradcam.axis('off')

                # Generate Integrated Gradients visualization
                ig_img, _ = xai_visualizer.integrated_gradients(model, image, predicted_class)
                ax_ig = fig.add_subplot(gs[0, 2])
                ax_ig.imshow(ig_img)
                ax_ig.set_title(f'{model_name}\nIntegrated Gradients', fontsize=12, fontweight='bold', pad=10)
                ax_ig.axis('off')

                # Generate LRP visualization
                lrp_img, _ = xai_visualizer.layer_wise_relevance_propagation(model, image, predicted_class)
                ax_lrp = fig.add_subplot(gs[0, 3])
                # ax_lrp.imshow(lrp_img)
                ax_lrp.imshow(cv2.cvtColor(lrp_img, cv2.COLOR_BGR2RGB))
                ax_lrp.set_title(f'{model_name}\nLRP', fontsize=12, fontweight='bold', pad=10)
                ax_lrp.axis('off')

                plt.suptitle(f'XAI Analysis for {model_name} - Image {idx+1}', fontsize=16, fontweight='bold', y=0.95)
                save_path = f"{self.viz_dir}/{model_name}_xai_image_{idx+1}.png"
                plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
                plt.show()
                print(f"XAI visualization saved: {save_path}")

            except Exception as e:
                print(f"Error generating XAI for {model_name}, image {idx+1}: {e}")
                continue

    def plot_confusion_matrix(self, results, model_name):
        """Enhanced confusion matrix with better spacing"""
        if 'true_labels' not in results or 'predictions' not in results:
            print(f"Error: Missing 'true_labels' or 'predictions' in results for {model_name}")
            return

        cm = confusion_matrix(results['true_labels'], results['predictions'])
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        # Create figure with better spacing
        fig, ax = plt.subplots(figsize=(12, 10))

        # Use better color scheme and formatting
        sns.heatmap(cm_normalized, annot=cm, fmt='d', cmap='Blues',
                    xticklabels=Config.CLASS_LABELS, yticklabels=Config.CLASS_LABELS,
                    cbar_kws={'label': 'Normalized Count', 'shrink': 0.8},
                    square=True, linewidths=0.5, annot_kws={'size': 12})

        # === Move colorbar label to the top ===
        cbar = ax.collections[0].colorbar
        cbar.ax.yaxis.set_label_position('left')   # keep label aligned left of bar
        cbar.set_label("Normalized Count", rotation=0, labelpad=15)
        cbar.ax.yaxis.set_label_coords(-1.2, 1.02)  # fine-tune position (x, y)

        ax.set_title(f'Confusion Matrix: {model_name}', fontsize=16, fontweight='bold', pad=25)
        ax.set_xlabel('Predicted Label', fontsize=14, fontweight='bold', labelpad=15)
        ax.set_ylabel('True Label', fontsize=14, fontweight='bold', labelpad=15)

        # Rotate labels for better readability
        plt.xticks(rotation=45, ha='right', fontsize=12)
        plt.yticks(rotation=0, fontsize=12)

        # Add performance metrics as text
        accuracy = accuracy_score(results['true_labels'], results['predictions'])
        f1_macro = f1_score(results['true_labels'], results['predictions'], average='macro')
        f1_weighted = f1_score(results['true_labels'], results['predictions'], average='weighted')

        metrics_text = f'Accuracy: {accuracy:.4f}\nF1 (Macro): {f1_macro:.4f}\nF1 (Weighted): {f1_weighted:.4f}'
        ax.text(1.15, 0.5, metrics_text, transform=ax.transAxes, fontsize=12,
                verticalalignment='center', bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.8))

        save_path = f"{self.viz_dir}/{model_name}_enhanced_confusion_matrix.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
        plt.show()
        print(f"Enhanced confusion matrix saved: {save_path}")





    def plot_roc_curves(self, results, model_name):
        """Plot ROC curves for multi-class classification."""
        import numpy as np
        """Enhanced ROC curves with better spacing and styling for multiclass"""
        if 'true_labels' not in results or 'probabilities' not in results:
            print(f"Error: Missing 'true_labels' or 'probabilities' in results for {model_name}")
            return

        fig, ax = plt.subplots(figsize=(12, 10))
        colors = plt.cm.Set3(np.linspace(0, 1, Config.NUM_CLASSES))
        probabilities = np.array(results['probabilities'])
        if len(results['true_labels']) == 0 or probabilities.size == 0 or probabilities.ndim != 2 or probabilities.shape[1] != Config.NUM_CLASSES:
            print(f"Warning: Invalid or empty probabilities shape {probabilities.shape} for {model_name}. Skipping ROC plot.")
            ax.text(0.5, 0.5, 'No data available for ROC', horizontalalignment='center', verticalalignment='center', fontsize=12)
            return
        colors = plt.cm.Set3(np.linspace(0, 1, Config.NUM_CLASSES))
        for i in range(Config.NUM_CLASSES):
            y_true_bin = (np.array(results['true_labels']) == i).astype(int)
            y_score = probabilities[:, i]

            fpr, tpr, _ = roc_curve(y_true_bin, y_score)
            roc_auc = auc(fpr, tpr)
            ax.plot(fpr, tpr, color=colors[i], linewidth=2,
                    label=f'{Config.CLASS_LABELS[i]} (AUC = {roc_auc:.4f})')

        # Diagonal line
        ax.plot([0, 1], [0, 1], 'k--', linewidth=2, alpha=0.6, label='Random Classifier')

        ax.set_title(f'{model_name} - ROC Curves', fontsize=16, fontweight='bold', pad=20)
        ax.set_xlabel('False Positive Rate', fontsize=14, fontweight='bold', labelpad=10)
        ax.set_ylabel('True Positive Rate', fontsize=14, fontweight='bold', labelpad=10)
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', frameon=True, shadow=True, fontsize=11)
        ax.grid(True, alpha=0.3)
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])

        # Compute micro-average ROC curve
        y_true_bin = label_binarize(results['true_labels'], classes=range(Config.NUM_CLASSES))
        fpr_micro, tpr_micro, _ = roc_curve(y_true_bin.ravel(), results['probabilities'].ravel())
        roc_auc_micro = auc(fpr_micro, tpr_micro)
        ax.plot(fpr_micro, tpr_micro, 'deeppink', linestyle=':', linewidth=4,
                label=f'Micro-average (AUC = {roc_auc_micro:.4f})')

        save_path = f"{self.viz_dir}/{model_name}_enhanced_roc_curves.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
        plt.show()
        print(f"Enhanced ROC curves saved: {save_path}")




    def plot_misclassified_images(self, misclassified, model_name):
        """Plot up to 4 misclassified images in a 2x2 grid with improved visibility."""
        import matplotlib.pyplot as plt
        import numpy as np

        if not misclassified:
            print(f"No misclassified images for {model_name}")
            return

        # Select up to 4 misclassified images
        misclassified = misclassified[:4]
        fig, axes = plt.subplots(2, 2, figsize=(10, 10))  # Slightly smaller size
        axes = axes.flatten()

        for idx, item in enumerate(misclassified):
            # image = item['image'].permute(1, 2, 0).cpu().numpy()  # Convert CHW -> HWC
            image = item['image']
            if isinstance(image, torch.Tensor):
                image = image.permute(1, 2, 0).cpu().numpy()  # CHW -> HWC
            elif isinstance(image, np.ndarray):
                if image.shape[0] in [1, 3]:  # If channel-first
                    image = np.transpose(image, (1, 2, 0))  # CHW -> HWC


            # Normalize for display
            if image.max() > 1.0:
                image = image / 255.0
            else:
                image = (image - image.min()) / (image.max() - image.min() + 1e-8)
            image = np.clip(image, 0, 1)

            # Grayscale vs RGB
            if image.shape[2] == 1:
                image = image.squeeze()
                axes[idx].imshow(image, cmap='gray')
            else:
                axes[idx].imshow(image)

            # Titles with color
            true_label = Config.CLASS_LABELS[item['true_label']]
            pred_label = Config.CLASS_LABELS[item['pred_label']]
            axes[idx].set_title(
                f"True: {true_label}\nPred: {pred_label}",
                fontsize=9,
                fontweight="bold",
                pad=10,
                color="black" #if true_label != pred_label else "darkgreen"
            )

            axes[idx].axis("off")
            # Add white border around each image
            for spine in axes[idx].spines.values():
                spine.set_edgecolor("lightgray")
                spine.set_linewidth(3)

        # Hide unused subplots (if <4 images)
        for idx in range(len(misclassified), 4):
            axes[idx].axis("off")

        plt.suptitle(
            f"Misclassified Images for {model_name}",
            fontsize=12,
            fontweight="bold",
            y=0.99,
            color="black"
        )
        plt.subplots_adjust(wspace=0.3, hspace=0.4)  # Increase padding between plots
        save_path = f"{self.viz_dir}/misclassified_{model_name}.png"
        plt.savefig(save_path, bbox_inches="tight", dpi=300, facecolor="white")
        plt.show()
        plt.close()
        print(f"Saved misclassified images plot for {model_name} at {save_path}")





    def plot_kfold_results(self, fold_results, model_name, test_loader):
        """Plot k-fold results: 2x2 grid per fold and 1x2 comparison across folds."""
        n_folds = len(fold_results)
        plt.style.use('seaborn-v0_8')  # Use modern Seaborn style

        # 2x2 Grid Plot for Each Fold
        for fold in range(n_folds):
            fig, axes = plt.subplots(2, 2, figsize=(12, 10))
            fig.suptitle(f'{model_name} - Fold {fold+1} Metrics', fontsize=16)
            history = fold_results[fold]['history']

            # Plot Validation Accuracy
            axes[0, 0].plot(history['val_acc'], label='Validation Accuracy', marker='s')
            axes[0, 0].set_title('Validation Accuracy')
            axes[0, 0].set_xlabel('Epoch')
            axes[0, 0].set_ylabel('Accuracy')
            axes[0, 0].legend()
            axes[0, 0].grid(True)

            # Plot Validation Loss
            axes[0, 1].plot(history['val_loss'], label='Validation Loss', marker='s')
            axes[0, 1].set_title('Validation Loss')
            axes[0, 1].set_xlabel('Epoch')
            axes[0, 1].set_ylabel('Loss')
            axes[0, 1].legend()
            axes[0, 1].grid(True)

            # Hide unused subplots
            axes[1, 0].set_visible(False)
            axes[1, 1].set_visible(False)

            plt.tight_layout()
            plt.show()

        # 1x2 Comparison Plot Across Folds
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        fig.suptitle(f'{model_name} - Cross-Fold Comparison', fontsize=16)

        # Plot Validation Accuracy Across Folds
        for fold in range(n_folds):
            history = fold_results[fold]['history']
            axes[0].plot(history['val_acc'], label=f'Fold {fold+1}', marker='s')
        axes[0].set_title('Validation Accuracy Across Folds')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Accuracy')
        axes[0].legend()
        axes[0].grid(True)

        # Plot Validation Loss Across Folds
        for fold in range(n_folds):
            history = fold_results[fold]['history']
            axes[1].plot(history['val_loss'], label=f'Fold {fold+1}', marker='s')
        axes[1].set_title('Validation Loss Across Folds')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Loss')
        axes[1].legend()
        axes[1].grid(True)

        plt.tight_layout()
        plt.show()


    def plot_model_results(self, model_name, fold_results, test_loader):
        """Plot model results: ROC curves, confusion matrix, misclassified images, XAI, and k-fold results."""
        print(f"Generating plots for {model_name}")

        # Plot single-model results using first fold's result
        first_result = fold_results[0]['result']
        self.plot_roc_curves(first_result, model_name)
        self.plot_confusion_matrix(first_result, model_name)
        self.plot_misclassified_images(first_result['misclassified'], model_name)

        # Recreate model for XAI plotting
        model = ModelFactory.create_model(
            model_name,
            num_classes=Config.NUM_CLASSES,
            dropout_rate=0.5,  # Use default or pass from hyperparameters
            hidden_dim_multiplier=0.5
        ).to(Config.DEVICE)
        self.plot_single_model_xai(model, model_name, test_loader)

        # Plot k-fold results
        self.plot_kfold_results(fold_results, model_name, test_loader)

        print("Completed plotting k-fold results")

        # Clean up
        del model, fold_results, first_result
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        gc.collect()
        # print(f"Plotting memory cleared: {torch.cuda.memory_summary()}")






    def generate_comprehensive_report(self, single_results, ensemble_results, best_ensemble):
        """Generate a comprehensive visual report"""
        # fig = plt.figure(figsize=(24, 16))
        fig = plt.figure(figsize=(30, 20))
        gs = fig.add_gridspec(3, 4, hspace=0.4, wspace=0.3, top=0.92, bottom=0.08, left=0.05, right=0.95)

        # Title
        fig.suptitle('Fish Species Classification - Comprehensive Analysis Report',
                    fontsize=24, fontweight='bold', y=0.96)

        # 1. Model Performance Comparison
        ax1 = fig.add_subplot(gs[0, :2])
        model_names = list(single_results.keys())
        accuracies = [single_results[name]['accuracy'] for name in model_names]
        f1_scores = [single_results[name]['f1_macro'] for name in model_names]

        x = np.arange(len(model_names))
        width = 0.35

        bars1 = ax1.bar(x - width/2, accuracies, width, label='Accuracy', alpha=0.8, color='skyblue')
        bars2 = ax1.bar(x + width/2, f1_scores, width, label='F1 Score', alpha=0.8, color='lightcoral')

        ax1.set_xlabel('Models', fontweight='bold', fontsize=12)
        ax1.set_ylabel('Score', fontweight='bold', fontsize=12)
        ax1.set_title('Individual Model Performance', fontweight='bold', fontsize=14, pad=15)
        ax1.set_xticks(x)
        ax1.set_xticklabels(model_names, rotation=45, ha='right')
        ax1.legend(fontsize=11)
        ax1.grid(True, alpha=0.3)
        ax1.set_ylim(0, 1.1)

        # Add value labels on bars
        for bar in bars1 + bars2:
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{height:.4f}', ha='center', va='bottom', fontsize=9)

        # 2. Best vs Worst Model Comparison
        ax2 = fig.add_subplot(gs[0, 2:])
        if ensemble_results and best_ensemble:
            best_single_f1 = max(f1_scores)
            worst_single_f1 = min(f1_scores)
            best_ensemble_f1 = best_ensemble[1]['f1']

            categories = ['Worst Single', 'Best Single', 'Best Ensemble']
            values = [worst_single_f1, best_single_f1, best_ensemble_f1]
            colors = ['lightcoral', 'lightblue', 'gold']

            bars = ax2.bar(categories, values, color=colors, alpha=0.8)
            ax2.set_ylabel('F1 Score', fontweight='bold', fontsize=12)
            ax2.set_title('Performance Comparison', fontweight='bold', fontsize=14, pad=15)
            ax2.grid(True, alpha=0.3)
            ax2.set_ylim(0, max(values) * 1.1)

            for bar, value in zip(bars, values):
                ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                        f'{value:.4f}', ha='center', va='bottom', fontweight='bold')

        # 3. Per-class Performance Heatmap
        ax3 = fig.add_subplot(gs[1, :2])
        per_class_f1 = []
        for model_name in model_names:
            per_class_f1.append(single_results[model_name]['f1_per_class'])

        per_class_f1 = np.array(per_class_f1)
        im = ax3.imshow(per_class_f1, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)

        ax3.set_xticks(range(len(Config.CLASS_LABELS)))
        ax3.set_xticklabels(Config.CLASS_LABELS, rotation=45, ha='right')
        ax3.set_yticks(range(len(model_names)))
        ax3.set_yticklabels(model_names)
        ax3.set_title('Per-Class F1 Scores Heatmap', fontweight='bold', fontsize=14, pad=15)

        # Add colorbar
        cbar = plt.colorbar(im, ax=ax3, shrink=0.8)
        cbar.set_label('F1 Score', rotation=270, labelpad=15)

        # Add text annotations
        for i in range(len(model_names)):
            for j in range(len(Config.CLASS_LABELS)):
                text_color = 'white' if per_class_f1[i, j] < 0.5 else 'black'
                ax3.text(j, i, f'{per_class_f1[i, j]:.2f}',
                        ha="center", va="center", color=text_color, fontsize=8, fontweight='bold')

        # 4. Ensemble Methods Performance
        ax4 = fig.add_subplot(gs[1, 2:])
        if ensemble_results:
            ensemble_items = list(ensemble_results.items())
            ensemble_items.sort(key=lambda x: x[1]['f1'], reverse=True)
            top_ensembles = ensemble_items[:8]

            ensemble_names = [name.split('_')[-1] for name, _ in top_ensembles]
            ensemble_f1s = [result['f1'] for _, result in top_ensembles]

            bars = ax4.barh(range(len(ensemble_names)), ensemble_f1s, alpha=0.8, color='lightgreen')
            ax4.set_yticks(range(len(ensemble_names)))
            ax4.set_yticklabels(ensemble_names)
            ax4.set_xlabel('F1 Score', fontweight='bold', fontsize=12)
            ax4.set_title('Top Ensemble Methods', fontweight='bold', fontsize=14, pad=15)
            ax4.grid(True, alpha=0.3)

            for bar, value in zip(bars, ensemble_f1s):
                ax4.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height()/2,
                        f'{value:.4f}', ha='left', va='center', fontsize=9, fontweight='bold')

        # 5. Summary Statistics Table
        ax5 = fig.add_subplot(gs[2, :])
        ax5.axis('off')

        # Create summary data
        summary_data = []
        for model_name in model_names:
            result = single_results[model_name]
            summary_data.append([
                model_name,
                f"{result['accuracy']:.4f}",
                f"{result['f1_macro']:.4f}",
                f"{result['f1_weighted']:.4f}",
                f"{result['precision_macro']:.4f}",
                f"{result['recall_macro']:.4f}"
            ])

        if ensemble_results and best_ensemble:
            best_result = best_ensemble[1]
            summary_data.append([
                f"Best Ensemble\n({best_ensemble[0]})",
                f"{best_result['accuracy']:.4f}",
                f"{best_result['f1']:.4f}",
                "N/A",
                "N/A",
                "N/A"
            ])

        columns = ['Model', 'Accuracy', 'F1 (Macro)', 'F1 (Weighted)', 'Precision', 'Recall']

        table = ax5.table(cellText=summary_data, colLabels=columns, loc='center', cellLoc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 2)

        # Style the table
        for i in range(len(columns)):
            table[(0, i)].set_facecolor('#4CAF50')
            table[(0, i)].set_text_props(weight='bold', color='white')

        for i in range(1, len(summary_data) + 1):
            for j in range(len(columns)):
                if i % 2 == 0:
                    table[(i, j)].set_facecolor('#f0f0f0')
                else:
                    table[(i, j)].set_facecolor('white')

        ax5.set_title('Model Performance Summary', fontweight='bold', fontsize=16, pad=20)

        save_path = f"{self.viz_dir}/comprehensive_report.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
        plt.show()
        print(f"Comprehensive report saved: {save_path}")
        return save_path





    def plot_model_comparison(self, single_results, ensemble_results):
        """Enhanced model comparison with better spacing"""
        # fig = plt.figure(figsize=(20, 16))
        fig = plt.figure(figsize=(30, 20))
        gs = fig.add_gridspec(2, 2, hspace=0.4, wspace=0.3, top=0.92, bottom=0.08)

        model_names = list(single_results.keys())
        accuracies = [single_results[name]['accuracy'] for name in model_names]
        f1_scores = [single_results[name]['f1_macro'] for name in model_names]
        f1_weighted = [single_results[name]['f1_weighted'] for name in model_names]
        # losses = [single_results[name]['loss'] for name in model_names]
        # Use f1_macro as substitute since loss is not available in single_results
        losses = [1 - single_results[name]['f1_macro'] for name in model_names]  # Convert F1 to loss-like metric

        x = np.arange(len(model_names))
        width = 0.2

        ax1 = fig.add_subplot(gs[0, 0])
        bars1 = ax1.bar(x - width*1.5, accuracies, width, label='Accuracy', alpha=0.8, color='skyblue')
        bars2 = ax1.bar(x - width/2, f1_scores, width, label='F1 (Macro)', alpha=0.8, color='lightcoral')
        bars3 = ax1.bar(x + width/2, f1_weighted, width, label='F1 (Weighted)', alpha=0.8, color='lightgreen')
        bars4 = ax1.bar(x + width*1.5, losses, width, label='Loss', alpha=0.8, color='salmon')

        ax1.set_xlabel('Models', fontweight='bold', fontsize=12)
        ax1.set_ylabel('Score', fontweight='bold', fontsize=12)
        ax1.set_title('Individual Model Performance', fontweight='bold', fontsize=14, pad=20)
        ax1.set_xticks(x)
        ax1.set_xticklabels(model_names, rotation=45, ha='right', fontsize=10)
        ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
        ax1.grid(True, alpha=0.3)
        ax1.set_ylim(0, max(max(accuracies), max(f1_scores), max(f1_weighted), max(losses)) * 1.1)

        if ensemble_results:
            ensemble_items = list(ensemble_results.items())
            ensemble_items.sort(key=lambda x: x[1]['f1'], reverse=True)
            top_ensembles = ensemble_items[:10]

            ensemble_names = [name.split('_')[-1] for name, _ in top_ensembles]
            ensemble_f1s = [result['f1'] for _, result in top_ensembles]
            ensemble_accs = [result['accuracy'] for _, result in top_ensembles]
            ensemble_losses = [result['loss'] for _, result in top_ensembles]

            x = np.arange(len(ensemble_names))
            ax2 = fig.add_subplot(gs[0, 1])
            ax2.bar(x - width, ensemble_accs, width, label='Accuracy', alpha=0.8, color='skyblue')
            ax2.bar(x, ensemble_f1s, width, label='F1 (Macro)', alpha=0.8, color='lightcoral')
            ax2.bar(x + width, ensemble_losses, width, label='Loss', alpha=0.8, color='salmon')

            ax2.set_xticks(x)
            ax2.set_xticklabels([f"{name}" for name in ensemble_names], rotation=45, ha='right', fontsize=10)
            ax2.set_xlabel('Ensemble Methods', fontweight='bold', fontsize=12)
            ax2.set_ylabel('Score', fontweight='bold', fontsize=12)
            ax2.set_title('Top Ensemble Methods', fontweight='bold', fontsize=14, pad=20)
            ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
            ax2.grid(True, alpha=0.3)
            ax2.set_ylim(0, max(max(ensemble_accs), max(ensemble_f1s), max(ensemble_losses)) * 1.1)

        if single_results:
            per_class_f1 = []
            for model_name in model_names:
                per_class_f1.append(single_results[model_name]['f1_per_class'])

            per_class_f1 = np.array(per_class_f1)
            ax3 = fig.add_subplot(gs[1, 0])
            im = ax3.imshow(per_class_f1, cmap='RdYlBu_r', aspect='auto', vmin=0, vmax=1)

            ax3.set_xticks(range(len(Config.CLASS_LABELS)))
            ax3.set_xticklabels(Config.CLASS_LABELS, rotation=45, ha='right', fontsize=10)
            ax3.set_yticks(range(len(model_names)))
            ax3.set_yticklabels(model_names, fontsize=10)
            ax3.set_title('Per-Class F1 Scores', fontweight='bold', fontsize=14, pad=20)

            cbar = plt.colorbar(im, ax=ax3, shrink=0.8)
            cbar.set_label('F1 Score', rotation=270, labelpad=15)

            for i in range(len(model_names)):
                for j in range(len(Config.CLASS_LABELS)):
                    text_color = 'white' if per_class_f1[i, j] < 0.5 else 'black'
                    ax3.text(j, i, f'{per_class_f1[i, j]:.2f}',
                            ha="center", va="center", color=text_color, fontsize=8)

        if ensemble_results:
            best_single_f1 = max(f1_scores)
            best_single_name = model_names[f1_scores.index(best_single_f1)]
            best_ensemble_f1 = max([result['f1'] for result in ensemble_results.values()])
            best_ensemble_name = max(ensemble_results.items(), key=lambda x: x[1]['f1'])[0]

            ax4 = fig.add_subplot(gs[1, 1])
            categories = ['Best Single\nModel', 'Best Ensemble']
            values = [best_single_f1, best_ensemble_f1]
            colors = ['lightblue', 'gold']

            bars = ax4.bar(categories, values, color=colors, alpha=0.8, width=0.6)
            ax4.set_ylabel('F1 Score', fontweight='bold', fontsize=12)
            ax4.set_title('Single vs Ensemble Comparison', fontweight='bold', fontsize=14, pad=20)
            ax4.grid(True, alpha=0.3)
            ax4.set_ylim(0, 1)

            for bar, value in zip(bars, values):
                ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                        f'{value:.4f}', ha='center', va='bottom', fontweight='bold', fontsize=12)

            improvement = ((best_ensemble_f1 - best_single_f1) / best_single_f1) * 100
            ax4.text(0.5, 0.5, f'Improvement:\n{improvement:.2f}%',
                    transform=ax4.transAxes, ha='center', va='center',
                    fontsize=14, fontweight='bold',
                    bbox=dict(boxstyle="round,pad=0.5", facecolor="yellow", alpha=0.8))

        plt.suptitle('Fish Species Classification - Model Comparison Analysis',
                    fontsize=18, fontweight='bold', y=0.96)
        save_path = f"{self.viz_dir}/enhanced_model_comparison.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
        plt.show()
        print(f"Enhanced model comparison saved: {save_path}")
        return save_path




    def plot_lrp_grid(self, single_models, test_loader):
        print("Generating LRP grid visualization for TP, TN, FP, FN...")
        xai_visualizer = XAIVisualizer()

        # Compute predictions and confusion matrix
        model = list(single_models.values())[0]
        model.eval()
        true_labels = []
        pred_labels = []
        with torch.no_grad():
            for batch in test_loader:
                # images, labels = batch['image'].to(Config.DEVICE), batch['label'].to(Config.DEVICE)
                # images, labels = batch[0].to(Config.DEVICE), batch[1].to(Config.DEVICE)  # Access tuple elements
                # Make it consistent with your data loader format:
                if isinstance(batch, dict):
                    images, labels = batch['image'].to(Config.DEVICE), batch['label'].to(Config.DEVICE)
                elif isinstance(batch, (list, tuple)) and len(batch) == 2:
                    images, labels = batch[0].to(Config.DEVICE), batch[1].to(Config.DEVICE)
                else:
                    raise ValueError(f"Unexpected batch format: {type(batch)}")

                outputs = model(images)
                preds = outputs.argmax(dim=1) if outputs.dim() > 1 else (outputs > 0.5).float()
                true_labels.extend(labels.cpu().numpy())
                pred_labels.extend(preds.cpu().numpy())

        cm = confusion_matrix(true_labels, pred_labels)
        classes = Config.CLASS_LABELS
        tp = np.diag(cm)
        fp = cm.sum(axis=0) - tp
        fn = cm.sum(axis=1) - tp
        tn = cm.sum() - (fp + fn + tp)
        metrics = [tp, tn, fp, fn]

        # Collect one image per class per category (TP, TN, FP, FN)
        sample_images = {cls: {"TP": None, "TN": None, "FP": None, "FN": None} for cls in range(5)}
        with torch.no_grad():
            for batch in test_loader:
                # images, labels = batch['image'].to(Config.DEVICE), batch['label'].to(Config.DEVICE)
                # images, labels = batch[0].to(Config.DEVICE), batch[1].to(Config.DEVICE)  # Access tuple elements
                # Make it consistent with your data loader format:
                if isinstance(batch, dict):
                    images, labels = batch['image'].to(Config.DEVICE), batch['label'].to(Config.DEVICE)
                elif isinstance(batch, (list, tuple)) and len(batch) == 2:
                    images, labels = batch[0].to(Config.DEVICE), batch[1].to(Config.DEVICE)
                else:
                    raise ValueError(f"Unexpected batch format: {type(batch)}")



                outputs = model(images)
                preds = outputs.argmax(dim=1) if outputs.dim() > 1 else (outputs > 0.5).float()
                for img, true, pred in zip(images, labels, preds):
                    cls = int(true.item())
                    pred_cls = int(pred.item())
                    for i in range(5):  # Check for each class
                        category = {
                            (1, 1): "TP",  # Predicted class i, true class i
                            (0, 0): "TN",  # Predicted not i, true not i
                            (1, 0): "FP",  # Predicted i, true not i
                            (0, 1): "FN"   # Predicted not i, true i
                        }[(1 if pred_cls == i else 0, 1 if cls == i else 0)]
                        if sample_images[cls][category] is None:
                            sample_images[cls][category] = (img, true)
                    if all(sample_images[c][cat] is not None for c in range(5) for cat in ["TP", "TN", "FP", "FN"]):
                        break
                if all(sample_images[c][cat] is not None for c in range(5) for cat in ["TP", "TN", "FP", "FN"]):
                    break

        # Visualization
        fig, axes = plt.subplots(5, 4, figsize=(30, 35))  # Large figure for clarity
        for i in range(5):  # Classes
            for j, category in enumerate(["TP", "TN", "FP", "FN"]):
                ax = axes[i, j]
                if sample_images[i][category] is not None:
                    img, true = sample_images[i][category]
                    img = img.squeeze().cpu()
                    lrp_img, _ = xai_visualizer.layer_wise_relevance_propagation(model, img, i)
                    img = img.permute(1, 2, 0).numpy()
                    img = np.clip(img, 0, 1)
                    img = img[..., [2, 1, 0]] if img.shape[2] == 3 else img  # Convert BGR to RGB
                    img = (img * 255).astype(np.uint8)
                    ax.imshow(img)
                    lrp_img = (lrp_img - lrp_img.min()) / (lrp_img.max() - lrp_img.min() + 1e-8)  # Normalize heatmap
                    ax.imshow(lrp_img, cmap='jet', alpha=0.4, vmin=0, vmax=np.percentile(lrp_img, 95))
                    ax.set_title(f'{classes[i]} - {category}: {metrics[j][i]}', fontsize=16, fontweight='bold', pad=10)
                ax.axis('off')

        plt.tight_layout()
        save_path = f"{self.viz_dir}/lrp_fish_metrics_grid.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
        plt.show()
        print(f"LRP grid visualization saved: {save_path}")




    def plot_comparative_xai(self, single_models, ensemble_results, test_loader, max_images=5):
        """Generate comparative XAI visualizations with Grad-CAM++, Integrated Gradients, and LRP in the same row"""
        print(f"Generating comparative XAI visualizations for {max_images} images...")
        xai_visualizer = XAIVisualizer()

        # Get sample images
        # sample_images, sample_labels = next(iter(test_loader))
        # sample_images = sample_images[:max_images].to(Config.DEVICE)
        # sample_labels = sample_labels[:max_images].numpy()
        sample_images = []
        sample_labels = []
        for i in range(Config.NUM_CLASSES):
            class_mask = np.where(test_loader.dataset.labels == i)[0]
            if len(class_mask) > 0:
                idx = np.random.choice(class_mask)
                image, label = test_loader.dataset[idx]
                if image.dim() == 4:
                    image = image.squeeze(0)
                sample_images.append(image.to(Config.DEVICE))
                sample_labels.append(label)
        sample_images = sample_images[:max_images]
        sample_labels = sample_labels[:max_images]

        # Get best ensemble if available
        best_ensemble_name = max(ensemble_results.items(), key=lambda x: x[1]['f1'])[0] if ensemble_results else None
        best_ensemble = ensemble_results.get(best_ensemble_name, None) if best_ensemble_name else None

        for idx, (image, true_label) in enumerate(zip(sample_images, sample_labels)):
            try:
                # Adjusted figure size with more height per row to prevent text cropping
                num_rows = len(single_models) + (1 if best_ensemble else 0)
                fig = plt.figure(figsize=(36, 14 * num_rows))  # Increased width and height per row
                gs = fig.add_gridspec(num_rows, 4, wspace=0.3, hspace=0.5)  # Increased wspace and hspace for better spacing

                # Prepare original image
                image_np = image.detach().permute(1, 2, 0).cpu().numpy()
                image_np = image_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                image_np = np.clip(image_np, 0, 1)
                image_np = (image_np * 255).astype(np.uint8)

                # Plot original image spanning all rows in first column
                ax_orig = fig.add_subplot(gs[:, 0])
                ax_orig.imshow(image_np)
                ax_orig.set_title(f'Original Image\nTrue: {Config.CLASS_LABELS[true_label]}',
                                fontsize=16, fontweight='bold', pad=30)  # Increased pad for title
                ax_orig.axis('off')

                # Plot XAI for single models
                for row, (model_name, model) in enumerate(single_models.items()):
                    model.eval()
                    with torch.no_grad():
                        outputs = model(image.unsqueeze(0))
                        probabilities = torch.softmax(outputs, dim=1)
                        predicted_class = outputs.argmax(dim=1).item()
                        confidence = probabilities[0, predicted_class].item()
                        predicted_label = Config.CLASS_LABELS[predicted_class]

                    # Generate Grad-CAM++ visualization
                    gradcam_img, _ = xai_visualizer.grad_cam_plus_plus(model, image, predicted_class)

                    # Plot Grad-CAM++ in second column
                    ax_gradcam = fig.add_subplot(gs[row, 1])
                    ax_gradcam.imshow(gradcam_img)
                    ax_gradcam.set_title(f'{model_name}\nGrad-CAM++\nPred: {predicted_label}\nConf: {confidence:.2%}',
                                        fontsize=14, fontweight='bold', pad=30)  # Increased pad
                    ax_gradcam.axis('off')

                    # Generate Integrated Gradients visualization
                    ig_img, _ = xai_visualizer.integrated_gradients(model, image, predicted_class)

                    # Plot Integrated Gradients in third column
                    ax_ig = fig.add_subplot(gs[row, 2])
                    ax_ig.imshow(ig_img)
                    ax_ig.set_title(f'{model_name}\nIntegrated Gradients', fontsize=14, fontweight='bold', pad=30)
                    ax_ig.axis('off')

                    # Generate LRP visualization
                    lrp_img, _ = xai_visualizer.layer_wise_relevance_propagation(model, image, predicted_class)

                    # Plot LRP in fourth column
                    ax_lrp = fig.add_subplot(gs[row, 3])
                    # ax_lrp.imshow(lrp_img)
                    ax_lrp.imshow(cv2.cvtColor(lrp_img, cv2.COLOR_BGR2RGB))
                    ax_lrp.set_title(f'{model_name}\nLRP', fontsize=14, fontweight='bold', pad=30)
                    ax_lrp.axis('off')

                # Plot XAI for best ensemble (if available)
                if best_ensemble:
                    ensemble_models = best_ensemble['models']
                    with torch.no_grad():
                        model_probs = []
                        for model_name in ensemble_models:
                            model = single_models[model_name]
                            outputs = model(image.unsqueeze(0))
                            probs = torch.softmax(outputs, dim=1)
                            model_probs.append(probs)
                        model_probs = torch.stack(model_probs, dim=1)

                        # Load learnable ensemble model if applicable
                        if 'learnable_weighted' in best_ensemble_name:
                            ensemble_model = LearnableWeightedEnsemble(
                                num_models=len(ensemble_models),
                                num_classes=Config.NUM_CLASSES
                            ).to(Config.DEVICE)
                            # ensemble_model.load_state_dict(
                            #     torch.load(f"{Config.OUTPUT_DIR}/models/learnable_ensemble_{'+'.join(ensemble_models)}.pt")
                            # )
                            # PROBLEM: This line tries to load a model that may not exist
                            # FIX: Add error handling and fallback
                            try:
                                model_path = f"{Config.OUTPUT_DIR}/models/learnable_ensemble_{'+'.join(ensemble_models)}.pt"
                                if os.path.exists(model_path):
                                    ensemble_model.load_state_dict(torch.load(model_path, map_location=Config.DEVICE))
                                else:
                                    # Fallback to simple weighted average if no saved learnable model
                                    weights = best_ensemble.get('weights', np.ones(len(ensemble_models)) / len(ensemble_models))
                                    outputs = torch.sum(model_probs * torch.tensor(weights, device=Config.DEVICE).view(1, -1, 1), dim=1)
                                    predicted_class = outputs.argmax(dim=1).item()
                                    confidence = torch.softmax(outputs, dim=1)[0, predicted_class].item()
                            except Exception as e:
                                print(f"Error loading learnable ensemble model: {e}, using simple averaging")
                                weights = best_ensemble.get('weights', np.ones(len(ensemble_models)) / len(ensemble_models))
                                outputs = torch.sum(model_probs * torch.tensor(weights, device=Config.DEVICE).view(1, -1, 1), dim=1)
                                predicted_class = outputs.argmax(dim=1).item()
                                confidence = torch.softmax(outputs, dim=1)[0, predicted_class].item()




                            ensemble_model.eval()
                            # outputs, _ = ensemble_model(model_probs)
                            # TO THIS (ensure correct input format):
                            if model_probs.dim() == 3:  # Already correct format
                                outputs, learned_weights = ensemble_model(model_probs)
                            else:  # Need to reshape
                                model_probs_reshaped = model_probs.view(1, len(ensemble_models), Config.NUM_CLASSES)
                                outputs, learned_weights = ensemble_model(model_probs_reshaped)

                        else:
                            weights = best_ensemble.get('weights', np.ones(len(ensemble_models)) / len(ensemble_models))
                            outputs = torch.sum(model_probs * torch.tensor(weights, device=Config.DEVICE).view(1, -1, 1), dim=1)

                        predicted_class = outputs.argmax(dim=1).item()
                        confidence = torch.softmax(outputs, dim=1)[0, predicted_class].item()
                        predicted_label = Config.CLASS_LABELS[predicted_class]

                    # Approximate ensemble Grad-CAM++ by averaging
                    gradcam_imgs = []
                    for model_name in ensemble_models:
                        gradcam_img, _ = xai_visualizer.grad_cam_plus_plus(single_models[model_name], image, predicted_class)
                        gradcam_imgs.append(gradcam_img)
                    ensemble_gradcam = np.mean(gradcam_imgs, axis=0).astype(np.uint8)

                    # Plot ensemble Grad-CAM++ in second column
                    ax_gradcam = fig.add_subplot(gs[num_rows-1, 1])
                    ax_gradcam.imshow(ensemble_gradcam)
                    ax_gradcam.set_title(f'Best Ensemble ({best_ensemble_name.split("_")[-1]})\nGrad-CAM++\nPred: {predicted_label}\nConf: {confidence:.2%}',
                                        fontsize=14, fontweight='bold', pad=30)
                    ax_gradcam.axis('off')

                    # Approximate ensemble Integrated Gradients by averaging
                    ig_imgs = []
                    for model_name in ensemble_models:
                        ig_img, _ = xai_visualizer.integrated_gradients(single_models[model_name], image, predicted_class)
                        ig_imgs.append(ig_img)
                    ensemble_ig = np.mean(ig_imgs, axis=0).astype(np.uint8)

                    # Plot ensemble Integrated Gradients in third column
                    ax_ig = fig.add_subplot(gs[num_rows-1, 2])
                    ax_ig.imshow(ensemble_ig)
                    ax_ig.set_title(f'Best Ensemble ({best_ensemble_name.split("_")[-1]})\nIntegrated Gradients', fontsize=14, fontweight='bold', pad=30)
                    ax_ig.axis('off')

                    # Approximate ensemble LRP by averaging
                    lrp_imgs = []
                    for model_name in ensemble_models:
                        lrp_img, _ = xai_visualizer.layer_wise_relevance_propagation(single_models[model_name], image, predicted_class)
                        lrp_imgs.append(lrp_img)
                    ensemble_lrp = np.mean(lrp_imgs, axis=0).astype(np.uint8)

                    # Plot ensemble LRP in fourth column
                    ax_lrp = fig.add_subplot(gs[num_rows-1, 3])
                    ax_lrp.imshow(ensemble_lrp)
                    ax_lrp.set_title(f'Best Ensemble ({best_ensemble_name.split("_")[-1]})\nLRP', fontsize=14, fontweight='bold', pad=30)
                    ax_lrp.axis('off')

                # Adjusted suptitle with increased padding
                plt.suptitle(f'Comparative XAI Analysis - Image {idx+1}', fontsize=18, fontweight='bold', y=0.97)
                plt.tight_layout(rect=[0, 0, 1, 0.95])  # Added rect to ensure suptitle is not cropped
                save_path = f"{self.viz_dir}/comparative_xai_image_{idx+1}.png"
                plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
                plt.show()
                print(f"Comparative XAI visualization saved: {save_path}")

            except Exception as e:
                print(f"Error generating comparative XAI for image {idx+1}: {e}")
                continue



# Purpose: Implement corrected Grad-CAM++ and Gradiant_Base & LRP with proper tensor handling.

class XAIVisualizer:
    @staticmethod
    def grad_cam_plus_plus(model, image, target_class):
        """Implement Grad-CAM++ for visualizing important image regions."""
        if image.dim() == 4:
            image = image.squeeze(0)  # Normalize to (C, H, W)

        model.eval()

        # Find the last convolutional layer
        target_layer = None
        for name, module in model.named_modules():  # Corrected: removed redundant reversed
            if isinstance(module, nn.Conv2d):
                target_layer = module

        if target_layer is None:
            raise ValueError("No convolutional layer found in the model")

        # Hook to capture gradients and activations
        gradients = []
        activations = []

        def forward_hook(module, input, output):
            activations.append(output.detach())

        def backward_hook(module, grad_input, grad_output):
            gradients.append(grad_output[0].detach())

        # Register hooks
        forward_handle = target_layer.register_forward_hook(forward_hook)
        backward_handle = target_layer.register_backward_hook(backward_hook)

        try:
            # Forward pass
            image_tensor = image.unsqueeze(0).requires_grad_(True)
            output = model(image_tensor)

            # Backward pass
            model.zero_grad()
            score = output[0, target_class]
            score.backward()

            # Get gradients and activations
            if not gradients or not activations:
                raise ValueError("No gradients or activations captured")

            grad = gradients[0]  # Shape: (1, C, H, W)
            act = activations[0]  # Shape: (1, C, H, W)

            # Grad-CAM++ calculations
            alpha_num = grad.pow(2)
            alpha_denom = 2.0 * grad.pow(2) + (act * grad.pow(3)).sum(dim=(2, 3), keepdim=True)
            alpha_denom = torch.where(alpha_denom != 0, alpha_denom, torch.ones_like(alpha_denom))
            alpha = alpha_num / alpha_denom

            weights = (alpha * F.relu(grad)).sum(dim=(2, 3))  # Shape: (1, C)
            cam = (weights.unsqueeze(-1).unsqueeze(-1) * act).sum(dim=1, keepdim=True)  # Shape: (1, 1, H, W)
            cam = F.relu(cam)

            # Normalize
            cam = cam - cam.min()
            cam = cam / (cam.max() + 1e-8)

            # Resize to input image size
            cam = F.interpolate(cam, size=(224, 224), mode='bilinear', align_corners=False)
            cam = cam.squeeze().detach().cpu().numpy()

            # Apply colormap
            cam = (cam * 255).astype(np.uint8)
            heatmap = cv2.applyColorMap(cam, cv2.COLORMAP_JET)

            # Convert original image for visualization
            image_np = image.detach().permute(1, 2, 0).cpu().numpy()
            image_np = image_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
            image_np = np.clip(image_np, 0, 1)
            image_np = (image_np * 255).astype(np.uint8)

            # Superimpose heatmap
            superimposed_img = cv2.addWeighted(image_np, 0.6, heatmap, 0.4, 0)
            # superimposed_img = cv2.addWeighted(image_np[:, :, ::-1], 0.6, heatmap, 0.4, 0)  # Convert image_np to BGR for cv2
            # superimposed_img = superimposed_img[:, :, ::-1]  # Back to RGB for display

        finally:
            # Remove hooks
            forward_handle.remove()
            backward_handle.remove()

        return superimposed_img, heatmap




    @staticmethod
    def integrated_gradients(model, image, target_class):
        """Integrated Gradients implementation."""
        if image.dim() == 4:
            image = image.squeeze(0)  # Normalize to (C, H, W)


        model.eval()

        # Detach the image first to avoid gradient issues
        image_detached = image.detach()

        # Create baseline (black image)
        baseline = torch.zeros_like(image_detached)

        # Generate path from baseline to image
        num_steps = 30
        alphas = torch.linspace(0, 1, num_steps).to(image_detached.device)

        gradients = []
        for alpha in alphas:
            interpolated = baseline + alpha * (image_detached - baseline)
            interpolated = interpolated.requires_grad_(True)

            output = model(interpolated.unsqueeze(0))
            score = output[0, target_class]

            # Clear gradients
            model.zero_grad()
            if interpolated.grad is not None:
                interpolated.grad.zero_()

            score.backward()

            # Detach gradient before storing
            if interpolated.grad is not None:
                gradients.append(interpolated.grad.detach().clone())

            # Clear the gradient to free memory
            interpolated.grad = None

        if not gradients:
            # Fallback: simple gradient
            image_grad = image_detached.requires_grad_(True)
            output = model(image_grad.unsqueeze(0))
            score = output[0, target_class]
            model.zero_grad()
            score.backward()
            gradients = [image_grad.grad.detach().clone()]

        # Average gradients
        avg_gradients = torch.stack(gradients).mean(dim=0)

        # Compute integrated gradients
        integrated_gradients = (image_detached - baseline) * avg_gradients

        # Sum across color channels
        relevance = integrated_gradients.abs().sum(dim=0).cpu().numpy()

        # Normalize
        relevance = relevance / (relevance.max() + 1e-8)

        # Apply colormap
        relevance = (relevance * 255).astype(np.uint8)
        heatmap = cv2.applyColorMap(relevance, cv2.COLORMAP_JET)

        # Convert original image for visualization
        image_np = image_detached.permute(1, 2, 0).cpu().numpy()
        image_np = image_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        image_np = np.clip(image_np, 0, 1)
        image_np = (image_np * 255).astype(np.uint8)

        # Superimpose heatmap
        superimposed_img = cv2.addWeighted(image_np, 0.6, heatmap, 0.4, 0)

        return superimposed_img, heatmap





    @staticmethod
    def layer_wise_relevance_propagation(
        model,
        image,
        target_class=None,
        device="cuda",
        input_size=None,
        epsilon=1e-6,
        imagenet_norm=True
    ):


        model.eval()
        model.to(device)

        # Ensure correct input shape
        if image.dim() == 4:
            image = image.squeeze(0)
        if image.dim() != 3:
            raise ValueError(f"Expected 3D image tensor (C,H,W), got {image.shape}")

        image_tensor = image.unsqueeze(0).to(device).requires_grad_(True)

        # Determine target class
        if target_class is None:
            with torch.no_grad():
                output = model(image_tensor)
                target_class = output.argmax(dim=1).item()

        try:
            from captum.attr import LRP

            lrp = LRP(model)
            attributions = lrp.attribute(image_tensor, target=target_class)
            if attributions is None:
                raise ValueError("LRP attribution returned None")

            relevance = attributions.detach().cpu().squeeze(0)  # (C,H,W)
            if relevance.dim() == 3:
                relevance = relevance.sum(dim=0)  # (H,W)

        except Exception as e:
            # Fallback: Gradient-based relevance
            try:
                model.zero_grad()
                image_tensor = image_tensor.detach().requires_grad_(True)
                output = model(image_tensor)

                target_score = output[0, target_class]
                target_score.backward()

                if image_tensor.grad is None:
                    raise ValueError("Gradients are None in fallback.")

                relevance = image_tensor.grad.detach().cpu().squeeze(0)
                if relevance.dim() == 3:
                    relevance = relevance.clamp(min=0).sum(dim=0)

            except Exception as fallback_e:
                print(f"LRP + fallback failed: {fallback_e}")
                # Return original image and blank heatmap
                image_np = image.permute(1, 2, 0).cpu().numpy()
                image_np = np.clip(image_np, 0, 1)
                image_np = (image_np * 255).astype(np.uint8)
                heatmap = np.zeros_like(image_np, dtype=np.uint8)
                return image_np, heatmap

        # Normalize relevance
        relevance = relevance.clamp(min=0)
        if relevance.max() > 0:
            relevance = relevance / (relevance.max() + epsilon)

        relevance_map = relevance.numpy()
        if input_size is not None and relevance_map.shape != (input_size, input_size):
            relevance_map = cv2.resize(relevance_map, (input_size, input_size))

        relevance_map = (relevance_map * 255).astype(np.uint8)
        heatmap = cv2.applyColorMap(relevance_map, cv2.COLORMAP_JET)

        # Prepare original image
        image_np = image_tensor.detach().squeeze(0).permute(1, 2, 0).cpu().numpy()

        if imagenet_norm:
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            image_np = image_np * std + mean

        image_np = np.clip(image_np, 0, 1)
        image_np = (image_np * 255).astype(np.uint8)

        # Superimpose heatmap
        superimposed_img = cv2.addWeighted(image_np, 0.7, heatmap, 0.3, 0.0)

        return superimposed_img, heatmap


# ✅ Step 11: 📊 Optuna Trials [Hyper-parameter Tuning]




In [ ]:
# Optimized for maximum GPU utilization and enhanced user experience
from threading import Lock
from termcolor import colored, cprint

import warnings
from optuna.exceptions import ExperimentalWarning
# Suppress only ExperimentalWarning
warnings.filterwarnings("ignore", category=ExperimentalWarning)
optuna.logging.set_verbosity(optuna.logging.WARNING)

# # Configure logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# logger = logging.getLogger(__name__)



def worker_init_fn(worker_id):
    #Initialize worker with different random seed
    np.random.seed(torch.initial_seed() % 2**32 + worker_id)

class Optuna_DataManager:

    @staticmethod
    def create_data_loaders(X, Y, train_batch_size=64, val_batch_size=128,
                                    test_size=0.2, augmentation_strength='medium',
                                    num_workers=8, pin_memory=True, persistent_workers=True):

        # Split data strategically
        X_temp, X_test, y_temp, y_test = train_test_split(
            X, Y, test_size=test_size, random_state=42, stratify=Y
        )
        X_train, X_val, y_train, y_val = train_test_split(
            X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
        )

        cprint(f"📊 Data Distribution:", 'cyan', attrs=['bold'])
        print(f"   Train: {len(X_train):,} samples")
        print(f"   Val:   {len(X_val):,} samples")
        print(f"   Test:  {len(X_test):,} samples")
        print(f"   Batch: Train={train_batch_size}, Val={val_batch_size}")

        # Create datasets with transforms (assuming these classes exist)
        # You need to define these or import them
        try:
            # from your_data_module import FishDataset, DataManager  # Replace with actual imports
            train_dataset = FishDataset(X_train, y_train, DataManager.get_transforms(True, augmentation_strength))
            val_dataset = FishDataset(X_val, y_val, DataManager.get_transforms(False))
            test_dataset = FishDataset(X_test, y_test, DataManager.get_transforms(False))
        except ImportError:
            raise ImportError("FishDataset and DataManager classes not found. Please ensure they are imported.")
        except Exception as e:
            raise Exception(f"Error creating datasets: {e}")

        # Calculate class weights for balanced training
        class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        sample_weights = [class_weights[y] for y in y_train]
        sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

        # GPU-optimized data loading parameters
        actual_num_workers = min(num_workers, os.cpu_count()) if torch.cuda.is_available() else 4
        prefetch_factor = min(8, 2)  # Fixed: prevent excessive prefetch factor

        # Create GPU-optimized data loaders
        train_loader = DataLoader(
            train_dataset,
            batch_size=train_batch_size,
            sampler=sampler,
            num_workers=actual_num_workers,
            pin_memory=pin_memory and torch.cuda.is_available(),
            prefetch_factor=prefetch_factor,
            persistent_workers=persistent_workers and actual_num_workers > 0,
            worker_init_fn=worker_init_fn,
            drop_last=True
        )

        val_loader = DataLoader(
            val_dataset,
            batch_size=val_batch_size,
            shuffle=False,
            num_workers=actual_num_workers,
            pin_memory=pin_memory and torch.cuda.is_available(),
            prefetch_factor=prefetch_factor,
            persistent_workers=persistent_workers and actual_num_workers > 0,
            worker_init_fn=worker_init_fn
        )

        test_loader = DataLoader(
            test_dataset,
            batch_size=val_batch_size,
            shuffle=False,
            num_workers=actual_num_workers,
            pin_memory=pin_memory and torch.cuda.is_available(),
            prefetch_factor=prefetch_factor,
            persistent_workers=persistent_workers and actual_num_workers > 0,
            worker_init_fn=worker_init_fn
        )

        return train_loader, val_loader, test_loader, (X_val, y_val), (X_test, y_test)


def setup_maximum_gpu_utilization() -> Tuple[int, float, float]:
    """Setup GPU optimizations with proper error handling"""
    print("\n" + "="*80)
    cprint("🚀 SETTING UP MAXIMUM GPU UTILIZATION", 'red', attrs=['bold'])
    print("="*80)

    gpu_memory_gb = 0.0
    if torch.cuda.is_available():
        # Aggressive GPU optimizations
        torch.cuda.empty_cache()
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.enabled = True

        # Maximum performance settings
        if hasattr(torch.backends.cuda.matmul, 'allow_tf32'):
            torch.backends.cuda.matmul.allow_tf32 = True
        if hasattr(torch.backends.cudnn, 'allow_tf32'):
            torch.backends.cudnn.allow_tf32 = True

        # Fixed: Proper Flash Attention setup
        if hasattr(torch.backends.cuda, 'enable_flash_sdp'):
            torch.backends.cuda.enable_flash_sdp(True)

        # Use maximum GPU memory (reduced from 98% to prevent OOM)
        torch.cuda.set_per_process_memory_fraction(0.90)

        # Get GPU specifications
        for i in range(torch.cuda.device_count()):
            gpu_props = torch.cuda.get_device_properties(i)
            gpu_memory_gb = gpu_props.total_memory / 1e9

            cprint(f"🎮 GPU {i}: {gpu_props.name}", 'green', attrs=['bold'])
            print(f"   Memory: {gpu_memory_gb:.1f}GB")
            print(f"   Compute: {gpu_props.major}.{gpu_props.minor}")
            print(f"   Cores: {gpu_props.multi_processor_count}")

        # Set multi-GPU if available
        if torch.cuda.device_count() > 1:
            cprint(f"🔥 Using {torch.cuda.device_count()} GPUs!", 'red', attrs=['bold'])
    else:
        cprint("⚠️  No GPU available - using CPU only", 'yellow', attrs=['bold'])

    # CPU optimizations for data loading
    cpu_count = os.cpu_count()
    optimal_threads = min(cpu_count, 16)  # Limit threads to prevent overhead

    torch.set_num_threads(optimal_threads)
    os.environ['OMP_NUM_THREADS'] = str(optimal_threads)
    os.environ['MKL_NUM_THREADS'] = str(optimal_threads)
    os.environ['NUMEXPR_NUM_THREADS'] = str(optimal_threads)

    # Memory information
    memory_info = psutil.virtual_memory()
    available_ram = memory_info.available / (1024**3)
    total_ram = memory_info.total / (1024**3)

    cprint(f"💻 CPU: {cpu_count} cores (using {optimal_threads})", 'blue', attrs=['bold'])
    cprint(f"🧠 RAM: {total_ram:.1f}GB total, {available_ram:.1f}GB available", 'blue', attrs=['bold'])

    return optimal_threads, available_ram, gpu_memory_gb



def get_maximum_batch_sizes(model_name: str, available_ram_gb: float, gpu_memory_gb: float) -> Tuple[int, int]:
    """Calculate maximum batch sizes for full GPU utilization"""

    # Conservative batch sizes to prevent OOM
    base_batch_sizes = {
        'resnet50': {'train': 64, 'val': 128},
        'efficientnet_b0': {'train': 96, 'val': 192},
        'mobilenet_v3_large': {'train': 128, 'val': 256},
        'vgg16': {'train': 32, 'val': 64},
        'densenet121': {'train': 48, 'val': 96},
        'inception_v3': {'train': 40, 'val': 80},
        'vit_b_16': {'train': 32, 'val': 64},
        'convnext_base': {'train': 36, 'val': 72},
        'regnet_y_32gf': {'train': 24, 'val': 48}
    }

    # Conservative scaling based on GPU memory
    if gpu_memory_gb >= 24:  # High-end GPU
        gpu_multiplier = 1.8
    elif gpu_memory_gb >= 16:  # Mid-range GPU
        gpu_multiplier = 1.5
    elif gpu_memory_gb >= 8:   # Entry-level GPU
        gpu_multiplier = 1.2
    else:
        gpu_multiplier = 1.0

    # RAM scaling
    ram_multiplier = min(2.0, available_ram_gb / 16)
    total_multiplier = min(gpu_multiplier, ram_multiplier)

    model_key = model_name.lower()
    if model_key not in base_batch_sizes:
        model_key = 'resnet50'

    base_train = base_batch_sizes[model_key]['train']
    base_val = base_batch_sizes[model_key]['val']

    train_batch = int(base_train * total_multiplier)
    val_batch = int(base_val * total_multiplier)

    # Ensure minimum viable sizes
    train_batch = max(16, train_batch)
    val_batch = max(32, val_batch)

    return train_batch, val_batch




# class EarlyStoppingCallback:
#     def __init__(self, target_accuracy=99.8, optimizer_instance=None):
#         self.target_accuracy = target_accuracy
#         self.optimizer = optimizer_instance

#     def __call__(self, study, trial):
#         if self.optimizer and self.optimizer.best_accuracy >= self.target_accuracy:
#             study.stop()
#             cprint(f"\n🎯 EARLY STOPPING: Target accuracy {self.target_accuracy}% achieved!", 'red', attrs=['bold'])



class HyperparameterOptimizer:
    """Enhanced hyperparameter optimizer with proper error handling"""

    def __init__(self, model_name: str, train_loader, val_loader, n_trials: int = 100):
        self.model_name = model_name
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.n_trials = n_trials

        # Set Google Drive path
        self.drive_path = '/content/drive/MyDrive/Fish_Parameters'
        os.makedirs(self.drive_path, exist_ok=True)

        # Use all available GPUs
        if torch.cuda.is_available():
            self.device = torch.device('cuda:0')
            self.use_multi_gpu = torch.cuda.device_count() > 1
        else:
            self.device = torch.device('cpu')
            self.use_multi_gpu = False

        self.lock = Lock()
        self.best_accuracy = 0.0
        self.current_trial = 0

        # Track best trial information
        self.best_trial_info = {
            'trial_number': 0,
            'accuracy': 0.0,
            'train_loss': 0.0,
            'val_loss': 0.0,
            'train_acc': 0.0,
            'val_acc': 0.0,
            'train_f1': 0.0,
            'val_f1': 0.0,
            'hyperparameters': {}
        }

    def suggest_hyperparameters(self, trial) -> Dict[str, Any]:
        """Suggest hyperparameters - you need to implement this"""
        # This is a placeholder - implement your hyperparameter suggestions
        return {
            'lr': trial.suggest_float('lr', 1e-5, 1e-2, log=True),
            'weight_decay': trial.suggest_float('weight_decay', 1e-6, 1e-2, log=True),
            'optimizer': trial.suggest_categorical('optimizer', ['adamw', 'adam', 'sgd']),
            'scheduler': trial.suggest_categorical('scheduler', ['cosine', 'step', 'plateau']),
            'label_smoothing': trial.suggest_float('label_smoothing', 0.0, 0.2),
            'gradient_clip': trial.suggest_float('gradient_clip', 0.5, 2.0),
            'warmup_epochs': trial.suggest_int('warmup_epochs', 0, 3),
            'dropout': trial.suggest_float('dropout', 0.1, 0.5)
        }

    def create_model_with_params(self, params: Dict[str, Any]):
        """Create model with parameters - you need to implement this"""
        # This is a placeholder - implement your model creation logic
        try:
            # from your_model_module import ModelFactory  # Replace with actual import
            model = ModelFactory.create_model(self.model_name, params)
            if self.use_multi_gpu:
                model = nn.DataParallel(model)
            return model.to(self.device)
        except ImportError:
            raise ImportError("ModelFactory not found. Please ensure it is imported.")
        except Exception as e:
            raise Exception(f"Error creating model: {e}")


    def display_hyperparameters(self, trial_num: int, params: Dict[str, Any]):
        """Display hyperparameters in a formatted way"""
        print("\n" + "🔧" * 80)
        cprint(f"📋 TRIAL {trial_num} HYPERPARAMETERS - {self.model_name.upper()}", 'cyan', attrs=['bold'])
        print("🔧" * 80)

        # Display all parameters
        cprint("  🎯 HYPERPARAMETERS:", 'yellow', attrs=['bold'])
        for key, value in params.items():
            if isinstance(value, float):
                print(f"    🔹 {key:<20}: {value:.8f}")
            else:
                print(f"    🔹 {key:<20}: {value}")

        print("🔧" * 80)


    def display_best_trial_status(self):
        """Display current best trial information"""
        # print("\n" + "🏆" * 80)
        cprint(f"👑 CURRENT BEST TRIAL STATUS - {self.model_name.upper()}", 'red', attrs=['bold'])
        print("🏆" * 80)

        if self.best_trial_info['trial_number'] > 0:
            cprint(f"  🥇 Best Trial : #{self.best_trial_info['trial_number']}", 'yellow', attrs=['bold'])
            cprint(f"  🎯 Best Accuracy: {self.best_trial_info['accuracy']:.4f}%", 'green', attrs=['bold'])

            # Display metrics
            print(f"  📊 METRICS:")
            print(f"    🔸 Train Loss:     {self.best_trial_info['train_loss']:.6f}")
            print(f"    🔸 Val Loss:       {self.best_trial_info['val_loss']:.6f}")
            print(f"    🔸 Train Accuracy: {self.best_trial_info['train_acc']:.4f}%")
            print(f"    🔸 Val Accuracy:   {self.best_trial_info['val_acc']:.4f}%")
            print(f"    🔸 Train F1:       {self.best_trial_info['train_f1']:.4f}%")
            print(f"    🔸 Val F1:         {self.best_trial_info['val_f1']:.4f}%")
        else:
            cprint("  🔄 No trials completed yet", 'yellow')

        # print("🏆" * 80)

    def create_optimizer_and_scheduler(self, model, params: Dict[str, Any], steps_per_epoch: int):
        """Create optimizer and scheduler with GPU optimizations"""

        # Create optimizer
        if params['optimizer'] == 'adamw':
            optimizer = torch.optim.AdamW(
                model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'],
                betas=(0.9, 0.999), eps=1e-8
            )
        elif params['optimizer'] == 'adam':
            optimizer = torch.optim.Adam(
                model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'],
                betas=(0.9, 0.999), eps=1e-8
            )
        elif params['optimizer'] == 'sgd':
            optimizer = torch.optim.SGD(
                model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'],
                momentum=0.9, nesterov=True
            )
        else:  # rmsprop
            optimizer = torch.optim.RMSprop(
                model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'],
                momentum=0.9, alpha=0.99
            )

        # Create scheduler
        if params['scheduler'] == 'cosine':
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=8)
        elif params['scheduler'] == 'cosine_warm':
            scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=3, T_mult=2)
        elif params['scheduler'] == 'step':
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.7)
        else:  # plateau
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='max', patience=2, factor=0.5
            )

        return optimizer, scheduler


    # def train_and_validate(self, model, params: Dict[str, Any], epochs: int = 8) -> Tuple[float, Dict]:
    def train_and_validate(self, model, params: Dict[str, Any], epochs: int=None , trial=None) -> Tuple[float, Dict]: #epochs: int = 8
        #Enhanced training with comprehensive metrics and GPU utilization
        if epochs is None:
            epochs = Config.OPTUNA_EPOCHS

        steps_per_epoch = len(self.train_loader)
        optimizer, scheduler = self.create_optimizer_and_scheduler(model, params, steps_per_epoch)
        criterion = nn.CrossEntropyLoss(label_smoothing=params.get('label_smoothing', 0.0))
        scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None

        best_val_acc = 0.0
        metrics_history = []
        patience = 0  # লুপের বাইরে initialize
        epoch_best_f1 = 0.0


        for epoch in range(epochs):
            # Training phase
            model.train()
            train_loss = 0.0
            train_correct = 0
            train_total = 0
            train_preds = []
            train_targets = []

            train_pbar = tqdm(
                self.train_loader,
                desc=f"  🏃 Epoch {epoch+1:2d} Train",
                leave=False,
                ncols=100
            )

            for batch_idx, (data, targets) in enumerate(train_pbar):
                data, targets = data.to(self.device, non_blocking=True), targets.to(self.device, non_blocking=True)
                optimizer.zero_grad()

                if scaler and torch.cuda.is_available():
                    with torch.cuda.amp.autocast():
                        outputs = model(data)
                        loss = criterion(outputs, targets)

                    scaler.scale(loss).backward()
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(model.parameters(), params.get('gradient_clip', 1.0))
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = model(data)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), params.get('gradient_clip', 1.0))
                    optimizer.step()

                # Metrics calculation
                train_loss += loss.item() * data.size(0)
                _, predicted = torch.max(outputs, 1)
                train_total += targets.size(0)
                train_correct += (predicted == targets).sum().item()

                train_preds.extend(predicted.cpu().numpy())
                train_targets.extend(targets.cpu().numpy())

                # Update progress bar
                current_acc = 100 * train_correct / train_total
                train_pbar.set_postfix({
                    'Loss': f'{loss.item():.4f}',
                    'Acc': f'{current_acc:.2f}%'
                })

            # Validation phase
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0
            val_preds = []
            val_targets = []


            with torch.no_grad():
                for data, targets in self.val_loader:
                    data, targets = data.to(self.device, non_blocking=True), targets.to(self.device, non_blocking=True)

                    if scaler and torch.cuda.is_available():
                        with torch.cuda.amp.autocast():
                            outputs = model(data)
                            loss = criterion(outputs, targets)
                    else:
                        outputs = model(data)
                        loss = criterion(outputs, targets)

                    val_loss += loss.item() * data.size(0)
                    _, predicted = torch.max(outputs, 1)
                    val_total += targets.size(0)
                    val_correct += (predicted == targets).sum().item()

                    val_preds.extend(predicted.cpu().numpy())
                    val_targets.extend(targets.cpu().numpy())

            # Calculate metrics
            train_accuracy = 100 * train_correct / train_total
            val_accuracy = 100 * val_correct / val_total
            avg_train_loss = train_loss / train_total
            avg_val_loss = val_loss / val_total

            train_f1 = f1_score(train_targets, train_preds, average='weighted') * 100
            val_f1 = f1_score(val_targets, val_preds, average='weighted') * 100

            val_wrong = val_total - val_correct

            # Print epoch results
            print(f"    📊 TL:{avg_train_loss:.4f} VL:{avg_val_loss:.4f} | " +
                  f"TA:{train_accuracy:.2f}% VA:{val_accuracy:.2f}% | " +
                  f"TF1:{train_f1:.2f}% VF1:{val_f1:.2f}% | " +
                  f"WP:{val_wrong}")

            # Store metrics
            epoch_metrics = {
                'epoch': epoch + 1,
                'train_loss': avg_train_loss,
                'val_loss': avg_val_loss,
                'train_acc': train_accuracy,
                'val_acc': val_accuracy,
                'train_f1': train_f1,
                'val_f1': val_f1,
                'wrong_predictions': val_wrong
            }
            metrics_history.append(epoch_metrics)

            # Update best accuracy
            if val_accuracy > best_val_acc:
                best_val_acc = val_accuracy

            # Update scheduler
            if params['scheduler'] == 'plateau':
                scheduler.step(val_accuracy)
            else:
                scheduler.step()



            # Early stopping for optimization speed
            if epoch+1 >= 5 and val_accuracy < 80.0:
                print(f"⚠️ Early stopping cause epoch {epoch+1} but still not satisfactory accuracy obtain.")
                break


            # Early stopping for not improvement
            if val_f1 > epoch_best_f1 * 1.001:  # improvement condition (0.1% increment)
                epoch_best_f1 = val_f1
                patience = 0  # reset patience, কারণ improvement হয়েছে
            else:
                patience += 1  # no improvement, patience বাড়াও
            if patience > Config.PATIENCE:
                print(f"⚠️ Early stopping: No improvement for {Config.PATIENCE} consecutive epochs")
                break


            #Early stopping for trial level pruning
            # Report intermediate value for pruning
            if trial is not None:
                trial.report(val_accuracy, epoch)
                # Check if trial should be pruned
                if trial.should_prune():
                    print(f"    ⚠️ Early stopping at epoch {epoch+1}: Low accuracy probability detected")
                    print(f"    🔄 Pruning trial - proceeding to next hyperparameter combination")
                    raise optuna.exceptions.TrialPruned()

        return best_val_acc, {'history': metrics_history, 'best_epoch_metrics': max(metrics_history, key=lambda x: x['val_acc'])}




    def objective(self, trial) -> float:
      """Enhanced objective function with detailed progress tracking"""
      self.current_trial += 1


      print("\n" + "█" * 100)
      cprint(f"🔥 TRIAL {self.current_trial:3d}/{self.n_trials} STARTING - {self.model_name.upper()}", 'red', attrs=['bold'])
      print("█" * 100)

      try:
          with self.lock:
              if torch.cuda.is_available():
                  torch.cuda.empty_cache()
                  for i in range(torch.cuda.device_count()):
                      memory_used = torch.cuda.memory_allocated(i) / 1e9
                      memory_total = torch.cuda.get_device_properties(i).total_memory / 1e9
                      print(f"  🎮 GPU {i}: {memory_used:.1f}/{memory_total:.1f}GB ({memory_used/memory_total*100:.1f}%)")
              gc.collect()

          # Get hyperparameters
          params = self.suggest_hyperparameters(trial)
          self.display_hyperparameters(self.current_trial, params)

          # Create and train model
          model = self.create_model_with_params(params)
          best_acc, detailed_metrics = self.train_and_validate(model, params, trial=trial)

          # SUCCESS HANDLING - Move this BEFORE the except blocks
          best_epoch_metrics = detailed_metrics['best_epoch_metrics']

          # Trial completion summary
          print("  " + "─" * 80)
          cprint(f"  ✅ TRIAL {self.current_trial} COMPLETED", 'green', attrs=['bold'])
          cprint(f"  🎯 Highest Validation Accuracy for this Trial: {best_acc:.4f}%", 'yellow', attrs=['bold'])

          # Update best trial info if this is better
          if best_acc > self.best_accuracy:
              self.best_accuracy = best_acc
              self.best_trial_info = {
                  'trial_number': self.current_trial,
                  'accuracy': best_acc,
                  'train_loss': best_epoch_metrics['train_loss'],
                  'val_loss': best_epoch_metrics['val_loss'],
                  'train_acc': best_epoch_metrics['train_acc'],
                  'val_acc': best_epoch_metrics['val_acc'],
                  'train_f1': best_epoch_metrics['train_f1'],
                  'val_f1': best_epoch_metrics['val_f1'],
                  'hyperparameters': params.copy()
              }
              cprint(f"  🏆 NEW BEST ACCURACY: {best_acc:.4f}%", 'red', attrs=['bold'])
              # Save immediately
              self.save_best_params_immediately(getattr(self, 'output_dir', './output'))



          self.display_best_trial_status()



          # Cleanup
          del model
          if torch.cuda.is_available():
              torch.cuda.empty_cache()
          gc.collect()

          return best_acc

      except optuna.exceptions.TrialPruned:
          cprint(f"  ✂️ TRIAL {self.current_trial} PRUNED: Low accuracy probability detected", 'yellow', attrs=['bold'])
          cprint(f"  🔄 Skipping to next hyperparameter combination for efficiency", 'cyan')
          # Cleanup
          if 'model' in locals():
              del model
          if torch.cuda.is_available():
              torch.cuda.empty_cache()
          gc.collect()
          raise  # Re-raise the TrialPruned exception

      except Exception as e:
          cprint(f"  ❌ TRIAL {self.current_trial} FAILED: {e}", 'red', attrs=['bold'])
          cprint(f"  📋 Error Details: {traceback.format_exc()}", 'yellow')
          # Cleanup
          if 'model' in locals():
              del model
          if torch.cuda.is_available():
              torch.cuda.empty_cache()
          gc.collect()
          return 0.0


    def optimize(self) -> Dict[str, Any]:
        """Run optimization with enhanced progress tracking"""

        # print("\n" + "🚀" * 40)
        cprint(f"STARTING HYPERPARAMETER OPTIMIZATION FOR {self.model_name.upper()}", 'red', attrs=['bold'])
        print("🚀" * 40)

        # Create study
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(
                # n_startup_trials=max(15, self.n_trials // 8),
                n_startup_trials = max(15, self.n_trials // 6),
                # Example: If you set n_startup_trials=10, the first 10 trials will be random, then trial 11 onwards will use TPE-guided sampling.
                n_ei_candidates=32,
                constant_liar=True,
                multivariate=True
            ),
            # Sampler’s n_startup_trials → when TPE optimization begins.
            # Pruner’s n_startup_trials → how many full trials to finish before pruning starts.
            # Pruner’s n_warmup_steps → how many epochs per trial to protect before pruning checks.
            pruner=optuna.pruners.MedianPruner(
                # n_startup_trials=max(6, self.n_trials // 15),
                n_startup_trials = max(10, self.n_trials //8),
                n_warmup_steps=3,
                interval_steps=1
            )
        )

        # Run optimization
        cprint(f"🎯 Target: {self.n_trials} trials", 'cyan', attrs=['bold'])
        # study.optimize(
        #     self.objective,
        #     n_trials=self.n_trials,
        #     timeout=10800,
        #     n_jobs=1
        # )
        # Run optimization with early stopping check
        for trial_num in range(self.n_trials):
            try:
                study.optimize(self.objective, timeout=None, n_jobs=1, n_trials=1)

                # Check for early stopping after each trial
                if self.best_accuracy >= 99.5:
                    cprint(f"\n🎯 TARGET ACCURACY ACHIEVED!", 'red', attrs=['bold'])
                    cprint(f"🏆 Best Accuracy: {self.best_accuracy:.4f}% >= 99.5%", 'green', attrs=['bold'])
                    cprint(f"⚡ Stopping optimization early after {self.current_trial} trials", 'yellow', attrs=['bold'])
                    cprint(f"🚀 Moving to next model for maximum efficiency!", 'cyan', attrs=['bold'])
                    break

            except KeyboardInterrupt:
                cprint(f"\n⚠️ Optimization interrupted by user", 'yellow', attrs=['bold'])
                break
            except Exception as e:
                cprint(f"⚠️ Trial failed: {e}", 'red')
                continue


        # # Create early stopping callback
        # early_stop_callback = EarlyStoppingCallback(target_accuracy=99.8, optimizer_instance=self)

        # # Run optimization with callback
        # cprint(f"🎯 Target: {self.n_trials} trials (will stop early if 99.8% accuracy achieved)", 'cyan', attrs=['bold'])
        # study.optimize(
        #     self.objective,
        #     n_trials=self.n_trials,
        #     timeout=10800,
        #     n_jobs=1,
        #     callbacks=[early_stop_callback]
        # )



        # Final results
        print("\n" + "🏁" * 40)
        # cprint(f"OPTIMIZATION COMPLETED FOR {self.model_name.upper()}", 'green', attrs=['bold'])
        if self.best_accuracy >= 99.5:
            cprint(f"🎯 OPTIMIZATION COMPLETED - TARGET ACHIEVED!", 'green', attrs=['bold'])
            cprint(f"⚡ Completed in {self.current_trial} trials (saved {self.n_trials - self.current_trial} trials)", 'yellow', attrs=['bold'])
        else:
            cprint(f"OPTIMIZATION COMPLETED FOR {self.model_name.upper()}", 'green', attrs=['bold'])

        cprint(f"🏆 OPTIMIZATION BEST ACCURACY: {self.best_accuracy:.4f}%", 'red', attrs=['bold'])
        print("🏁" * 40)

        return study.best_params if study.best_params else {}



    def save_best_params_immediately(self, output_dir: str = './output') -> None:
        if self.best_trial_info['trial_number'] == 0:
            return

        # Local save
        os.makedirs(f"{output_dir}/hyperparameters/immediate", exist_ok=True)
        local_file = f"{output_dir}/hyperparameters/immediate/{self.model_name}_best_params_trial_{self.best_trial_info['trial_number']}.json"

        # Google Drive save
        drive_file = f"{self.drive_path}/{self.model_name}_best_params_trial_{self.best_trial_info['trial_number']}.json"

        results_with_meta = {
            'model_name': self.model_name,
            'trial_number': self.best_trial_info['trial_number'],
            'accuracy': self.best_trial_info['accuracy'],
            'timestamp': time.strftime("%Y-%m-%d %H:%M:%S"),
            'metrics': {
                'train_loss': self.best_trial_info['train_loss'],
                'val_loss': self.best_trial_info['val_loss'],
                'train_acc': self.best_trial_info['train_acc'],
                'val_acc': self.best_trial_info['val_acc'],
                'train_f1': self.best_trial_info['train_f1'],
                'val_f1': self.best_trial_info['val_f1']
            },
            'hyperparameters': self.best_trial_info['hyperparameters']
        }

        # Save to both locations
        with open(local_file, 'w') as f:
            json.dump(results_with_meta, f, indent=4, sort_keys=True)

        with open(drive_file, 'w') as f:
            json.dump(results_with_meta, f, indent=4, sort_keys=True)

        cprint(f"  💾 Best params saved locally: {local_file}", 'green')
        cprint(f"  ☁️ Best params saved to Drive: {drive_file}", 'green')




# def optimize_single_model(model_name: str, config: Dict[str, Any]) -> Dict[str, Any]:
def optimize_single_model(model_name: str, config: Dict[str, Any], output_dir: str = './output') -> Dict[str, Any]:
    """Optimize single model with maximum GPU utilization"""

    print("\n" + "⚡" * 50)
    cprint(f"OPTIMIZING {model_name.upper()}", 'red', attrs=['bold'])
    print("⚡" * 50)

    try:
        # Setup environment
        optimal_threads, available_ram, gpu_memory_gb = setup_maximum_gpu_utilization()

        # Get maximum batch sizes
        train_batch_size, val_batch_size = get_maximum_batch_sizes(
            model_name, available_ram, gpu_memory_gb
        )

        cprint(f"🎯 Maximum Batch Sizes - Train: {train_batch_size}, Val: {val_batch_size}", 'green', attrs=['bold'])

        # Create data loaders
        train_loader, val_loader, test_loader, val_data, test_data = Optuna_DataManager.create_data_loaders(
            config['X'], config['Y'],
            train_batch_size=train_batch_size,
            val_batch_size=val_batch_size,
            num_workers=optimal_threads,
            pin_memory=True,
            persistent_workers=True
        )

        # Run optimization
        optimizer = HyperparameterOptimizer(
            model_name, train_loader, val_loader,
            n_trials=Config.OPTUNA_TRIALS  # Reduced for testing
        )

        # Store output_dir for immediate saving
        optimizer.output_dir = output_dir



        best_params = optimizer.optimize()

        # Cleanup
        del optimizer, train_loader, val_loader, test_loader
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

        return best_params

    except Exception as e:
        cprint(f"❌ OPTIMIZATION FAILED FOR {model_name}: {e}", 'red', attrs=['bold'])
        cprint(f"📋 Error: {traceback.format_exc()}", 'yellow')
        return {}



# def parallel_hyperparameter_optimization(model_configs: Dict[str, Any], max_workers: int = 1) -> Dict[str, Any]:
def parallel_hyperparameter_optimization(model_configs: Dict[str, Any], max_workers: int = 1, output_dir: str = './output') -> Dict[str, Any]:
    # Run optimization with sequential processing for maximum GPU utilization

    results = {}

    # print("\n" + "🎪" * 50)
    cprint("STARTING PARALLEL HYPERPARAMETER OPTIMIZATION", 'red', attrs=['bold'])
    # print("🎪" * 50)

    # Sequential processing for maximum GPU utilization per model
    for i, (model_name, config) in enumerate(model_configs.items(), 1):
        cprint(f"\n📍 MODEL {i}/{len(model_configs)}: {model_name.upper()}", 'cyan', attrs=['bold'])

        try:
            # best_params = optimize_single_model(model_name, config)
            best_params = optimize_single_model(model_name, config, './output')
            results[model_name] = best_params

            if best_params:
                cprint(f"✅ {model_name.upper()} OPTIMIZATION COMPLETED!", 'green', attrs=['bold'])
            else:
                cprint(f"❌ {model_name.upper()} OPTIMIZATION FAILED!", 'red', attrs=['bold'])

        except Exception as e:
            cprint(f"❌ {model_name.upper()} CRASHED: {e}", 'red', attrs=['bold'])
            results[model_name] = {}

    return results



def save_optimization_results(results: Dict[str, Any], output_dir: str) -> None:
    """Save optimization results with enhanced formatting"""

    os.makedirs(f"{output_dir}/hyperparameters", exist_ok=True)

    print("\n" + "💾" * 50)
    cprint("SAVING OPTIMIZATION RESULTS", 'cyan', attrs=['bold'])
    print("💾" * 50)

    # Save individual model results
    successful_models = 0
    for model_name, best_params in results.items():
        if best_params:
            params_file = f"{output_dir}/hyperparameters/{model_name}_best_params.json"

            # Enhanced metadata
            results_with_meta = {
                'model_name': model_name,
                'timestamp': time.strftime("%Y-%m-%d %H:%M:%S"),
                'gpu_optimized': True,
                'hyperparameters': best_params,
                'optimization_config': {
                    'framework': 'optuna',
                    'sampler': 'TPE_Multivariate',
                    'pruner': 'Median',
                    'trials': 40,
                    'gpu_acceleration': torch.cuda.is_available(),
                    'multi_gpu': torch.cuda.device_count() > 1 if torch.cuda.is_available() else False
                }
            }

            with open(params_file, 'w') as f:
                json.dump(results_with_meta, f, indent=4, sort_keys=True)

            cprint(f"✅ {model_name.upper()} parameters saved!", 'green')

            # Display best parameters
            print(f"  📋 {model_name.upper()} BEST PARAMETERS:")
            for key, value in best_params.items():
                if isinstance(value, float):
                    print(f"    🔹 {key:<20}: {value:.6f}")
                else:
                    print(f"    🔹 {key:<20}: {value}")
            print()

            successful_models += 1

    # Save master results file
    master_file = f"{output_dir}/hyperparameters/all_best_params.json"

    # GPU information
    gpu_info = {}
    if torch.cuda.is_available():
        gpu_info = {
            'gpu_count': torch.cuda.device_count(),
            'gpu_names': [torch.cuda.get_device_properties(i).name for i in range(torch.cuda.device_count())],
            'total_gpu_memory_gb': sum(torch.cuda.get_device_properties(i).total_memory / 1e9 for i in range(torch.cuda.device_count()))
        }

    master_results = {
        'optimization_summary': {
            'total_models': len(results),
            'successful_optimizations': successful_models,
            'failed_optimizations': len(results) - successful_models,
            'success_rate_percent': (successful_models / len(results)) * 100 if results else 0,
            'gpu_accelerated': torch.cuda.is_available(),
            'system_info': {
                'cpu_cores': os.cpu_count(),
                'ram_gb': psutil.virtual_memory().total / (1024**3),
                **gpu_info
            }
        },
        'results': results
    }

    with open(master_file, 'w') as f:
        json.dump(master_results, f, indent=4, sort_keys=True)

    cprint(f"💾 Master results saved to: {master_file}", 'cyan', attrs=['bold'])

    # Final summary
    print("\n" + "📊" * 50)
    cprint("OPTIMIZATION SUMMARY", 'yellow', attrs=['bold'])
    print("📊" * 50)
    print(f"  🎯 Total Models: {len(results)}")
    print(f"  ✅ Successful: {successful_models}")
    print(f"  ❌ Failed: {len(results) - successful_models}")
    print(f"  📈 Success Rate: {(successful_models / len(results)) * 100:.1f}%")
    if torch.cuda.is_available():
        print(f"  🎮 GPU Acceleration: Enabled ({torch.cuda.device_count()} GPUs)")
    else:
        print(f"  💻 GPU Acceleration: Disabled (CPU only)")





def display_startup_banner():
    #Display an impressive startup banner
    banner = """
┌─ THE FISH OPTIMIZER ─┐
│   🐟 Optimizing 🐟   │
└──────────────────────┘
    """

    # print("\n" + "="*120)
    cprint(banner, 'red', attrs=['bold'])
    # print("="*120)
    # cprint("🚀 MAXIMUM GPU-ACCELERATED HYPERPARAMETER OPTIMIZATION 🚀", 'yellow', attrs=['bold'])
    # cprint("🔥 DESIGNED FOR MAXIMUM PERFORMANCE AND USER EXPERIENCE 🔥", 'cyan', attrs=['bold'])
    # print("="*120)



def main():
    """Enhanced main function with spectacular UI and maximum GPU utilization"""

    # Display startup banner
    display_startup_banner()

    # Environment setup with detailed reporting
    print("\n🔧 SYSTEM INITIALIZATION")
    print("─" * 50)

    optimal_threads, available_ram, gpu_memory_gb = setup_maximum_gpu_utilization()

    # Data loading with progress
    print("\n📊 DATA LOADING AND PREPROCESSING")
    print("─" * 50)

    try:
        # You need to implement or import these classes
        # from your_data_module import DataManager  # Replace with actual import

        cprint("🔄 Loading and balancing dataset...", 'cyan', attrs=['bold'])
        X, Y = DataManager.load_and_balance_data()

        cprint(f"✅ Dataset loaded successfully!", 'green', attrs=['bold'])
        print(f"   📈 Total samples: {len(X):,}")
        print(f"   🏷️  Total labels: {len(Y):,}")
        print(f"   📊 Classes: {len(np.unique(Y))}")

        if len(X) != len(Y) or len(X) == 0:
            raise ValueError("Invalid dataset: inconsistent or empty data")

    except ImportError:
        cprint("❌ DataManager not found. Please ensure it is imported.", 'red', attrs=['bold'])
        # # Create dummy data for testing
        # cprint("🔄 Creating dummy data for testing...", 'yellow', attrs=['bold'])
        # X = np.random.randn(1000, 224, 224, 3)
        # Y = np.random.randint(0, 10, 1000)
        # cprint("✅ Dummy dataset created!", 'green', attrs=['bold'])
    except Exception as e:
        cprint(f"❌ Data loading failed: {e}", 'red', attrs=['bold'])
        return

    # Prepare model configurations
    print("\n🤖 MODEL CONFIGURATION")
    print("─" * 50)

    # Default models if Config.MODELS is not available
    try:
        # from your_config_module import Config  # Replace with actual import
        models = Config.MODELS
        output_dir = Config.OUTPUT_DIR
    except ImportError:
        cprint("⚠️  Config not found. Using default models.", 'yellow', attrs=['bold'])
        models = ['resnet50', 'efficientnet_b0', 'mobilenet_v3_large']
        output_dir = './output'

    model_configs = {}
    for i, model_name in enumerate(models, 1):
        model_configs[model_name] = {'X': X, 'Y': Y}
        print(f"  {i:2d}. {model_name}")

    cprint(f"🎯 Configured {len(models)} models for optimization", 'green', attrs=['bold'])

    # Create output directory
    os.makedirs(f"{output_dir}/hyperparameters", exist_ok=True)

    # Run optimization
    print("\n🚀 STARTING HYPERPARAMETER OPTIMIZATION")
    print("─" * 50)

    start_time = time.time()

    all_best_params = parallel_hyperparameter_optimization(
        model_configs,
        max_workers=1,
        output_dir=output_dir  # Add this
    )

    end_time = time.time()
    total_time = end_time - start_time

    # Save results
    save_optimization_results(all_best_params, output_dir)

    # Final summary
    print("\n" + "🎉" * 60)
    cprint("🏆 HYPERPARAMETER OPTIMIZATION COMPLETED! 🏆", 'red', attrs=['bold'])
    print("🎉" * 60)

    successful = sum(1 for params in all_best_params.values() if params)
    total = len(all_best_params)

    print(f"⏱️  Total Time: {total_time//3600:.0f}h {(total_time%3600)//60:.0f}m {total_time%60:.0f}s")
    print(f"📊 Models Processed: {total}")
    print(f"✅ Successful Optimizations: {successful}")
    print(f"❌ Failed Optimizations: {total - successful}")
    print(f"📈 Success Rate: {100*successful/total:.1f}%")
    print(f"💾 Results Location: {output_dir}/hyperparameters/")

    if torch.cuda.is_available():
        print(f"🎮 GPU Utilization: Maximum")
        print(f"🔥 Multi-GPU: {'Yes' if torch.cuda.device_count() > 1 else 'No'}")

    print("\n" + "🎉" * 60)
    cprint("🚀 READY FOR TRAINING WITH OPTIMIZED HYPERPARAMETERS! 🚀", 'green', attrs=['bold'])


if __name__ == "__main__":
    main()


┌─ THE FISH OPTIMIZER ─┐
│   🐟 Optimizing 🐟   │
└──────────────────────┘
    

🔧 SYSTEM INITIALIZATION
──────────────────────────────────────────────────

🚀 SETTING UP MAXIMUM GPU UTILIZATION
🎮 GPU 0: NVIDIA L4
   Memory: 23.8GB
   Compute: 8.9
   Cores: 58
💻 CPU: 12 cores (using 12)
🧠 RAM: 53.0GB total, 48.9GB available

📊 DATA LOADING AND PREPROCESSING
──────────────────────────────────────────────────
🔄 Loading and balancing dataset...
Loading and preprocessing data...
Applying SMOTE for class balancing...
Balanced data shape: (15000, 3, 224, 224)
Balanced class distribution: [3000 3000 3000 3000 3000]
✅ Dataset loaded successfully!
   📈 Total samples: 15,000
   🏷️  Total labels: 15,000
   📊 Classes: 5

🤖 MODEL CONFIGURATION
──────────────────────────────────────────────────
   1. resnet50
   2. efficientnet_b0
   3. mobilenet_v3_large
   4. vgg16
   5. densenet121
   6. inception_v3
🎯 Configured 6 models for optimization

🚀 STARTING HYPERPARAMETER OPTIMIZATION
────────────────────

100%|██████████| 97.8M/97.8M [00:00<00:00, 226MB/s]


    📊 TL:1.7595 VL:1.6545 | TA:20.17% VA:21.90% | TF1:20.17% VF1:18.69% | WP:2343


    📊 TL:1.7552 VL:1.6585 | TA:19.93% VA:22.20% | TF1:19.89% VF1:21.03% | WP:2334


    📊 TL:1.7501 VL:1.6433 | TA:20.88% VA:23.23% | TF1:20.88% VF1:22.33% | WP:2303


...

    📊 TL:0.7484 VL:0.3666 | TA:77.65% VA:96.90% | TF1:77.62% VF1:96.90% | WP:93


    📊 TL:0.7228 VL:0.3693 | TA:78.84% VA:96.60% | TF1:78.78% VF1:96.59% | WP:102


    📊 TL:0.7394 VL:0.3515 | TA:77.45% VA:97.30% | TF1:77.40% VF1:97.30% | WP:81


    📊 TL:0.7154 VL:0.3532 | TA:79.49% VA:97.33% | TF1:79.44% VF1:97.33% | WP:80


  🏃 Epoch 10 Train:  53%|█████████        | 33/62 [00:04<00:02, 10.22it/s, Loss=0.7483, Acc=77.86%]

# ✅ Step 12: Training Pipeline

In [ ]:
class EnhancedModelTrainer:
    def __init__(self, model, model_name, hyperparameters):
        self.model = model.to(Config.DEVICE)
        self.model_name = model_name
        self.hyperparameters = hyperparameters
        self.best_val_acc = 0.0
        self.best_val_f1 = 0.0
        self.patience_counter = 0

        # Resource management
        self.resource_manager = ResourceManager()
        self.memory_check_interval = 15



        self._setup_training_components()



        # Initialize history
        self.history = {
            'train_loss': [], 'train_acc': [], 'val_loss': [],
            'val_acc': [], 'val_f1': [], 'learning_rates': []
        }




    def _setup_training_components(self):
        """Setup optimizer, criterion, and scheduler"""
        allowed_keys = ['lr', 'weight_decay', 'dropout', 'hidden_dim_multiplier',
                       'augmentation_strength', 'batch_size', 'optimizer_type',
                       'scheduler_type', 'label_smoothing']
        self.hyperparameters = {k: v for k, v in self.hyperparameters.items() if k in allowed_keys}

        # Optimizer setup
        lr = self.hyperparameters.get('lr', Config.LEARNING_RATE)
        weight_decay = self.hyperparameters.get('weight_decay', Config.WEIGHT_DECAY)
        optimizer_type = self.hyperparameters.get('optimizer_type', 'adamw')

        if optimizer_type == 'adamw':
            self.optimizer = optim.AdamW(
                self.model.parameters(), lr=lr, weight_decay=weight_decay,
                fused=torch.cuda.is_available()
            )
        elif optimizer_type == 'adam':
            self.optimizer = optim.Adam(
                self.model.parameters(), lr=lr, weight_decay=weight_decay,
                fused=torch.cuda.is_available()
            )
        else:
            self.optimizer = optim.SGD(
                self.model.parameters(), lr=lr, weight_decay=weight_decay,
                momentum=0.9, nesterov=True
            )



        # Criterion
        label_smoothing = self.hyperparameters.get('label_smoothing', 0.1)
        self.criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)

        # Scheduler
        scheduler_type = self.hyperparameters.get('scheduler_type', 'cosine')
        if scheduler_type == 'cosine':
            self.scheduler = optim.lr_scheduler.CosineAnnealingLR(
                self.optimizer, T_max=Config.EPOCHS, eta_min=1e-6
            )
        elif scheduler_type == 'plateau':
            self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer, mode='min', factor=0.5, patience=5
            )
        else:
            self.scheduler = optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.95)


        # Mixed precision scaler
        self.scaler = torch.cuda.amp.GradScaler(enabled=Config.USE_MIXED_PRECISION)



    def train_epoch(self, train_loader, progress_tracker):
        """Enhanced training epoch with smart memory management"""
        self.model.train()
        total_loss = 0
        correct = 0
        total = 0
        batch_count = len(train_loader)

        tqdm.write(f"Total Data: {len(train_loader.dataset):,}  Training : "
                   f"{batch_count:,} batches, Batch_size: {train_loader.batch_size}")

        try:
            for batch_idx, (images, labels) in enumerate(train_loader):
                try:
                    # Smart memory management
                    if batch_idx % self.memory_check_interval == 0:
                        if self.resource_manager.should_cleanup_aggressive():
                            self.resource_manager.aggressive_cleanup()
                            tqdm.write(f"  Memory cleanup at batch {batch_idx}")

                    # Move data to device with optimized memory format
                    images = images.to(Config.DEVICE, non_blocking=True, memory_format=torch.channels_last)
                    # images = images.contiguous(memory_format=torch.channels_last)
                    labels = labels.to(Config.DEVICE, non_blocking=True)

                    # Forward pass
                    self.optimizer.zero_grad(set_to_none=True)
                    with torch.cuda.amp.autocast(enabled=Config.USE_MIXED_PRECISION):
                        outputs = self.model(images)
                        loss = self.criterion(outputs, labels)

                    # Backward pass
                    self.scaler.scale(loss).backward()
                    self.scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                    self.scaler.step(self.optimizer)
                    self.scaler.update()

                    # Calculate metrics
                    _, predicted = torch.max(outputs, 1)
                    batch_acc = (predicted == labels).float().mean().item()
                    batch_loss = loss.item()

                    # Update totals
                    total_loss += batch_loss * images.size(0)
                    total += images.size(0)
                    correct += (predicted == labels).sum().item()

                    # Update progress tracker
                    progress_tracker.update_batch(batch_idx, batch_loss, batch_acc, is_training=True, total_batches=batch_count)

                    # Progress logging every 50 batches
                    # if batch_idx % 40 == 0 and batch_idx > 0:
                    #     avg_loss = total_loss / total
                    #     avg_acc = correct / total
                    #     stats = self.resource_manager.get_memory_stats()
                    #     tqdm.write(f"  Batch {batch_idx:4d}/{batch_count} - "
                    #                f"Loss: {avg_loss:.4f}, Acc: {avg_acc:.4f}, "
                    #                f"GPU: {stats['gpu_percent']:.1f}%, CPU: {stats['cpu_percent']:.1f}%")

                    # Memory cleanup for large batches
                    del outputs, loss, predicted, images, labels

                except Exception as e:
                    tqdm.write(f"Error in batch {batch_idx}: {str(e)}")
                    self.resource_manager.aggressive_cleanup()
                    continue

            # Final cleanup
            self.resource_manager.aggressive_cleanup()

            return total_loss / max(1, total), correct / max(1, total)

        except Exception as e:
            tqdm.write(f"Training epoch failed: {str(e)}")
            self.resource_manager.aggressive_cleanup()
            return float('inf'), 0.0



    def validate_epoch(self, val_loader, progress_tracker):
        """Enhanced validation epoch with memory optimization"""
        self.model.eval()
        total_loss = 0
        total_samples = 0
        all_predictions = []
        all_labels = []
        all_probs = []
        batch_count = len(val_loader)

        tqdm.write(f"Validation epoch: {len(val_loader.dataset):,} samples, "
                   f"{batch_count:,} batches, batch_size: {val_loader.batch_size}")

        try:
            with torch.no_grad():
                for batch_idx, (images, labels) in enumerate(val_loader):
                    try:
                        images = images.to(Config.DEVICE, non_blocking=True, memory_format=torch.channels_last)
                        # images = images.contiguous(memory_format=torch.channels_last)
                        labels = labels.to(Config.DEVICE, non_blocking=True)

                        with torch.cuda.amp.autocast(enabled=Config.USE_MIXED_PRECISION):
                            outputs = self.model(images)
                            loss = self.criterion(outputs, labels)

                        _, predicted = torch.max(outputs, 1)
                        batch_acc = (predicted == labels).float().mean().item()
                        batch_loss = loss.item()

                        # Store results
                        total_loss += batch_loss * images.size(0)
                        total_samples += images.size(0)
                        all_predictions.extend(predicted.cpu().numpy())
                        all_labels.extend(labels.cpu().numpy())
                        all_probs.append(torch.softmax(outputs, dim=1).cpu().numpy())

                        # Update progress
                        progress_tracker.update_batch(batch_idx, batch_loss, batch_acc,
                                                    is_training=False, total_batches=batch_count)

                        # Memory cleanup
                        del outputs, loss, predicted, images, labels

                        # # Periodic memory management
                        # if batch_idx % 20 == 0:
                        #     self.resource_manager.aggressive_cleanup()

                    except Exception as e:
                        tqdm.write(f"Error in validation batch {batch_idx}: {str(e)}")
                        continue

            # Calculate final metrics
            all_probs = np.concatenate(all_probs, axis=0) if all_probs else np.array([])
            val_acc = accuracy_score(all_labels, all_predictions)
            val_f1 = f1_score(all_labels, all_predictions, average='macro', zero_division=0)

            return total_loss / max(1, total_samples), val_acc, val_f1, all_predictions, all_labels, all_probs

        except Exception as e:
            tqdm.write(f"Validation epoch failed: {str(e)}")
            self.resource_manager.aggressive_cleanup()
            return float('inf'), 0.0, 0.0, [], [], np.array([])



    def train_main_model(self, train_loader, val_loader, test_loader=None):
        """Main model training with comprehensive logging"""
        if not train_loader or len(train_loader.dataset) == 0:
            tqdm.write(f"Skipping {self.model_name}: No training data")
            return None, None

        if not val_loader or len(val_loader.dataset) == 0:
            tqdm.write(f"Skipping {self.model_name}: No validation data")
            return None, None

        tqdm.write(f"\nTraining {self.model_name}")
        tqdm.write(f"Training samples: {len(train_loader.dataset):,}")
        tqdm.write(f"Validation samples: {len(val_loader.dataset):,}")
        tqdm.write(f"Total epochs: {Config.EPOCHS}")
        tqdm.write(f"Batch size: {train_loader.batch_size}")

        # Print hyperparameters
        tqdm.write("Hyperparameters:")
        for key, value in self.hyperparameters.items():
            if key in ['lr', 'weight_decay', 'dropout', 'hidden_dim_multiplier', 'label_smoothing']:
                tqdm.write(f"  {key}: {value:.4f}")
            else:
                tqdm.write(f"  {key}: {value}")

        # Setup model for training
        self.model = self.model.to(Config.DEVICE, memory_format=torch.channels_last)

        # Add this line after model creation:
        # self.model = torch.compile(self.model, mode="max-autotune")

        # torch.backends.cudnn.benchmark = True
        # torch.backends.cudnn.deterministic = False


        # Progress tracker
        progress_tracker = TrainingProgressTracker(self.model_name, Config.EPOCHS, len(train_loader))

        # Training loop
        training_start_time = time.time()

        # # Before loop
        # single_models = {}
        # single_results = {}

        for epoch in range(Config.EPOCHS):
            epoch_start_time = time.time()
            tqdm.write(f"\nEpoch {epoch + 1}/{Config.EPOCHS}")

            progress_tracker.start_epoch(epoch)

            # Training phase
            train_loss, train_acc = self.train_epoch(train_loader, progress_tracker)

            # Validation phase
            val_loss, val_acc, val_f1, predictions, labels, probs = self.validate_epoch(val_loader, progress_tracker)

            # Learning rate scheduling
            if isinstance(self.scheduler, optim.lr_scheduler.ReduceLROnPlateau):
                self.scheduler.step(val_loss)
            else:
                self.scheduler.step()

            # Progress tracking
            is_best = val_f1 > self.best_val_f1 * 1.001
            current_lr = self.optimizer.param_groups[0]['lr']

            progress_tracker.finish_epoch(train_loss, train_acc, val_loss, val_acc, val_f1, is_best=is_best, lr=current_lr)

            # Store history
            self.history['train_loss'].append(train_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_loss'].append(val_loss)
            self.history['val_acc'].append(val_acc)
            self.history['val_f1'].append(val_f1)
            self.history['learning_rates'].append(current_lr)

            # Early stopping and best model saving
            # if is_best:
            #     self.best_val_f1 = val_f1
            #     self.best_val_acc = val_acc
            #     self.patience_counter = 0
            #     # torch.save(f"{Config.OUTPUT_DIR}/best_model/{self.model_name}_best.pt")
            #     # Save complete model with architecture and hyperparameters
            #     torch.save({
            #         'state_dict': self.model.state_dict(),
            #         'hyperparameters': self.hyperparameters,
            #         'model_name': self.model_name
            #     }, f"{Config.OUTPUT_DIR}/best_model/{self.model_name}_best.pt")

            #     tqdm.write(f"New best model saved: F1={val_f1:.4f}, Acc={val_acc:.4f}")
            # else:
            #     self.patience_counter += 1
            # Early stopping and best model saving
            if is_best:
                self.best_val_f1 = val_f1
                self.best_val_acc = val_acc
                self.patience_counter = 0

                # Create comprehensive checkpoint
                checkpoint = {
                    # Model architecture and weights
                    'state_dict': self.model.state_dict(),
                    'model_name': self.model_name,
                    'hyperparameters': self.hyperparameters,

                    # Training metadata
                    'epoch': epoch + 1,
                    'best_val_f1': val_f1,
                    'best_val_acc': val_acc,
                    'optimizer_state': self.optimizer.state_dict(),
                    'scheduler_state': self.scheduler.state_dict() if self.scheduler else None,

                    # Architecture info for reconstruction
                    'num_classes': Config.NUM_CLASSES,
                    'dropout_rate': self.hyperparameters.get('dropout', 0.5),
                    'hidden_dim_multiplier': self.hyperparameters.get('hidden_dim_multiplier', 0.5),

                    # Save format version for future compatibility
                    'save_format_version': '1.0'
                }

                # Save with error handling
                try:
                    torch.save(checkpoint, f"{Config.OUTPUT_DIR}/best_model/{self.model_name}_best.pt")
                    tqdm.write(f"✅ Best model saved: F1={val_f1:.4f}, Acc={val_acc:.4f}")
                except Exception as e:
                    tqdm.write(f"❌ Error saving model {self.model_name}: {e}")
            else:
                self.patience_counter += 1




            # Epoch summary
            epoch_time = time.time() - epoch_start_time
            tqdm.write(f"\nEpoch {epoch + 1} Summary:")
            tqdm.write(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
            tqdm.write(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")
            tqdm.write(f"  Learning Rate: {current_lr:.6f}")
            tqdm.write(f"  Epoch Time: {epoch_time:.1f}s")
            tqdm.write(f"  Best F1 so far: {self.best_val_f1:.4f}")

            # Early stopping check
            if self.patience_counter >= Config.PATIENCE:
                total_time = time.time() - training_start_time
                tqdm.write(f"Early stopping at epoch {epoch + 1}")
                tqdm.write(f"Total training time: {total_time:.1f}s")
                break

        # # Load best model for evaluation
        # best_model_path = f"{Config.OUTPUT_DIR}/best_model/{self.model_name}_best.pt"
        # if os.path.exists(best_model_path):
        #     # self.model.load_state_dict(torch.load(best_model_path, map_location=Config.DEVICE))

        #     tqdm.write(f"Loaded best model from {best_model_path}")
        # else:
        #     tqdm.write("Warning: Best model not found, using current weights")
        # Load best model for evaluation with robust error handling
        best_model_path = f"{Config.OUTPUT_DIR}/best_model/{self.model_name}_best.pt"
        if os.path.exists(best_model_path):
            try:
                checkpoint = torch.load(best_model_path, map_location=Config.DEVICE, weights_only=False)

                if isinstance(checkpoint, dict) and 'state_dict' in checkpoint:
                    # New comprehensive format
                    self.model.load_state_dict(checkpoint['state_dict'])
                    tqdm.write(f"✅ Loaded best model (v{checkpoint.get('save_format_version', '1.0')})")
                else:
                    # Legacy format or direct state_dict
                    self.model.load_state_dict(checkpoint)
                    tqdm.write(f"✅ Loaded best model (legacy format)")

            except Exception as e:
                tqdm.write(f"⚠️ Could not load best model: {e}")
                tqdm.write(f"Continuing with current weights...")
        else:
            tqdm.write(f"⚠️ Best model file not found, using current weights")





        # Final evaluation
        evaluator = ModelEvaluator()
        eval_loader = test_loader if test_loader else val_loader

        with torch.no_grad():
            result = evaluator.evaluate_model(self.model, eval_loader, self.model_name)

        # Save results
        # result_to_save = {
        #     'history': {k: torch.tensor(v, dtype=torch.float32) for k, v in self.history.items()},
        #     'result': {
        #         'accuracy': torch.tensor(result.get('accuracy', 0.0), dtype=torch.float32),
        #         'f1': torch.tensor(result.get('f1_macro', 0.0), dtype=torch.float32),
        #         'conf_matrix': torch.tensor(result.get('conf_matrix', np.array([[]])), dtype=torch.int64),
        #         'true_labels': torch.tensor(result.get('true_labels', []), dtype=torch.int64),
        #         'predictions': torch.tensor(result.get('predictions', []), dtype=torch.int64),
        #         'probabilities': torch.tensor(result.get('probabilities', np.zeros((0, Config.NUM_CLASSES))), dtype=torch.float32),
        #         'misclassified': [{'image': item['image'].cpu(), 'true_label': item['true_label'], 'pred_label': item['pred_label']}
        #                         for item in result.get('misclassified', [])] if result.get('misclassified') else []
        #     }
        # }

        result_to_save = {
            'history': {k: torch.as_tensor(v, dtype=torch.float32).detach().cpu() for k, v in self.history.items()},
            'result': {
                'accuracy': torch.as_tensor(result.get('accuracy', 0.0), dtype=torch.float32).detach().cpu(),
                'f1': torch.as_tensor(result.get('f1_macro', 0.0), dtype=torch.float32).detach().cpu(),
                'conf_matrix': torch.as_tensor(result.get('conf_matrix', np.array([[]])), dtype=torch.int64).detach().cpu(),
                'true_labels': torch.as_tensor(result.get('true_labels', []), dtype=torch.int64).detach().cpu(),
                'predictions': torch.as_tensor(result.get('predictions', []), dtype=torch.int64).detach().cpu(),
                # 'probabilities': torch.as_tensor(result.get('probabilities', np.zeros((0, Config.NUM_CLASSES))), dtype=torch.float32).detach().cpu(),
                'probabilities': torch.as_tensor(
                    result.get('probabilities') if result.get('probabilities') is not None else np.zeros((0, Config.NUM_CLASSES)),
                    dtype=torch.float32
                ).detach().cpu(),
                # 'probabilities': torch.as_tensor(result.get('probabilities') if result.get('probabilities') is not None else np.zeros((0, Config.NUM_CLASSES)), dtype=torch.float32).detach().cpu(),
                'misclassified': [
                    {'image': item['image'].detach().cpu(),
                    'true_label': item['true_label'],
                    'pred_label': item['pred_label']}
                    for item in result.get('misclassified', [])
                ] if result.get('misclassified') else []
            }
        }



        main_result_path = f"{Config.OUTPUT_DIR}/model_results/{self.model_name}_main_results.pt"
        torch.save(result_to_save, main_result_path)
        tqdm.write(f"Results saved to {main_result_path}")

        # Training summary
        total_training_time = time.time() - training_start_time
        tqdm.write(f"\nTraining Summary for {self.model_name}:")
        tqdm.write(f"  Final Accuracy: {result.get('accuracy', 0.0):.4f}")
        tqdm.write(f"  Final F1 Score: {result.get('f1_macro', 0.0):.4f}")
        tqdm.write(f"  Best Validation F1: {self.best_val_f1:.4f}")
        tqdm.write(f"  Total Training Time: {total_training_time:.1f}s")
        tqdm.write(f"  Epochs Completed: {epoch + 1}/{Config.EPOCHS}")

        # Cleanup
        del result_to_save, result, evaluator, eval_loader
        self.resource_manager.aggressive_cleanup()

        # return self.history, None
        # return None, None  # Don't return data since it's saved and also change calling function from main




    def train_kfold(self, train_loader, val_loader, test_loader, n_folds=3):
        """K-fold cross-validation with detailed logging"""
        if n_folds <= 0:
            tqdm.write(f"Skipping k-fold for {self.model_name}: n_folds <= 0")
            return []


        from torch.utils.data import ConcatDataset
        combined_dataset = ConcatDataset([
            train_loader.dataset,
            val_loader.dataset
            # test_loader.dataset if test_loader is not None else []
        ])
        dataset = combined_dataset
        test_loader_fold=test_loader

        # dataset = train_loader.dataset
        total_samples = len(dataset)
        min_samples_per_fold = 500

        # Adjust folds based on data availability
        if total_samples < n_folds * min_samples_per_fold:
            n_folds = max(1, total_samples // min_samples_per_fold)
            tqdm.write(f"Adjusted to {n_folds} folds due to insufficient data")

        if n_folds < 2:
            tqdm.write(f"Skipping k-fold: need at least {min_samples_per_fold*2} samples, have {total_samples}")
            return []

        # Calculate fold statistics
        samples_per_fold = total_samples // n_folds
        train_samples_per_fold = total_samples - samples_per_fold

        tqdm.write(f"\nK-fold Cross-Validation Setup for {self.model_name}:")
        tqdm.write(f"  Total samples: {total_samples:,}")
        tqdm.write(f"  Number of folds: {n_folds}")
        tqdm.write(f"  Samples per fold (validation): {samples_per_fold:,}")
        tqdm.write(f"  Samples per fold (training): {train_samples_per_fold:,}")

        # Create fold indices
        fold_indices = []
        for i in range(n_folds):
            val_start = i * samples_per_fold
            val_end = min(val_start + samples_per_fold, total_samples)
            val_idx = list(range(val_start, val_end))
            train_idx = list(range(0, val_start)) + list(range(val_end, total_samples))
            fold_indices.append((train_idx, val_idx))

            tqdm.write(f"  Fold {i+1}: Train={len(train_idx):,}, Val={len(val_idx):,}")

        # Optimize settings for k-fold
        # fold_batch_size = self.resource_manager.optimize_batch_size(64, 1.0)
        # Use the ResourceManager method directly
        base_batch_size = self.hyperparameters.get('batch_size', Config.BATCH_SIZE)
        # Use the same complexity calculation as in main()
        model_complexity_map = {
            'efficientnet': 1.5,
            'resnet': 1.0,
            'vgg': 0.8,
            'mobilenet': 0.6,
            'densenet': 1.3,
            'convnext': 1.4
        }
        model_complexity = model_complexity_map.get(self.model_name.split('_')[0].lower(), 1.0)
        fold_batch_size = self.resource_manager.optimize_batch_size(base_batch_size, model_complexity)
        # fold_batch_size = self.resource_manager.optimize_batch_size(base_batch_size, model_name=self.model_name)

        prefetch_factor = min(4, max(2, self.resource_manager.cpu_memory_gb // 10))
        num_workers = min(8, mp.cpu_count() // 2)

        tqdm.write(f"K-fold optimized settings:")
        tqdm.write(f"  Batch size: {fold_batch_size}")
        tqdm.write(f"  Workers: {num_workers}")
        tqdm.write(f"  Prefetch factor: {prefetch_factor}")

        fold_results = []
        total_kfold_start = time.time()

        for fold, (train_idx, val_idx) in enumerate(fold_indices, 1):
            fold_start_time = time.time()
            tqdm.write(f"\nTraining Fold {fold}/{n_folds}")

            try:
                # Create fold-specific data loaders
                train_subsampler = SubsetRandomSampler(train_idx)
                val_subsampler = SubsetRandomSampler(val_idx)

                train_loader_fold = DataLoader(
                    dataset,
                    batch_size=fold_batch_size,
                    sampler=train_subsampler,
                    num_workers=num_workers,
                    pin_memory=torch.cuda.is_available(),
                    prefetch_factor=prefetch_factor,
                    persistent_workers=num_workers > 0,
                    worker_init_fn=worker_init_fn if 'worker_init_fn' in globals() else None
                )

                val_loader_fold = DataLoader(
                    dataset,
                    batch_size=fold_batch_size,
                    sampler=val_subsampler,
                    num_workers=num_workers,
                    pin_memory=torch.cuda.is_available(),
                    prefetch_factor=prefetch_factor,
                    persistent_workers=num_workers > 0,
                    worker_init_fn=worker_init_fn if 'worker_init_fn' in globals() else None
                )

                # Create fold model
                fold_model = ModelFactory.create_model(
                    self.model_name,
                    num_classes=Config.NUM_CLASSES,
                    dropout_rate=self.hyperparameters.get('dropout', 0.5),
                    hidden_dim_multiplier=self.hyperparameters.get('hidden_dim_multiplier', 0.5)
                ).to(Config.DEVICE, memory_format=torch.channels_last)

                # Create fold trainer
                fold_trainer = EnhancedModelTrainer(fold_model, f"{self.model_name}_fold_{fold}", self.hyperparameters)

                # Train fold
                # fold_history, fold_result =
                fold_trainer.train_main_model(train_loader_fold, val_loader_fold, test_loader=None)
                fold_time = time.time() - fold_start_time

                # Save trained model
                fold_model_path = f"{Config.OUTPUT_DIR}/kfold_results/{self.model_name}_fold_{fold}_model.pt"
                torch.save(fold_model.state_dict(), fold_model_path)


                # Load the model back (optional but ensures evaluation uses saved model)
                fold_model.load_state_dict(torch.load(fold_model_path))
                fold_model.to(Config.DEVICE)

                # Evaluate the fold
                evaluator = ModelEvaluator()
                eval_loader = test_loader_fold  # or test_loader if you have it
                with torch.no_grad():
                    fold_result = evaluator.evaluate_model(fold_model, eval_loader, f"{self.model_name}_fold_{fold}")

                # # Handle fold results
                # if fold_history is None or fold_result is None:
                #     tqdm.write(f"Fold {fold} training failed")
                #     fold_result = {
                #         'accuracy': 0.0, 'f1_macro': 0.0, 'conf_matrix': np.array([[]]),
                #         'true_labels': [], 'predictions': [], 'probabilities': np.zeros((0, Config.NUM_CLASSES)),
                #         'misclassified': []
                #     }
                #     fold_history = {'val_loss': [], 'val_acc': []}


                # # Store results
                # fold_results.append({
                #     'history': {
                #         'val_loss': fold_history.get('val_loss', []),
                #         'val_acc': fold_history.get('val_acc', [])
                #     },
                #     'result': {
                #         'accuracy': torch.tensor(fold_result.get('accuracy', 0.0), dtype=torch.float32),
                #         'f1': torch.tensor(fold_result.get('f1_macro', 0.0), dtype=torch.float32),
                #         'conf_matrix': torch.tensor(fold_result.get('conf_matrix', np.array([[]])), dtype=torch.int64),
                #         'misclassified': fold_result.get('misclassified', [])
                #     }
                # })

                # Save fold results
                # fold_result_to_save = {
                #     'history': {
                #         'val_loss': torch.tensor(fold_history.get('val_loss', []), dtype=torch.float32),
                #         'val_acc': torch.tensor(fold_history.get('val_acc', []), dtype=torch.float32)
                #     },
                #     'result': {
                #         'accuracy': torch.tensor(fold_result.get('accuracy', 0.0), dtype=torch.float32),
                #         'f1': torch.tensor(fold_result.get('f1_macro', 0.0), dtype=torch.float32),
                #         'conf_matrix': torch.tensor(fold_result.get('conf_matrix', np.array([[]])), dtype=torch.int64),
                #         'misclassified': fold_result.get('misclassified', [])
                #     }
                # }
                # Save fold results
                fold_result_to_save = {
                    'history': {  # assuming train_main_model updates self.history inside fold_trainer
                        'val_loss': torch.tensor(fold_trainer.history.get('val_loss', []), dtype=torch.float32),
                        'val_acc': torch.tensor(fold_trainer.history.get('val_acc', []), dtype=torch.float32)
                    },
                    'result': {
                        'accuracy': torch.tensor(fold_result.get('accuracy', 0.0), dtype=torch.float32),
                        'f1': torch.tensor(fold_result.get('f1_macro', 0.0), dtype=torch.float32),
                        'conf_matrix': torch.tensor(fold_result.get('conf_matrix', np.array([[]])), dtype=torch.int64),
                        'misclassified': fold_result.get('misclassified', [])
                    }
                }



                fold_result_path = f"{Config.OUTPUT_DIR}/kfold_results/{self.model_name}_fold_{fold}_results.pt"
                torch.save(fold_result_to_save, fold_result_path)

                tqdm.write(f"Fold {fold} completed in {fold_time:.1f}s - "
                          f"Accuracy: {fold_result.get('accuracy', 0.0):.4f}, "
                          f"F1: {fold_result.get('f1_macro', 0.0):.4f}")

                # Cleanup fold
                del fold_trainer, fold_model #, fold_history, fold_result
                del train_loader_fold, val_loader_fold, train_subsampler, val_subsampler
                self.resource_manager.aggressive_cleanup()

            except Exception as e:
                tqdm.write(f"Error in fold {fold}: {str(e)}")
                import traceback
                traceback.print_exc()
                continue

        total_kfold_time = time.time() - total_kfold_start
        # successful_folds = len(fold_results)

        tqdm.write(f"\nK-fold Summary for {self.model_name}:")
        # tqdm.write(f"  Successful folds: {successful_folds}/{n_folds}")
        tqdm.write(f"  Total k-fold time: {total_kfold_time:.1f}s")
        # tqdm.write(f"  Average time per fold: {total_kfold_time/max(1,successful_folds):.1f}s")

        # return []
        # return fold_results



    def cleanup_trainer(self):
        """Complete cleanup of trainer resources"""
        try:
            if hasattr(self, 'model'):
                del self.model
            if hasattr(self, 'optimizer'):
                del self.optimizer
            if hasattr(self, 'scheduler'):
                del self.scheduler
            if hasattr(self, 'criterion'):
                del self.criterion
            if hasattr(self, 'scaler'):
                del self.scaler

            self.history.clear()
            self.resource_manager.aggressive_cleanup()

        except Exception as e:
            tqdm.write(f"Cleanup error: {e}")

# ✅ Step 13: Model Training

In [ ]:
# orig_numpy = torch.Tensor.numpy
# def patched_numpy(self, *args, **kwargs):
#     if self.is_cuda:
#         raise RuntimeError(f"Tried to call .numpy() on CUDA tensor at line ???. Move it to CPU first!")
#     return orig_numpy(self, *args, **kwargs)

# torch.Tensor.numpy = patched_numpy

class ResourceManager:
    """Smart resource management for optimal GPU/CPU utilization"""

    def __init__(self):
        self.gpu_memory_gb = 20
        self.cpu_memory_gb = 50
        self.max_gpu_usage = 0.85  # 85% of 20GB = 17GB
        self.max_cpu_usage = 0.90  # 80% of 50GB = 40GB

    def get_memory_stats(self):
        """Get current memory usage statistics"""
        stats = {'cpu_percent': psutil.virtual_memory().percent}

        if torch.cuda.is_available():
            stats['gpu_allocated_gb'] = torch.cuda.memory_allocated() / (1024**3)
            stats['gpu_reserved_gb'] = torch.cuda.memory_reserved() / (1024**3)
            stats['gpu_percent'] = (stats['gpu_reserved_gb'] / self.gpu_memory_gb) * 100
        else:
            stats.update({'gpu_allocated_gb': 0, 'gpu_reserved_gb': 0, 'gpu_percent': 0})

        return stats

    def should_cleanup_aggressive(self):
        """Check if aggressive cleanup is needed"""
        stats = self.get_memory_stats()
        return (stats['gpu_percent'] > 90 or stats['cpu_percent'] > 90)

    def aggressive_cleanup(self):
        """Perform comprehensive memory cleanup"""
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
        gc.collect()
        time.sleep(0.1)  # Brief pause for system cleanup

    def optimize_batch_size(self, base_size, model_complexity=1.0):
        """Calculate optimal batch size based on current memory state"""
        stats = self.get_memory_stats()

        # Reduce batch size if memory usage is high
        memory_factor = max(0.4, 1.0 - (stats['gpu_percent'] / 100))
        optimal_size = int(base_size * memory_factor / model_complexity)

        return max(32, min(256, optimal_size))  # Keep within reasonable bounds


def safe_gpu_convert(data):
    """Safely convert GPU tensors to CPU numpy arrays or lists"""
    if torch.is_tensor(data):
        if data.is_cuda:
            data = data.detach().cpu()
        if data.dim() == 0:
            return data.item()
        elif data.dtype == torch.bool or data.dtype == torch.uint8:
            return data.numpy()
        else:
            return data.numpy() if data.dim() > 0 else data.item()
    elif isinstance(data, list):
        return [safe_gpu_convert(item) for item in data]
    elif isinstance(data, dict):
        return {k: safe_gpu_convert(v) for k, v in data.items()}
    elif isinstance(data, np.ndarray):
        return data  # Already numpy
    else:
        return data


def main():
    print("\nStarting Fish Species Model Training...")
    print("="*70)

    # Environment setup for maximum performance
    setup_environment()


    # Set optimal thread count for CPU utilization
    print(f"CPU : {os.cpu_count()}")
    print(f"USER: {os.getenv('USER')}")
    print(f"HOME: {os.getenv('HOME')}")
    torch.set_num_threads(min(16, os.cpu_count()))
    os.environ['OMP_NUM_THREADS'] = str(min(16, os.cpu_count()))

    # OpenMP → Open Multi-Processing.
    # OpenMP is a widely used API for parallel programming on CPUs.
    # Many scientific libraries (like NumPy, PyTorch, TensorFlow,
    # -OpenCV, Scikit-learn) rely on OpenMP to run operations in parallel across CPU cores.
    # In main() after line 720:
    torch.backends.cudnn.benchmark = True
    # Auto-benchmarks and selects fastest cuDNN algorithms for your specific hardware and input sizes
    # Takes ~5-10 seconds at start but can provide 10-20% speedup throughout training
    torch.backends.cudnn.deterministic = False
    # Allows cuDNN to use fastest available algorithms even if they produce slightly different results each run
    # Trades reproducibility for speed - essential for maximum performance in production training
    # torch.backends.cuda.matmul.allow_tf32 = True
    # Uses TensorFloat-32 (19-bit precision) instead of full Float-32 for matrix multiplications
    # Provides ~1.6x speedup on Ampere GPUs with negligible accuracy loss for deep learning
    # torch.backends.cudnn.allow_tf32 = True
    # Enables TF32 precision for cuDNN operations (convolutions, pooling, batch norm)
    # Accelerates CNN layers by 1.3-1.6x on RTX 30/40 series and A100+ GPUs
    torch.backends.cuda.enable_flash_sdp(True)
    # Activates Flash Attention algorithm for memory-efficient attention computations
    # Reduces GPU memory usage by 2-4x and increases attention speed, crucial for transformer layers

    # GPU optimizations
    if torch.cuda.is_available():
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
        torch.backends.cuda.matmul.allow_tf32 = True

        gpu_props = torch.cuda.get_device_properties(0)
        print(f"GPU: {gpu_props.name}, Memory: {gpu_props.total_memory / 1024**3:.1f}GB")

    # CPU info
    print(f"CPU Cores: {os.cpu_count()}, Using threads: {torch.get_num_threads()}")

    # Ensure all output directories exist
    os.makedirs(f"{Config.OUTPUT_DIR}/models", exist_ok=True)
    os.makedirs(f"{Config.OUTPUT_DIR}/kfold_results", exist_ok=True)
    os.makedirs(f"{Config.OUTPUT_DIR}/model_results", exist_ok=True)
    os.makedirs(f"{Config.OUTPUT_DIR}/best_model", exist_ok=True)

    # Initialize resource manager
    resource_manager = ResourceManager()

    # Load and balance data (only once)
    print("\nLoading and balancing data...")
    X, Y = DataManager.load_and_balance_data()
    print(f"Total samples after balancing: {len(X):,}, Labels: {len(Y):,}")

    # Validate data consistency
    if len(X) != len(Y):
        raise ValueError(f"Inconsistent data: X has {len(X)} samples, Y has {len(Y)} labels")
    if len(X) == 0:
        raise ValueError("No data available after loading and balancing")

    # Load all best parameters from hyperparameter optimization
    master_params_file = f"{Config.OUTPUT_DIR}/hyperparameters/all_best_params.json"
    if os.path.exists(master_params_file):
        with open(master_params_file, 'r') as f:
            all_best_params = json.load(f)
        print(f"Loaded best parameters for {len(all_best_params)} models")
    else:
        print("No hyperparameters found, using default parameters")
        all_best_params = {}

    # Process each model individually
    for model_name in Config.MODELS:
        print(f"\n{'='*70}")
        print(f"TRAINING MODEL: {model_name}")
        print(f"{'='*70}")

        # Pre-model memory state
        pre_stats = resource_manager.get_memory_stats()
        print(f"Pre-training memory - GPU: {pre_stats['gpu_allocated_gb']:.2f}GB ({pre_stats['gpu_percent']:.1f}%), "
              f"CPU: {pre_stats['cpu_percent']:.1f}%")

        try:
            # Load best parameters for this model
            if model_name in all_best_params:
                best_params = all_best_params[model_name]
                print(f"Using optimized parameters for {model_name}")
            else:
                # Load individual parameter file as fallback
                params_file = f"{Config.OUTPUT_DIR}/hyperparameters/{model_name}_best_params.json"
                if os.path.exists(params_file):
                    with open(params_file, 'r') as f:
                        best_params = json.load(f)
                    print(f"Loaded individual parameters for {model_name}")
                else:
                    # Default parameters as last resort
                    best_params = {
                        'lr': 0.001,
                        'weight_decay': 0.0001,
                        'dropout': 0.5,
                        'batch_size': 32,
                        'hidden_dim_multiplier': 0.5,
                        'augmentation_strength': 'medium'
                    }
                    print(f"Using default parameters for {model_name}")

            # Display parameters being used
            print(f"\n{model_name.upper()} TRAINING PARAMETERS:")
            for key, value in best_params.items():
                if key in ['lr', 'weight_decay', 'dropout', 'hidden_dim_multiplier', 'label_smoothing']:
                    print(f"  {key}: {value:.4f}")
                else:
                    print(f"  {key}: {value}")

            # Step 1: Create data loaders with optimized parameters
            print(f"\n1. CREATING DATA LOADERS WITH OPTIMIZED PARAMETERS FOR {model_name}")
            print("-"*50)

            # Calculate optimal batch size based on model complexity
            model_complexity_map = {
                'efficientnet_': 1.5,  # matches efficientnet_b0, efficientnet_b7 etc.
                'resnet': 1.0,         # matches resnet18, resnet50 etc.
                'vgg': 0.8,            # matches vgg16, vgg19 etc.
                'mobilenet_': 0.6,     # matches mobilenet_v2, mobilenet_v3 etc.
                'densenet': 1.3,       # matches densenet121, densenet201 etc.
                'convnext_': 1.4       # matches convnext_tiny, convnext_base etc.
            }

            model_complexity = model_complexity_map.get(model_name.split('_')[0].lower(), 1.0) #model_name gets from for loop.
            #"EfficientNet_B0".split('_')[0].lower() → "efficientnet"
            base_batch_size = best_params.get('batch_size', Config.BATCH_SIZE)
            optimized_batch_size = resource_manager.optimize_batch_size(base_batch_size, model_complexity)

            # Create with optimized batch size and GPU-optimized settings
            train_loader, val_loader, test_loader, val_data, test_data = DataManager.create_data_loaders(
                X, Y,
                test_size=0.2,
                batch_size=optimized_batch_size,
                augmentation_strength=best_params.get('augmentation_strength', 'medium')
            )

            # Optimize data loaders for GPU
            optimal_workers = min(16, max(8, os.cpu_count() // 2))
            for loader in [train_loader, val_loader, test_loader]:
                if loader:
                    loader.pin_memory = torch.cuda.is_available()
                    loader.num_workers = optimal_workers
                    loader.prefetch_factor = 4 if optimal_workers > 0 else 2

            # Validate data loaders
            print(f"Train: {len(train_loader.dataset)}, Val: {len(val_loader.dataset)}, Test: {len(test_loader.dataset)}")
            print(f"Optimized batch size: {optimized_batch_size} (complexity factor: {model_complexity})")

            if not train_loader or len(train_loader.dataset) == 0:
                print(f"Skipping {model_name}: No training data available")
                continue #To Go Next Model
            if not val_loader or len(val_loader.dataset) == 0:
                print(f"Skipping {model_name}: No validation data available")
                continue

            # Additional validation for val_data and test_data tuples
            if val_data is not None and len(val_data) == 2:
                print(f"Val data tuple: X={len(val_data[0])}, Y={len(val_data[1])}")
                if len(val_data[0]) != len(val_data[1]):
                    print(f"WARNING: Validation data inconsistency: {len(val_data[0])} samples vs {len(val_data[1])} labels")

            if test_data is not None and len(test_data) == 2:
                print(f"Test data tuple: X={len(test_data[0])}, Y={len(test_data[1])}")
                if len(test_data[0]) != len(test_data[1]):
                    print(f"WARNING: Test data inconsistency: {len(test_data[0])} samples vs {len(test_data[1])} labels")

            # Step 2: Train main model
            print(f"\n2. MAIN MODEL TRAINING FOR {model_name}")
            print("-"*50)

            model = ModelFactory.create_model(
                model_name,
                num_classes=Config.NUM_CLASSES,
                dropout_rate=best_params.get('dropout', 0.5),
                hidden_dim_multiplier=best_params.get('hidden_dim_multiplier', 0.5)
            ).to(Config.DEVICE, memory_format=torch.channels_last)  # GPU optimization

            # Model info
            total_model_params = sum(p.numel() for p in model.parameters())
            trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
            print(f"Model created: {total_model_params:,} total params, {trainable_params:,} trainable")

            # Additional GPU optimizations
            torch.backends.cudnn.benchmark = True
            torch.backends.cudnn.deterministic = False
            # Tells cuDNN (NVIDIA's deep learning library) to find the fastest algorithm for your hardware and input size.
            # Benchmarking finds the fastest algorithm.
            # Non-determinism allows cuDNN to use even faster (but slightly variable) methods.
            # Together → maximum training speed, but with non-reproducible results (F1 score may vary slightly between runs).
            # For speed → benchmark=True, deterministic=False (your case).
            # For reproducibility → benchmark=False, deterministic=True.

            trainer = EnhancedModelTrainer(model, model_name, best_params)
            # history, result = trainer.train_main_model(train_loader, val_loader, test_loader)
            trainer.train_main_model(train_loader, val_loader, test_loader)

            # if history is None or result is None: #No Need cause Data is saved to Disk Directly
            #     print(f"Training failed for {model_name}, skipping...")
            #     continue

            print(f"Main model training completed for {model_name}")

            # Step 3: K-fold cross-validation
            print(f"\n3. K-FOLD CROSS-VALIDATION FOR {model_name}")
            print("-"*50)

            # Ensure we have sufficient data for k-fold
            total_samples = len(train_loader.dataset)
            min_samples_per_fold = 500
            max_folds = total_samples // min_samples_per_fold
            n_folds = min(3, max_folds) if max_folds > 1 else 0

            if n_folds > 1:
                print(f"Performing {n_folds}-fold cross-validation...")
                # fold_results =
                trainer.train_kfold(train_loader, val_loader, test_loader,n_folds=n_folds)
                print(f"K-fold validation completed for {model_name}")
            else:
                print(f"Skipping k-fold validation for {model_name}: insufficient data (need >{min_samples_per_fold*2} samples)")
                fold_results = []

            # Step 4: Save model in both .pt and .keras formats
            print(f"\n4. SAVING MODEL FILES FOR {model_name}")
            print("-"*50)

            # Save model for ensemble
            model_state_dict = model.state_dict()
            torch.save(model_state_dict, f"{Config.OUTPUT_DIR}/models/{model_name}_for_ensemble.pt")
            print(f"Model saved for ensemble: {model_name}")

            # Save model as .pt file (PyTorch format)
            torch.save({
                'model_state_dict': model.state_dict(),
                'model_name': model_name,
                'num_classes': Config.NUM_CLASSES,
                'hyperparameters': best_params,
                'architecture_info': {
                    'dropout_rate': best_params.get('dropout', 0.5),
                    'hidden_dim_multiplier': best_params.get('hidden_dim_multiplier', 0.5)
                }
            }, f"{Config.OUTPUT_DIR}/best_model/{model_name}_best.pt")
            print(f"Model saved as .pt file: {model_name}_best.pt")

            # Save model as .keras file (if TensorFlow/Keras conversion is available)
            try:
                # This would require conversion logic if needed
                # For now, we'll save a placeholder or skip
                print(f"Keras format saving for {model_name} - conversion logic needed")
            except Exception as e:
                print(f"Could not save {model_name} in Keras format: {e}")

            # Step 5: Memory cleanup for this model
            print(f"\n5. MEMORY CLEANUP FOR {model_name}")
            print("-"*50)

            # Clear all variables specific to this model
            trainer.cleanup_trainer()
            # del trainer, history, result, model_state_dict
            del train_loader, val_loader, test_loader
            if 'model' in locals():
                del model
            if 'model_state_dict' in locals():
                del model_state_dict

            # Force garbage collection and GPU cleanup
            resource_manager.aggressive_cleanup()

            # Post-model memory state
            post_stats = resource_manager.get_memory_stats()
            memory_freed = pre_stats['gpu_allocated_gb'] - post_stats['gpu_allocated_gb']

            print(f"Memory cleanup completed for {model_name}:")
            print(f"  GPU memory freed: {memory_freed:.2f}GB")
            print(f"  Current GPU usage: {post_stats['gpu_percent']:.1f}%")
            print(f"  Current CPU usage: {post_stats['cpu_percent']:.1f}%")

            print(f"✓ {model_name} TRAINING COMPLETED!")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"Error processing {model_name}: {e}")
            import traceback
            traceback.print_exc()

            # Emergency cleanup
            try:
                if 'trainer' in locals():
                    trainer.cleanup_trainer()
                    del trainer
                if 'model' in locals():
                    del model
                resource_manager.aggressive_cleanup()
            except:
                pass
            continue

    # Final cleanup
    print("\nFinal cleanup and summary...")
    try:
        # Clean up all remaining variables
        if 'X' in locals():
            del X
        if 'Y' in locals():
            del Y
        if 'all_best_params' in locals():
            del all_best_params

        resource_manager.aggressive_cleanup()

        # Final memory stats
        final_stats = resource_manager.get_memory_stats()
        print(f"Final GPU memory: {final_stats['gpu_allocated_gb']:.2f}GB allocated ({final_stats['gpu_percent']:.1f}%)")
        print(f"Final CPU usage: {final_stats['cpu_percent']:.1f}%")

    except Exception as e:
        print(f"Error in final cleanup: {e}")

    print("\n" + "="*70)
    print("MODEL TRAINING COMPLETED!")
    print("="*70)
    print("\nGenerated Files:")
    print(f"- Model checkpoints: {Config.OUTPUT_DIR}/models/")
    print(f"- Training results: {Config.OUTPUT_DIR}/model_results/")
    print(f"- K-fold results: {Config.OUTPUT_DIR}/kfold_results/")
    print(f"- Best models: {Config.OUTPUT_DIR}/best_model/")


# Environment setup and multiprocessing configuration
import torch.multiprocessing as mp
if __name__ == "__main__":
    main()

# ✅ Step 14: Visualization Function Call

In [ ]:
"""
15.Fish Species Classification Pipeline - Visualization Module
========================================================

This module handles all visualization generation using saved models and results.
It loads saved .pt model files and generates comprehensive visualizations.
"""

import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import gc
import time
import psutil
import warnings
warnings.filterwarnings('ignore')

# orig_numpy = torch.Tensor.numpy
# def patched_numpy(self, *args, **kwargs):
#     if self.is_cuda:
#         raise RuntimeError(f"Tried to call .numpy() on CUDA tensor at line ???. Move it to CPU first!")
#     return orig_numpy(self, *args, **kwargs)

# torch.Tensor.numpy = patched_numpy

class ResourceManager:
    """Smart resource management for optimal GPU/CPU utilization"""

    def __init__(self):
        self.gpu_memory_gb = 20
        self.cpu_memory_gb = 50
        self.max_gpu_usage = 0.85  # 85% of 20GB = 17GB
        self.max_cpu_usage = 0.90  # 80% of 50GB = 40GB

    def get_memory_stats(self):
        """Get current memory usage statistics"""
        stats = {'cpu_percent': psutil.virtual_memory().percent}

        if torch.cuda.is_available():
            stats['gpu_allocated_gb'] = torch.cuda.memory_allocated() / (1024**3)
            stats['gpu_reserved_gb'] = torch.cuda.memory_reserved() / (1024**3)
            stats['gpu_percent'] = (stats['gpu_reserved_gb'] / self.gpu_memory_gb) * 100
        else:
            stats.update({'gpu_allocated_gb': 0, 'gpu_reserved_gb': 0, 'gpu_percent': 0})

        return stats

    def should_cleanup_aggressive(self):
        """Check if aggressive cleanup is needed"""
        stats = self.get_memory_stats()
        return (stats['gpu_percent'] > 90 or stats['cpu_percent'] > 90)

    def aggressive_cleanup(self):
        """Perform comprehensive memory cleanup"""
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
        gc.collect()
        time.sleep(0.1)  # Brief pause for system cleanup

    def optimize_batch_size(self, base_size, model_complexity=1.0):
        """Calculate optimal batch size based on current memory state"""
        stats = self.get_memory_stats()

        # Reduce batch size if memory usage is high
        memory_factor = max(0.4, 1.0 - (stats['gpu_percent'] / 100))
        optimal_size = int(base_size * memory_factor / model_complexity)

        return max(32, min(256, optimal_size))  # Keep within reasonable bounds


def safe_gpu_convert(data):
    """Safely convert GPU tensors to CPU numpy arrays or lists"""
    if torch.is_tensor(data):
        if data.is_cuda:
            data = data.detach().cpu()
        if data.dim() == 0:
            return data.item()
        elif data.dtype == torch.bool or data.dtype == torch.uint8:
            return data.numpy()
        else:
            return data.numpy() if data.dim() > 0 else data.item()
    elif isinstance(data, list):
        return [safe_gpu_convert(item) for item in data]
    elif isinstance(data, dict):
        return {k: safe_gpu_convert(v) for k, v in data.items()}
    elif isinstance(data, np.ndarray):
        return data  # Already numpy
    else:
        return data


def load_model_safely(model_name, model_path):
    """Safely load model with multiple fallback strategies"""
    try:
        checkpoint = torch.load(model_path, map_location=Config.DEVICE, weights_only=False)

        if isinstance(checkpoint, dict) and 'state_dict' in checkpoint:
            # New comprehensive format - use saved architecture info
            model = ModelFactory.create_model(
                model_name,
                num_classes=checkpoint.get('num_classes', Config.NUM_CLASSES),
                dropout_rate=checkpoint.get('dropout_rate', 0.5),
                hidden_dim_multiplier=checkpoint.get('hidden_dim_multiplier', 0.5)
            )
            model.to(Config.DEVICE)
            model.load_state_dict(checkpoint['state_dict'])
            return model, f"new format v{checkpoint.get('save_format_version', '1.0')}"

        elif isinstance(checkpoint, dict):
            # Legacy format - try with hyperparameters
            hyperparams = checkpoint.get('hyperparameters', {})
            model = ModelFactory.create_model(
                model_name,
                num_classes=Config.NUM_CLASSES,
                dropout_rate=hyperparams.get('dropout', 0.5),
                hidden_dim_multiplier=hyperparams.get('hidden_dim_multiplier', 0.5)
            )
            model.to(Config.DEVICE)
            model.load_state_dict(checkpoint)
            return model, "legacy nested format"

        else:
            # Try direct loading with parameter combinations
            param_combinations = [
                (0.5, 0.5), (0.3, 0.5), (0.7, 0.5),
                (0.5, 0.3), (0.5, 0.7), (0.5, 1.0), (0.5, 1.5)
            ]

            for dropout, hidden_mult in param_combinations:
                try:
                    model = ModelFactory.create_model(
                        model_name,
                        num_classes=Config.NUM_CLASSES,
                        dropout_rate=dropout,
                        hidden_dim_multiplier=hidden_mult
                    )
                    model.to(Config.DEVICE)
                    model.load_state_dict(checkpoint)
                    return model, f"direct format (dropout={dropout}, hidden={hidden_mult})"
                except RuntimeError:
                    continue

            return None, "failed all combinations"

    except Exception as e:
        return None, f"loading error: {str(e)}"


def main():
    print("\nStarting Fish Species Visualization Generation...")
    print("="*70)

    # Environment setup for maximum performance
    setup_environment()


    # Set optimal thread count for CPU utilization
    print(f"CPU : {os.cpu_count()}")
    print(f"USER: {os.getenv('USER')}")
    print(f"HOME: {os.getenv('HOME')}")
    torch.set_num_threads(min(16, os.cpu_count()))
    os.environ['OMP_NUM_THREADS'] = str(min(16, os.cpu_count()))

    # OpenMP → Open Multi-Processing.
    # OpenMP is a widely used API for parallel programming on CPUs.
    # Many scientific libraries (like NumPy, PyTorch, TensorFlow,
    # -OpenCV, Scikit-learn) rely on OpenMP to run operations in parallel across CPU cores.
    # In main() after line 720:
    torch.backends.cudnn.benchmark = True
    # Auto-benchmarks and selects fastest cuDNN algorithms for your specific hardware and input sizes
    # Takes ~5-10 seconds at start but can provide 10-20% speedup throughout training
    torch.backends.cudnn.deterministic = False
    # Allows cuDNN to use fastest available algorithms even if they produce slightly different results each run
    # Trades reproducibility for speed - essential for maximum performance in production training
    # torch.backends.cuda.matmul.allow_tf32 = True
    # Uses TensorFloat-32 (19-bit precision) instead of full Float-32 for matrix multiplications
    # Provides ~1.6x speedup on Ampere GPUs with negligible accuracy loss for deep learning
    # torch.backends.cudnn.allow_tf32 = True
    # Enables TF32 precision for cuDNN operations (convolutions, pooling, batch norm)
    # Accelerates CNN layers by 1.3-1.6x on RTX 30/40 series and A100+ GPUs
    torch.backends.cuda.enable_flash_sdp(True)
    # Activates Flash Attention algorithm for memory-efficient attention computations
    # Reduces GPU memory usage by 2-4x and increases attention speed, crucial for transformer layers

    # GPU optimizations
    if torch.cuda.is_available():
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
        torch.backends.cuda.matmul.allow_tf32 = True

        gpu_props = torch.cuda.get_device_properties(0)
        print(f"GPU: {gpu_props.name}, Memory: {gpu_props.total_memory / 1024**3:.1f}GB")

    # CPU info
    print(f"CPU Cores: {os.cpu_count()}, Using threads: {torch.get_num_threads()}")

    # Ensure visualization output directory exists
    os.makedirs(f"{Config.OUTPUT_DIR}/visualizations", exist_ok=True)

    # Initialize resource manager
    resource_manager = ResourceManager()

    # Load and balance data for visualization (if needed for test data)
    print("\nLoading data for visualization...")
    X, Y = DataManager.load_and_balance_data()
    print(f"Total samples: {len(X):,}, Labels: {len(Y):,}")

    # Initialize components
    visualizer = EnhancedVisualizations()
    single_models = {}
    single_results = {}

    # Load trained models and results
    print(f"\n{'='*70}")
    print("LOADING TRAINED MODELS AND RESULTS")
    print(f"{'='*70}")

    # Load models with enhanced error handling
    for model_name in Config.MODELS:
        model_path = os.path.join(Config.OUTPUT_DIR, "best_model", f"{model_name}_best.pt")

        if not os.path.exists(model_path):
            print(f"⚠️  Model file not found: {model_name}")
            continue

        model, load_info = load_model_safely(model_name, model_path)

        if model is not None:
            model.eval()
            single_models[model_name] = model
            print(f"✅ Loaded {model_name} ({load_info})")

            # Load evaluation results for visualization compatibility
            result_path = f"{Config.OUTPUT_DIR}/model_results/{model_name}_main_results.pt"
            if os.path.exists(result_path):
                try:
                    saved_results = torch.load(result_path, map_location='cpu', weights_only=False)
                    result_data = saved_results.get('result', {})

                    single_results[model_name] = {
                        'model_name': model_name,
                        'accuracy': float(result_data.get('accuracy', 0.0)),
                        'f1_macro': float(result_data.get('f1', 0.0)),
                        'f1_weighted': float(result_data.get('f1', 0.0)),  # Add f1_weighted key
                        'predictions': result_data.get('predictions', []),
                        'true_labels': result_data.get('true_labels', []),
                        'probabilities': result_data.get('probabilities', np.zeros((0, Config.NUM_CLASSES))),
                        'conf_matrix': result_data.get('conf_matrix', np.zeros((Config.NUM_CLASSES, Config.NUM_CLASSES))),
                        'misclassified': result_data.get('misclassified', [])
                    }

                    # Convert tensors to numpy for visualization compatibility
                    for key in ['predictions', 'true_labels', 'probabilities', 'conf_matrix']:
                        if torch.is_tensor(single_results[model_name][key]):
                            single_results[model_name][key] = single_results[model_name][key].numpy()

                    print(f"📊 Results loaded: Acc={single_results[model_name]['accuracy']:.4f}")

                except Exception as e:
                    print(f"⚠️  Could not load results for {model_name}: {e}")
                    single_results[model_name] = {'model_name': model_name, 'accuracy': 0.0, 'f1_macro': 0.0, 'f1_weighted': 0.0}
            else:
                print(f"⚠️  No results file for {model_name}")
                single_results[model_name] = {'model_name': model_name, 'accuracy': 0.0, 'f1_macro': 0.0, 'f1_weighted': 0.0}
        else:
            print(f"❌ Failed to load {model_name}: {load_info}")

    print(f"\n🎯 Loaded: {len(single_models)} models, {len(single_results)} result sets")

    # Create test data loader for visualizations that need it
    if len(single_models) > 0:
        print("\nCreating test data loader for visualizations...")
        _, _, test_loader, val_data, test_data = DataManager.create_data_loaders(X, Y, test_size=0.2)
        print(f"Test loader created with {len(test_loader.dataset)} samples")

    # Generate visualizations for each model
    for model_name in single_models.keys():
        print(f"\n{'='*70}")
        print(f"GENERATING VISUALIZATIONS FOR: {model_name}")
        print(f"{'='*70}")

        # Pre-visualization memory state
        pre_stats = resource_manager.get_memory_stats()
        print(f"Pre-visualization memory - GPU: {pre_stats['gpu_allocated_gb']:.2f}GB ({pre_stats['gpu_percent']:.1f}%), "
              f"CPU: {pre_stats['cpu_percent']:.1f}%")

        try:
            # Load saved results for plotting (ensuring consistency)
            main_result_path = f"{Config.OUTPUT_DIR}/model_results/{model_name}_main_results.pt"

            if os.path.exists(main_result_path):
                saved_data = torch.load(main_result_path, map_location=Config.DEVICE, weights_only=False)
                history_for_viz = saved_data['history']
                result_for_viz = saved_data['result']
                print(f"Using saved results for {model_name} visualization")
            else:
                result_for_viz = {}
                history_for_viz = {}
                def add_strikethrough(text):
                    return ''.join(c + '\u0336' for c in text)
                print(f"{add_strikethrough('Using current')} No results for {model_name} visualization")

            print(f"Result keys: {list(result_for_viz.keys()) if result_for_viz else 'None'}")

            # Convert all data to CPU/numpy for visualization (GPU-safe conversion)
            history_viz = safe_gpu_convert(history_for_viz)
            result_viz = safe_gpu_convert(result_for_viz)

            # Validate result data
            true_labels = result_viz.get('true_labels', np.array([]))
            predictions = result_viz.get('predictions', np.array([]))
            probabilities = result_viz.get('probabilities', np.array([]))

            # ADD THE CONVERSION LINES HERE:
            if torch.is_tensor(true_labels):
                true_labels = true_labels.detach().cpu().numpy()
            if torch.is_tensor(predictions):
                predictions = predictions.detach().cpu().numpy()
            if torch.is_tensor(probabilities):
                probabilities = probabilities.detach().cpu().numpy()
            # ADD THESE ADDITIONAL CONVERSIONS:
            # Convert history data to safe formats
            for key in ['train_loss', 'train_acc', 'val_loss', 'val_acc', 'val_f1']:
                if key in history_viz and torch.is_tensor(history_viz[key]):
                    history_viz[key] = history_viz[key].detach().cpu().numpy().tolist()

            print(f"Data validation for {model_name}:")
            print(f"  true_labels: {type(true_labels)}, shape: {getattr(true_labels, 'shape', len(true_labels) if hasattr(true_labels, '__len__') else 'scalar')}")
            print(f"  predictions: {type(predictions)}, shape: {getattr(predictions, 'shape', len(predictions) if hasattr(predictions, '__len__') else 'scalar')}")
            print(f"  probabilities: {type(probabilities)}, shape: {getattr(probabilities, 'shape', 'unknown')}")

            # Check if we have valid data for plotting
            if (true_labels.size > 0 and predictions.size > 0 and true_labels.shape[0] == predictions.shape[0]):
                plots_generated = 0

                # Training history plot
                train_loss = history_viz.get('train_loss')
                if train_loss is not None and hasattr(train_loss, '__len__') and len(train_loss) > 0:
                    try:
                        # Add this conversion:
                        if history_viz.get('train_loss') is not None:
                            if isinstance(history_viz['train_loss'], np.ndarray):
                                history_viz['train_loss'] = history_viz['train_loss'].tolist()
                        if history_viz.get('train_acc') is not None:
                            if isinstance(history_viz['train_acc'], np.ndarray):
                                history_viz['train_acc'] = history_viz['train_acc'].tolist()
                        if history_viz.get('val_loss') is not None:
                            if isinstance(history_viz['val_loss'], np.ndarray):
                                history_viz['val_loss'] = history_viz['val_loss'].tolist()
                        if history_viz.get('val_acc') is not None:
                            if isinstance(history_viz['val_acc'], np.ndarray):
                                history_viz['val_acc'] = history_viz['val_acc'].tolist()

                        visualizer.plot_single_model_history(history_viz, model_name)
                        plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_history.png", dpi=150, bbox_inches='tight')
                        plt.close('all')
                        print(f"Training history plot generated for {model_name}")
                        plots_generated += 1
                    except Exception as e:
                        print(f"Error plotting training history for {model_name}: {e}")

                try:
                    # ROC Curves
                    visualizer.plot_roc_curves(result_viz, model_name)
                    plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_roc.png", dpi=150, bbox_inches='tight')
                    plt.close('all')
                    print(f"ROC curves plot generated for {model_name}")
                    plots_generated += 1

                    # Confusion Matrix
                    visualizer.plot_confusion_matrix(result_viz, model_name)
                    plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_confusion.png", dpi=150, bbox_inches='tight')
                    plt.close('all')
                    print(f"Confusion matrix plot generated for {model_name}")
                    plots_generated += 1

                    # Misclassified Images
                    misclassified = result_viz.get('misclassified', [])
                    if misclassified is not None and len(misclassified) > 0:
                        # Convert any GPU tensors in misclassified images to CPU
                        for item in result_viz['misclassified']:
                            if isinstance(item, dict) and 'image' in item:
                                if torch.is_tensor(item['image']):
                                    item['image'] = item['image'].detach().cpu()
                                elif isinstance(item['image'], np.ndarray):
                                    # Optionally transpose if channel-first
                                    if item['image'].shape[0] in [1,3]:
                                        item['image'] = np.transpose(item['image'], (1, 2, 0))

                        visualizer.plot_misclassified_images(result_viz['misclassified'], model_name)
                        plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_misclassified.png", dpi=150, bbox_inches='tight')
                        plt.close('all')
                        print(f"Misclassified images plot generated for {model_name}")
                        plots_generated += 1
                    else:
                        print(f"No misclassified images to plot for {model_name}")

                    # XAI Visualization
                    if model_name in single_models and test_loader is not None and len(test_loader.dataset) > 0:
                        try:
                            model = single_models[model_name]
                            visualizer.plot_single_model_xai(model, model_name, test_loader)
                            plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_xai.png", dpi=150, bbox_inches='tight')
                            plt.close('all')
                            print(f"XAI visualization generated for {model_name}")
                            plots_generated += 1
                        except Exception as xai_error:
                            print(f"XAI visualization error for {model_name}: {xai_error}")
                    else:
                        print(f"Skipping XAI visualization for {model_name}: model or test_loader not available")

                except Exception as plot_error:
                    print(f"Visualization error for {model_name}: {plot_error}")
                    import traceback
                    traceback.print_exc()

                print(f"Generated {plots_generated} main visualization plots for {model_name}")

            else:
                print(f"Skipping main visualizations for {model_name}: data validation failed")
                print(f"  true_labels length: {true_labels.shape[0] if hasattr(true_labels, 'shape') else 'scalar'}")
                print(f"  predictions length: {predictions.shape[0] if hasattr(predictions, 'shape') else 'scalar'}")

            # K-fold results plotting (load from saved files)
            print(f"Loading and plotting k-fold results for {model_name}...")
            fold_results_loaded = []

            # Check for k-fold results
            for fold in range(1, 4):  # Assuming max 3 folds
                fold_result_path = f"{Config.OUTPUT_DIR}/kfold_results/{model_name}_fold_{fold}_results.pt"
                if os.path.exists(fold_result_path):
                    try:
                        fold_data = torch.load(fold_result_path, map_location=Config.DEVICE, weights_only=False)

                        # Convert GPU tensors to CPU/numpy
                        fold_converted = {
                            'history': safe_gpu_convert(fold_data['history']),
                            'result': safe_gpu_convert(fold_data['result'])
                        }
                        fold_results_loaded.append(fold_converted)

                    except Exception as e:
                        print(f"Error loading fold {fold} results: {e}")

            if fold_results_loaded is not None and len(fold_results_loaded) > 0:
                try:
                    visualizer.plot_kfold_results(fold_results_loaded, model_name, test_loader)
                    plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{model_name}_kfold.png", dpi=150, bbox_inches='tight')
                    plt.close('all')
                    print(f"K-fold results plot generated for {model_name}")
                except Exception as e:
                    print(f"Error plotting k-fold results for {model_name}: {e}")
                    import traceback
                    traceback.print_exc()
            else:
                print(f"No k-fold results to plot for {model_name}")

            print(f"All visualizations completed for {model_name}")

            # Memory cleanup for this model's visualizations
            if 'history_viz' in locals():
                del history_viz
            if 'result_viz' in locals():
                del result_viz
            if 'fold_results_loaded' in locals():
                del fold_results_loaded
            if 'history_for_viz' in locals():
                del history_for_viz
            if 'result_for_viz' in locals():
                del result_for_viz

            # Force garbage collection and GPU cleanup
            plt.close('all')
            resource_manager.aggressive_cleanup()

            # Post-visualization memory state
            post_stats = resource_manager.get_memory_stats()
            memory_freed = pre_stats['gpu_allocated_gb'] - post_stats['gpu_allocated_gb']

            print(f"Memory cleanup completed for {model_name}:")
            print(f"  GPU memory freed: {memory_freed:.2f}GB")
            print(f"  Current GPU usage: {post_stats['gpu_percent']:.1f}%")
            print(f"  Current CPU usage: {post_stats['cpu_percent']:.1f}%")

            print(f"✓ {model_name} VISUALIZATIONS COMPLETED!")
            print(f"{'='*70}\n")

        except Exception as e:
            print(f"Error in visualization process for {model_name}: {e}")
            import traceback
            traceback.print_exc()

    # Step: Ensemble processing and visualizations (only if we have trained models)
    if single_models and 'val_data' in locals():
        print("\n" + "="*70)
        print("GENERATING ENSEMBLE VISUALIZATIONS")
        print("="*70)

        try:
            if (val_data is not None and len(val_data) == 2 and len(val_data[0]) > 0 and len(val_data[1]) > 0 and len(val_data[0]) == len(val_data[1])):

                ensemble_manager = EnsembleManager(single_models, val_data)
                ensemble_results, best_ensemble = ensemble_manager.test_ensemble_combinations()

                # Generate visualizations for ensemble methods
                print("\nGenerating visualizations for ensemble methods...")
                for ensemble_name, ensemble_result in ensemble_results.items():
                    try:
                        # Convert GPU tensors to CPU for visualization
                        ensemble_result_viz = safe_gpu_convert(ensemble_result)

                        true_labels_ens = ensemble_result_viz.get('true_labels', [])
                        predictions_ens = ensemble_result_viz.get('predictions', [])

                        if (len(true_labels_ens) > 0 and
                            len(predictions_ens) > 0 and
                            len(true_labels_ens) == len(predictions_ens)):

                            visualizer.plot_roc_curves(ensemble_result_viz, ensemble_name)
                            plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{ensemble_name}_roc.png", dpi=150, bbox_inches='tight')
                            plt.close('all')

                            visualizer.plot_confusion_matrix(ensemble_result_viz, ensemble_name)
                            plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/{ensemble_name}_confusion.png", dpi=150, bbox_inches='tight')
                            plt.close('all')

                            # F1 per class visualization
                            from sklearn.metrics import f1_score
                            f1_per_class = f1_score(ensemble_result_viz['true_labels'],
                                                   ensemble_result_viz['predictions'], average=None)
                            fig, ax = plt.subplots(figsize=(10, 6))
                            ax.bar(range(Config.NUM_CLASSES), f1_per_class, color='lightgreen', alpha=0.8)
                            ax.set_xticks(range(Config.NUM_CLASSES))
                            ax.set_xticklabels(Config.CLASS_LABELS, rotation=45, ha='right')
                            ax.set_title(f'F1 Scores per Class - {ensemble_name}',
                                       fontsize=14, fontweight='bold', pad=15)
                            ax.set_xlabel('Class', fontsize=12, fontweight='bold')
                            ax.set_ylabel('F1 Score', fontsize=12, fontweight='bold')
                            ax.grid(True, alpha=0.3)
                            for i, v in enumerate(f1_per_class):
                                ax.text(i, v + 0.01, f'{v:.2f}', ha='center', va='bottom', fontsize=10)
                            save_path = f"{Config.OUTPUT_DIR}/visualizations/{ensemble_name}_f1_per_class.png"
                            plt.savefig(save_path, dpi=300, bbox_inches='tight',
                                      facecolor='white', edgecolor='none')
                            plt.close()
                            print(f"F1 scores per class saved: {save_path}")
                        else:
                            print(f"Skipping visualizations for {ensemble_name}: invalid data")
                    except Exception as e:
                        print(f"Error generating ensemble visualizations for {ensemble_name}: {e}")

                    # Clear memory
                    if 'ensemble_result_viz' in locals():
                        del ensemble_result_viz
                    resource_manager.aggressive_cleanup()

                # Comment out the problematic ensemble test evaluation
                print("Skipping ensemble test evaluation (method not implemented)")
                # if (best_ensemble and test_data is not None and len(test_data) == 2 and
                #     len(test_data[0]) == len(test_data[1]) and len(test_data[0]) > 0):
                #     print("Evaluating best ensemble on test set...")
                #     # Ensemble test evaluation code commented out

            else:
                print("Skipping ensemble analysis: no valid validation data")

        except Exception as e:
            print(f"Error in ensemble processing: {e}")
            import traceback
            traceback.print_exc()

    # Step: Generate comprehensive visualizations (only if we have results)
    if single_results:
        print("\n" + "="*70)
        print("GENERATING COMPREHENSIVE ANALYSIS VISUALIZATIONS")
        print("="*70)

        try:
            # Convert all results to CPU/numpy for final plotting
            single_results_viz = safe_gpu_convert(single_results)
            ensemble_results_viz = safe_gpu_convert(ensemble_results) if 'ensemble_results' in locals() else {}

            # Plot overall model comparison
            if single_results_viz and len(single_results_viz) > 0:
                visualizer.plot_model_comparison(single_results_viz,ensemble_results_viz)
                plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/model_comparison.png", dpi=300, bbox_inches='tight')
                plt.close('all')

            # Generate comprehensive report
            visualizer.generate_comprehensive_report(
                single_results_viz,
                ensemble_results_viz,
                best_ensemble if 'best_ensemble' in locals() else None
            )
            plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/comprehensive_report.png", dpi=300, bbox_inches='tight')
            plt.close('all')

            # Additional visualizations
            if 'test_loader' in locals() and test_loader is not None and len(test_loader.dataset) > 0:
                try:
                    visualizer.plot_comparative_xai(
                        single_models,
                        ensemble_results_viz,
                        test_loader,
                        max_images=2
                    )
                    plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/comparative_xai.png", dpi=300, bbox_inches='tight')
                    plt.close('all')

                    visualizer.plot_lrp_grid(single_models, test_loader)
                    plt.savefig(f"{Config.OUTPUT_DIR}/visualizations/lrp_grid.png", dpi=300, bbox_inches='tight')
                    plt.close('all')
                except Exception as e:
                    print(f"Error in additional visualizations: {e}")

            print("Comprehensive analysis completed successfully")

        except Exception as e:
            print(f"Error in comprehensive analysis: {e}")
            import traceback
            traceback.print_exc()

    # Final cleanup
    print("\nFinal cleanup and summary...")
    try:
        # Clean up all remaining variables
        if 'single_models' in locals():
            del single_models
        if 'test_loader' in locals():
            del test_loader
        if 'single_results' in locals():
            del single_results
        if 'single_results_viz' in locals():
            del single_results_viz
        if 'ensemble_results' in locals():
            del ensemble_results
        if 'ensemble_results_viz' in locals():
            del ensemble_results_viz
        if 'best_ensemble' in locals():
            del best_ensemble
        if 'visualizer' in locals():
            del visualizer
        if 'X' in locals():
            del X
        if 'Y' in locals():
            del Y

        resource_manager.aggressive_cleanup()

        # Final memory stats
        final_stats = resource_manager.get_memory_stats()
        print(f"Final GPU memory: {final_stats['gpu_allocated_gb']:.2f}GB allocated ({final_stats['gpu_percent']:.1f}%)")
        print(f"Final CPU usage: {final_stats['cpu_percent']:.1f}%")

    except Exception as e:
        print(f"Error in final cleanup: {e}")

    print("\n" + "="*70)
    print("VISUALIZATION GENERATION COMPLETED!")
    print("="*70)
    print("\nGenerated Files:")
    print(f"- All visualizations: {Config.OUTPUT_DIR}/visualizations/")


# Environment setup and multiprocessing configuration
import torch.multiprocessing as mp
if __name__ == "__main__":
    main()

# ✅ Step 15: Analysis on Real_World Data

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import requests
from io import BytesIO

# ---------------------------
# CONFIG
# ---------------------------
MODEL_PATH = "best_model.pth"   # তোমার trained model file
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# তোমার dataset এর class লিস্ট (নিজের dataset অনুযায়ী বদলাবে)
CLASS_NAMES = ["ilish", "chandana", "sardin", "sardinella", "punctatus"]

# ---------------------------
# Load Model
# ---------------------------
def load_model():
    model = models.resnet50(weights=None)   # তুমি যেটা ব্যবহার করেছো সেটা বসাও
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, len(CLASS_NAMES))

    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint["model_state_dict"])  # তোমার save format অনুযায়ী adjust করো
    model.to(DEVICE)
    model.eval()
    return model

# ---------------------------
# Preprocess
# ---------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),    # training সময় যা দিয়েছো, সেটা মেলাতে হবে
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

def load_image(img_path=None, img_url=None):
    if img_path:
        image = Image.open(img_path).convert("RGB")
    elif img_url:
        response = requests.get(img_url)
        image = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        raise ValueError("Provide either img_path or img_url")
    return image

# ---------------------------
# Prediction
# ---------------------------
def predict_image(model, image):
    img_tensor = transform(image).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        outputs = model(img_tensor)
        probs = torch.softmax(outputs, dim=1)
        conf, pred = torch.max(probs, 1)
    return CLASS_NAMES[pred.item()], conf.item()

# ---------------------------
# Main
# ---------------------------
if __name__ == "__main__":
    model = load_model()

    # Option 1: লোকাল ফাইল থেকে
    img_path = "test_fish.jpg"
    image = load_image(img_path=img_path)
    label, confidence = predict_image(model, image)
    print(f"Prediction: {label} ({confidence:.2f})")

    # Option 2: URL থেকে
    img_url = "https://example.com/sample_fish.jpg"
    image = load_image(img_url=img_url)
    label, confidence = predict_image(model, image)
    print(f"Prediction: {label} ({confidence:.2f})")


#End